In [ ]:

import duckdb

# Path to your DB file
DB_PATH = "../ProjectMain/db/data.duckdb"

con = duckdb.connect(DB_PATH)

# List tables
print("Tables:", con.execute("SHOW TABLES").fetchall())

# Look at the first few rows
df = con.execute("SELECT * FROM LoadForecast LIMIT 10").df()
print(df)

# Example: filter by date
df2 = con.execute("""
    SELECT *
    FROM LoadForecast
    WHERE DATE(OperatingDTM) = '2025-08-16'
    ORDER BY Interval
""").df()

print(df2)

con.close()


Tables: [('LoadForecast',), ('LoadForecastPivot',), ('Prices',), ('SessionID',), ('WeatherFcst',), ('WeatherHist',)]
Empty DataFrame
Columns: [timestamp, load]
Index: []


BinderException: Binder Error: Referenced column "OperatingDTM" not found in FROM clause!
Candidate bindings: "load"

LINE 4:     WHERE OperatingDTM = '2025-08-16'
                  ^

In [ ]:

import duckdb
con = duckdb.connect("../ProjectMain/db/data.duckdb")
print(con.execute("SHOW TABLES").fetchall())
print(con.execute("PRAGMA table_info('LoadForecast')").df())
print(con.execute("SELECT COUNT(*) FROM LoadForecast").fetchall())
print(con.execute("""
  SELECT OperatingDTM, Interval, Location, ForecastMW
  FROM LoadForecast
  ORDER BY OperatingDTM, Interval, Location
  LIMIT 10
""").df())
con.close()

[('LoadForecast',), ('LoadForecastPivot',), ('Prices',), ('SessionID',), ('WeatherFcst',), ('WeatherHist',)]
   cid       name       type  notnull dflt_value     pk
0    0  timestamp  TIMESTAMP    False       None  False
1    1       load     DOUBLE    False       None  False
[(0,)]


BinderException: Binder Error: Referenced column "OperatingDTM" not found in FROM clause!
Candidate bindings: "load"

LINE 2:   SELECT OperatingDTM, Interval, Location, ForecastMW
                 ^

In [1]:
import duckdb
con = duckdb.connect("../ProjectMain/db/data.duckdb")


print(con.execute("""
  DROP TABLE LoadForecast;
"""))

con.close()

In [ ]:
import duckdb
con = duckdb.connect("../ProjectMain/db/data.duckdb")

con.execute(f"""
            CREATE TABLE IF NOT EXISTS "LoadForecast" (
                OperatingDTM TIMESTAMP,
                Interval     INTEGER,
                Month        TEXT,
                LocationType TEXT,
                Location     TEXT,
                DSTFlag      TEXT,
                ForecastMW   DOUBLE
            );
        """)
con.close()

: 

In [20]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("SELECT * FROM historical_weather").fetchdf()

print(df)

con.close()


                timestamp  interval      lat      lon  temperature  humidity  \
0     2022-12-31 19:00:00        19  29.7604 -95.3698         22.0      47.0   
1     2022-12-31 20:00:00        20  29.7604 -95.3698         21.0      53.0   
2     2022-12-31 21:00:00        21  29.7604 -95.3698         19.0      60.0   
3     2022-12-31 22:00:00        22  29.7604 -95.3698         18.0      78.0   
4     2022-12-31 23:00:00        23  29.7604 -95.3698         17.0      88.0   
...                   ...       ...      ...      ...          ...       ...   
23013 2024-01-03 15:00:00        15  29.7604 -95.3698         13.0      67.0   
23014 2024-01-03 16:00:00        16  29.7604 -95.3698         13.0      67.0   
23015 2024-01-03 17:00:00        17  29.7604 -95.3698         13.0      67.0   
23016 2024-01-03 18:00:00        18  29.7604 -95.3698         13.0      67.0   
23017 2024-01-03 19:00:00        19  29.7604 -95.3698         13.0      67.0   

       windspeed  precipitation     sou

In [21]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""WITH bounds AS (
  SELECT
    lat, lon,
    date_trunc('hour', min(timestamp)) AS start_ts,
    date_trunc('hour', max(timestamp)) AS end_ts
  FROM historical_weather
  GROUP BY lat, lon
),
expected AS (
  SELECT b.lat, b.lon, gs.ts
  FROM bounds b
  CROSS JOIN LATERAL generate_series(b.start_ts, b.end_ts, INTERVAL 1 HOUR) AS gs(ts)
),
normalized_hw AS (
  SELECT DISTINCT lat, lon, date_trunc('hour', timestamp) AS ts_hour
  FROM historical_weather
)
SELECT e.lat, e.lon, e.ts AS missing_timestamp
FROM expected e
LEFT JOIN normalized_hw n
  ON e.lat = n.lat AND e.lon = n.lon AND e.ts = n.ts_hour
WHERE n.ts_hour IS NULL
ORDER BY e.lat, e.lon, e.ts;
""").fetchdf()

print(df)

con.close()


Empty DataFrame
Columns: [lat, lon, missing_timestamp]
Index: []


Backfill missing historic weather data

In [22]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""WITH bounds AS (
  SELECT
    lat, lon,
    date_trunc('hour', MIN(timestamp)) AS start_ts,
    date_trunc('hour', MAX(timestamp)) AS end_ts
  FROM historical_weather
  GROUP BY lat, lon
),
expected AS (
  SELECT b.lat, b.lon, gs.ts
  FROM bounds b
  CROSS JOIN LATERAL generate_series(b.start_ts, b.end_ts, INTERVAL 1 HOUR) AS gs(ts)
),
normalized_hw AS (
  SELECT DISTINCT lat, lon, date_trunc('hour', timestamp) AS ts_hour
  FROM historical_weather
),
missing_only AS (
  SELECT e.lat, e.lon, e.ts AS missing_ts
  FROM expected e
  LEFT JOIN normalized_hw n
    ON e.lat = n.lat AND e.lon = n.lon AND e.ts = n.ts_hour
  WHERE n.ts_hour IS NULL
),
nearest_before AS (
  SELECT
    mo.lat,
    mo.lon,
    mo.missing_ts,
    MAX(h.timestamp) AS prev_ts
  FROM missing_only mo
  JOIN historical_weather h
    ON h.lat = mo.lat
   AND h.lon = mo.lon
   AND h.timestamp < mo.missing_ts
  GROUP BY mo.lat, mo.lon, mo.missing_ts
),
nearest_after AS (
  SELECT
    mo.lat,
    mo.lon,
    mo.missing_ts,
    MIN(h.timestamp) AS next_ts
  FROM missing_only mo
  JOIN historical_weather h
    ON h.lat = mo.lat
   AND h.lon = mo.lon
   AND h.timestamp > mo.missing_ts
  GROUP BY mo.lat, mo.lon, mo.missing_ts
),
interp AS (
  SELECT
    b.lat,
    b.lon,
    b.missing_ts,
    'meteostat_interp' AS source,
    /* ratio of how far missing_ts is between prev_ts and next_ts, in seconds */
    CAST(DATEDIFF('second', b.prev_ts, b.missing_ts) AS DOUBLE)
      / NULLIF(CAST(DATEDIFF('second', b.prev_ts, a.next_ts) AS DOUBLE), 0) AS ratio,
    hb.temperature AS temp_prev,
    ha.temperature AS temp_next,
    hb.humidity    AS hum_prev,
    ha.humidity    AS hum_next,
    hb.windspeed   AS wind_prev,
    ha.windspeed   AS wind_next,
    hb.precipitation AS prcp_prev,
    ha.precipitation AS prcp_next
  FROM nearest_before b
  JOIN nearest_after  a
    ON b.lat = a.lat AND b.lon = a.lon AND b.missing_ts = a.missing_ts
  JOIN historical_weather hb
    ON hb.lat = b.lat AND hb.lon = b.lon AND hb.timestamp = b.prev_ts
  JOIN historical_weather ha
    ON ha.lat = a.lat AND ha.lon = a.lon AND ha.timestamp = a.next_ts
)
INSERT INTO historical_weather (
  timestamp, interval, lat, lon, temperature, humidity, windspeed, precipitation, source
)
SELECT
  i.missing_ts AS timestamp,
  CASE WHEN EXTRACT(hour FROM i.missing_ts) = 0 THEN 24 ELSE EXTRACT(hour FROM i.missing_ts) END AS interval,
  i.lat,
  i.lon,
  COALESCE(i.temp_prev + (i.temp_next - i.temp_prev) * i.ratio, i.temp_prev) AS temperature,
  COALESCE(i.hum_prev  + (i.hum_next  - i.hum_prev)  * i.ratio, i.hum_prev)  AS humidity,
  COALESCE(i.wind_prev + (i.wind_next - i.wind_prev) * i.ratio, i.wind_prev) AS windspeed,
  COALESCE(i.prcp_prev + (i.prcp_next - i.prcp_prev) * i.ratio, i.prcp_prev) AS precipitation,
  'meteostat_interp' AS source
FROM interp i
WHERE i.ratio IS NOT NULL;

""").fetchdf()

print(df)

con.close()


   Count
0      0


what was just inserted

In [17]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""SELECT * FROM historical_weather WHERE source = 'meteostat_interp' ORDER BY timestamp, lat, lon LIMIT 20;

""").fetchdf()

print(df)

con.close()


Empty DataFrame
Columns: [timestamp, interval, lat, lon, temperature, humidity, windspeed, precipitation, source]
Index: []


In [23]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""SELECT
  lat,
  lon,
  DATE(timestamp) AS operating_date,
  COUNT(DISTINCT interval) AS interval_count
FROM historical_weather
GROUP BY lat, lon, operating_date
HAVING COUNT(DISTINCT interval) <> 24
ORDER BY operating_date, lat, lon;


""").fetchdf()

print(df)

con.close()


       lat      lon operating_date  interval_count
0  29.7604 -95.3698     2022-12-31               5
1  29.7604 -95.3698     2025-08-16              21


what prices we got?

In [30]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("SELECT * FROM prices_hourly").fetchdf()

print(df)

con.close()


       OperatingDTM  TIME    Location   Price
0        2023-01-01   900  hb_houston   19.50
1        2023-01-01   960     hb_west   16.90
2        2023-01-01  1080      hb_pan   33.44
3        2023-01-01  1260  hb_houston   16.09
4        2023-01-01  1440      hb_pan    5.20
...             ...   ...         ...     ...
138373   2025-08-18   360   hb_hubavg   29.44
138374   2025-08-18   240     hb_west   30.30
138375   2025-08-18  1320   hb_hubavg  136.14
138376   2025-08-18  1080    hb_north   74.97
138377   2025-08-18   660    hb_south   25.22

[138378 rows x 4 columns]


refreshesr on all the tables and fields available so far for view creation: 

In [31]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 SELECT
                    c.table_schema,
                    c.table_name,
                    c.ordinal_position,
                    c.column_name,
                    c.data_type,
                    c.is_nullable
                FROM information_schema.columns AS c
                  JOIN information_schema.tables  AS t
                    ON c.table_schema = t.table_schema
                    AND c.table_name   = t.table_name
                WHERE t.table_type = 'BASE TABLE'            -- change to IN ('BASE TABLE','VIEW') to include views
                    AND c.table_schema NOT IN ('information_schema','pg_catalog')
                ORDER BY c.table_schema, c.table_name, c.ordinal_position;
""").fetchdf()

print(df)

con.close()


   table_schema          table_name  ordinal_position    column_name  \
0          main        LoadForecast                 1   OperatingDTM   
1          main        LoadForecast                 2       Interval   
2          main        LoadForecast                 3          Month   
3          main        LoadForecast                 4   LocationType   
4          main        LoadForecast                 5       Location   
5          main        LoadForecast                 6        DSTFlag   
6          main        LoadForecast                 7     ForecastMW   
7          main   LoadForecastPivot                 1   OperatingDTM   
8          main   LoadForecastPivot                 2       Interval   
9          main   LoadForecastPivot                 3   LocationType   
10         main   LoadForecastPivot                 4       Location   
11         main   LoadForecastPivot                 5        DSTFlag   
12         main   LoadForecastPivot                 6     Foreca

top records of all tables 

In [33]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 SELECT
                    c.table_schema,
                    c.table_name,
                    c.ordinal_position,
                    c.column_name,
                    c.data_type,
                    c.is_nullable
                FROM information_schema.columns AS c
                  JOIN information_schema.tables  AS t
                    ON c.table_schema = t.table_schema
                    AND c.table_name   = t.table_name
                WHERE t.table_type = 'BASE TABLE'            -- change to IN ('BASE TABLE','VIEW') to include views
                    AND c.table_schema NOT IN ('information_schema','pg_catalog')
                ORDER BY c.table_schema, c.table_name, c.ordinal_position;
""").fetchdf()

print(df)

con.close()


   table_schema          table_name  ordinal_position    column_name  \
0          main        LoadForecast                 1   OperatingDTM   
1          main        LoadForecast                 2       Interval   
2          main        LoadForecast                 3          Month   
3          main        LoadForecast                 4   LocationType   
4          main        LoadForecast                 5       Location   
5          main        LoadForecast                 6        DSTFlag   
6          main        LoadForecast                 7     ForecastMW   
7          main   LoadForecastPivot                 1   OperatingDTM   
8          main   LoadForecastPivot                 2       Interval   
9          main   LoadForecastPivot                 3   LocationType   
10         main   LoadForecastPivot                 4       Location   
11         main   LoadForecastPivot                 5        DSTFlag   
12         main   LoadForecastPivot                 6     Foreca

In [35]:

import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                SELECT 'prices_hourly'      AS table_name, COUNT(*) AS row_count FROM prices_hourly
UNION ALL SELECT 'historical_weather', COUNT(*) FROM historical_weather
UNION ALL SELECT 'forecast_weather',  COUNT(*) FROM forecast_weather
UNION ALL SELECT 'LoadForecast',      COUNT(*) FROM "LoadForecast"
UNION ALL SELECT 'WeatherFcst',       COUNT(*) FROM "WeatherFcst"
UNION ALL SELECT 'WeatherHist',       COUNT(*) FROM "WeatherHist";
""").fetchdf()

print(df)

con.close()


           table_name  row_count
0       prices_hourly     138378
1  historical_weather      23018
2    forecast_weather          0
3        LoadForecast     278196
4         WeatherFcst          0
5         WeatherHist          0


In [43]:

import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                SELECT 'prices_hourly'      AS table_name, COUNT(*) AS row_count FROM prices_hourly
UNION ALL SELECT 'historical_weather', COUNT(*) FROM historical_weather
UNION ALL SELECT 'forecast_weather',  COUNT(*) FROM forecast_weather
UNION ALL SELECT 'LoadForecast',      COUNT(*) FROM "LoadForecast"
UNION ALL SELECT 'WeatherFcst',       COUNT(*) FROM "WeatherFcst"
UNION ALL SELECT 'WeatherHist',       COUNT(*) FROM "WeatherHist";
""").fetchdf()

print(df)

con.close()

import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ercot_app/ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                SELECT
                    c.table_schema,
                    c.table_name,
                    c.ordinal_position,
                    c.column_name,
                    c.data_type,
                    c.is_nullable
                FROM information_schema.columns AS c
                  JOIN information_schema.tables  AS t
                    ON c.table_schema = t.table_schema
                    AND c.table_name   = t.table_name
                WHERE t.table_type = 'BASE TABLE'            -- change to IN ('BASE TABLE','VIEW') to include views
                    AND c.table_schema NOT IN ('information_schema','pg_catalog')
                ORDER BY c.table_schema, c.table_name, c.ordinal_position;
""").fetchdf()

print(df)

con.close()


           table_name  row_count
0       prices_hourly     138378
1  historical_weather      23018
2    forecast_weather          0
3        LoadForecast     278196
4         WeatherFcst          0
5         WeatherHist          0
   table_schema          table_name  ordinal_position    column_name  \
0          main              Prices                 1      timestamp   
1          main              Prices                 2            lmp   
2          main           SessionID                 1             ts   
3          main           SessionID                 2     session_id   
4          main    forecast_weather                 1  forecast_time   
5          main    forecast_weather                 2      timestamp   
6          main    forecast_weather                 3       interval   
7          main    forecast_weather                 4            lat   
8          main    forecast_weather                 5            lon   
9          main    forecast_weather              

In [39]:
# save as print_tree.py and run: python print_tree.py > scaffold.txt
from pathlib import Path

IGNORE = {'.git', '.venv', '__pycache__'}
root = Path('.').resolve()

def walk(dir_path: Path, prefix=""):
  entries = sorted([p for p in dir_path.iterdir() if p.name not in IGNORE], key=lambda p: (p.is_file(), p.name.lower()))
  for i, p in enumerate(entries):
    connector = "└── " if i == len(entries)-1 else "├── "
    print(prefix + connector + p.name + ("/" if p.is_dir() else ""))
    if p.is_dir():
      extension = "    " if i == len(entries)-1 else "│   "
      walk(p, prefix + extension)

print(root.name + "/")
walk(root)


utilities/
├── hw_maintenance.py
├── lookup_locations.py
├── preview_load_request.py
├── queries.ipynb
└── query_duck.py


In [40]:

import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                PRAGMA database_list;
""").fetchdf()

print(df)

con.close()


   seq  name                                               file
0  570  data  c:\users\db\drewbcapstone\utilities\..\project...


comparing ercot_app db to main db

In [45]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ercot_app/ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                SELECT 'prices'      AS table_name, COUNT(*) AS row_count FROM prices
UNION ALL SELECT 'historical_weather', COUNT(*) FROM historical_weather
UNION ALL SELECT 'forecast_weather',  COUNT(*) FROM forecast_weather;
""").fetchdf()

print(df)

con.close()


import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                SELECT 'prices_hourly'      AS table_name, COUNT(*) AS row_count FROM prices_hourly
UNION ALL SELECT 'historical_weather', COUNT(*) FROM historical_weather
UNION ALL SELECT 'forecast_weather',  COUNT(*) FROM forecast_weather
UNION ALL SELECT 'LoadForecast',      COUNT(*) FROM "LoadForecast"
UNION ALL SELECT 'WeatherFcst',       COUNT(*) FROM "WeatherFcst"
UNION ALL SELECT 'WeatherHist',       COUNT(*) FROM "WeatherHist";
""").fetchdf()

print(df)

con.close()


           table_name  row_count
0              prices          0
1  historical_weather     114380
2    forecast_weather       3768
           table_name  row_count
0       prices_hourly     138378
1  historical_weather      23310
2    forecast_weather          0
3        LoadForecast     278196
4         WeatherFcst          0
5         WeatherHist          0


In [46]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ercot_app/ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                SELECT 'prices'      AS table_name, COUNT(*) AS row_count FROM prices
UNION ALL SELECT 'historical_weather', COUNT(*) FROM historical_weather
UNION ALL SELECT 'forecast_weather',  COUNT(*) FROM forecast_weather;
""").fetchdf()

print(df)

con.close()


import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                SELECT 'prices_hourly'      AS table_name, COUNT(*) AS row_count FROM prices_hourly
UNION ALL SELECT 'historical_weather', COUNT(*) FROM historical_weather
UNION ALL SELECT 'forecast_weather',  COUNT(*) FROM forecast_weather
UNION ALL SELECT 'LoadForecast',      COUNT(*) FROM "LoadForecast"
UNION ALL SELECT 'WeatherFcst',       COUNT(*) FROM "WeatherFcst"
UNION ALL SELECT 'WeatherHist',       COUNT(*) FROM "WeatherHist";
""").fetchdf()

print(df)

con.close()


           table_name  row_count
0              prices          0
1  historical_weather     114380
2    forecast_weather       3768
           table_name  row_count
0       prices_hourly     138378
1  historical_weather     114719
2    forecast_weather          0
3        LoadForecast     278196
4         WeatherFcst          0
5         WeatherHist          0


In [47]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ercot_app/ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                SELECT 'prices'      AS table_name, COUNT(*) AS row_count FROM prices
UNION ALL SELECT 'historical_weather', COUNT(*) FROM historical_weather
UNION ALL SELECT 'forecast_weather',  COUNT(*) FROM forecast_weather;
""").fetchdf()

print(df)

con.close()


import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                SELECT 'prices_hourly'      AS table_name, COUNT(*) AS row_count FROM prices_hourly
UNION ALL SELECT 'historical_weather', COUNT(*) FROM historical_weather
UNION ALL SELECT 'forecast_weather',  COUNT(*) FROM forecast_weather
UNION ALL SELECT 'LoadForecast',      COUNT(*) FROM "LoadForecast"
UNION ALL SELECT 'WeatherFcst',       COUNT(*) FROM "WeatherFcst"
UNION ALL SELECT 'WeatherHist',       COUNT(*) FROM "WeatherHist";
""").fetchdf()

print(df)

con.close()


           table_name  row_count
0              prices          0
1  historical_weather     114380
2    forecast_weather       3768
           table_name  row_count
0       prices_hourly     138378
1  historical_weather     114719
2    forecast_weather        960
3        LoadForecast     278196
4         WeatherFcst          0
5         WeatherHist          0


FInally, the main database is populated with all the data needed to build a view for training. 

In [50]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 SELECT
                    c.table_schema,
                    c.table_name,
                    c.ordinal_position,
                    c.column_name,
                    c.data_type,
                    c.is_nullable
                FROM information_schema.columns AS c
                  JOIN information_schema.tables  AS t
                    ON c.table_schema = t.table_schema
                    AND c.table_name   = t.table_name
                WHERE t.table_type = 'BASE TABLE'            -- change to IN ('BASE TABLE','VIEW') to include views
                    AND c.table_schema NOT IN ('information_schema','pg_catalog')
                ORDER BY c.table_schema, c.table_name, c.ordinal_position;
""").fetchdf()

print(df)

con.close()


   table_schema          table_name  ordinal_position    column_name  \
0          main        LoadForecast                 1   OperatingDTM   
1          main        LoadForecast                 2       Interval   
2          main        LoadForecast                 3          Month   
3          main        LoadForecast                 4   LocationType   
4          main        LoadForecast                 5       Location   
5          main        LoadForecast                 6        DSTFlag   
6          main        LoadForecast                 7     ForecastMW   
7          main   LoadForecastPivot                 1   OperatingDTM   
8          main   LoadForecastPivot                 2       Interval   
9          main   LoadForecastPivot                 3   LocationType   
10         main   LoadForecastPivot                 4       Location   
11         main   LoadForecastPivot                 5        DSTFlag   
12         main   LoadForecastPivot                 6     Foreca

Pivoted LoadForecast query

In [54]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
-- Wide, one row per OperatingDTM+Interval (and Month/DSTFlag), one column per Location
SELECT
  OperatingDTM,
  CAST(Interval AS INTEGER) AS Interval,
  Month,
  DSTFlag,
  SUM(CASE WHEN lower(Location) = 'lz_houston'  THEN ForecastMW END) AS lz_houston,
  SUM(CASE WHEN lower(Location) = 'lz_north'    THEN ForecastMW END) AS lz_north,
  SUM(CASE WHEN lower(Location) = 'lz_south'    THEN ForecastMW END) AS lz_south,
  SUM(CASE WHEN lower(Location) = 'lz_west'     THEN ForecastMW END) AS lz_west
  -- add more locations as needed...
FROM "LoadForecast"
GROUP BY 1,2,3,4
ORDER BY OperatingDTM, Interval;

""").fetchdf()

print(df)

con.close()

      OperatingDTM  Interval   Month DSTFlag  lz_houston  lz_north  lz_south  \
0       2023-01-01         1  1-2023       N         NaN       NaN       NaN   
1       2023-01-01         2  1-2023       N         NaN       NaN       NaN   
2       2023-01-01         3  1-2023       N         NaN       NaN       NaN   
3       2023-01-01         4  1-2023       N         NaN       NaN       NaN   
4       2023-01-01         5  1-2023       N         NaN       NaN       NaN   
...            ...       ...     ...     ...         ...       ...       ...   
23178   2025-08-23        20  8-2025       N         NaN       NaN       NaN   
23179   2025-08-23        21  8-2025       N         NaN       NaN       NaN   
23180   2025-08-23        22  8-2025       N         NaN       NaN       NaN   
23181   2025-08-23        23  8-2025       N         NaN       NaN       NaN   
23182   2025-08-23        24  8-2025       N         NaN       NaN       NaN   

       lz_west  
0          NaN  
1    

In [58]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
-- Wide, one row per OperatingDTM+Interval (and Month/DSTFlag), one column per Location
SELECT Distinct Location,LocationType          
  FROM "LoadForecast"
;
""").fetchdf()

print(df)

con.close()

        Location  LocationType
0   SouthCentral  Weather Zone
1           East  Weather Zone
2           West  Weather Zone
3   NorthCentral  Weather Zone
4        FarWest  Weather Zone
5          North     Load Zone
6           West     Load Zone
7          North  Weather Zone
8       Southern  Weather Zone
9          Coast  Weather Zone
10         South     Load Zone
11       Houston     Load Zone


In [57]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
select * from LoadForecast
                 LIMIT 10;
""").fetchdf()

print(df)

con.close()

  OperatingDTM  Interval   Month  LocationType      Location DSTFlag  \
0   2023-01-01         1  1-2023  Weather Zone         North       N   
1   2023-01-01         1  1-2023  Weather Zone      Southern       N   
2   2023-01-01         1  1-2023  Weather Zone  SouthCentral       N   
3   2023-01-01         1  1-2023  Weather Zone  NorthCentral       N   
4   2023-01-01         1  1-2023  Weather Zone         Coast       N   
5   2023-01-01         1  1-2023  Weather Zone          East       N   
6   2023-01-01         1  1-2023  Weather Zone       FarWest       N   
7   2023-01-01         1  1-2023  Weather Zone          West       N   
8   2023-01-01         1  1-2023     Load Zone         North       N   
9   2023-01-01         1  1-2023     Load Zone         South       N   

     ForecastMW  
0    758.773000  
1   2815.000000  
2   5831.879900  
3   9965.370100  
4   8766.250000  
5   1219.980000  
6   5324.109900  
7   1129.740000  
8  11568.836588  
9   9328.544071  


In [ ]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
select * from LoadForecast
                 LIMIT 10;
""").fetchdf()

print(df)

con.close()

Pivoted Load Forecast data by Location Type and Location

In [59]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 WITH base AS (
  SELECT
      OperatingDTM,
      CAST(Interval AS INTEGER) AS Interval,
      Month,
      DSTFlag,
      lower(LocationType) AS lt,
      lower(Location)     AS loc,
      ForecastMW
  FROM "LoadForecast"
)
SELECT
  OperatingDTM,
  Interval,
  Month,
  DSTFlag,

  -- Weather Zone columns
  SUM(CASE WHEN lt='weather zone' AND loc='southcentral' THEN ForecastMW END) AS wz_southcentral,
  SUM(CASE WHEN lt='weather zone' AND loc='east'         THEN ForecastMW END) AS wz_east,
  SUM(CASE WHEN lt='weather zone' AND loc='west'         THEN ForecastMW END) AS wz_west,
  SUM(CASE WHEN lt='weather zone' AND loc='northcentral' THEN ForecastMW END) AS wz_northcentral,
  SUM(CASE WHEN lt='weather zone' AND loc='farwest'      THEN ForecastMW END) AS wz_farwest,
  SUM(CASE WHEN lt='weather zone' AND loc='north'        THEN ForecastMW END) AS wz_north,
  SUM(CASE WHEN lt='weather zone' AND loc='southern'     THEN ForecastMW END) AS wz_southern,
  SUM(CASE WHEN lt='weather zone' AND loc='coast'        THEN ForecastMW END) AS wz_coast,

  -- Load Zone columns
  SUM(CASE WHEN lt='load zone' AND loc='north'           THEN ForecastMW END) AS lz_north,
  SUM(CASE WHEN lt='load zone' AND loc='west'            THEN ForecastMW END) AS lz_west,
  SUM(CASE WHEN lt='load zone' AND loc='south'           THEN ForecastMW END) AS lz_south,
  SUM(CASE WHEN lt='load zone' AND loc='houston'         THEN ForecastMW END) AS lz_houston

FROM base
GROUP BY 1,2,3,4
ORDER BY OperatingDTM, Interval;
""").fetchdf()

print(df)

con.close()

      OperatingDTM  Interval   Month DSTFlag  wz_southcentral    wz_east  \
0       2023-01-01         1  1-2023       N        5831.8799  1219.9800   
1       2023-01-01         2  1-2023       N        5677.6401  1219.2800   
2       2023-01-01         3  1-2023       N        5544.1802  1244.1200   
3       2023-01-01         4  1-2023       N        5460.6899  1201.3199   
4       2023-01-01         5  1-2023       N        5512.1699  1204.2400   
...            ...       ...     ...     ...              ...        ...   
23178   2025-08-23        20  8-2025       N       12319.8994  2547.4299   
23179   2025-08-23        21  8-2025       N       11860.6748  2438.8701   
23180   2025-08-23        22  8-2025       N       11418.9746  2320.7075   
23181   2025-08-23        23  8-2025       N       10797.6992  2187.1775   
23182   2025-08-23        24  8-2025       N       10105.1426  2033.3350   

         wz_west  wz_northcentral  wz_farwest   wz_north  wz_southern  \
0      1129.74

Create a view with that query

In [60]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""CREATE OR REPLACE VIEW vw_loadforecast_by_zone AS
WITH base AS (
  SELECT
      OperatingDTM,
      CAST(Interval AS INTEGER) AS Interval,
      Month,
      DSTFlag,
      lower(LocationType) AS lt,
      lower(Location)     AS loc,
      ForecastMW
  FROM "LoadForecast"
)
SELECT
  OperatingDTM,
  Interval,
  Month,
  DSTFlag,

  -- Weather Zone columns
  SUM(CASE WHEN lt='weather zone' AND loc='southcentral' THEN ForecastMW END) AS wz_southcentral,
  SUM(CASE WHEN lt='weather zone' AND loc='east'         THEN ForecastMW END) AS wz_east,
  SUM(CASE WHEN lt='weather zone' AND loc='west'         THEN ForecastMW END) AS wz_west,
  SUM(CASE WHEN lt='weather zone' AND loc='northcentral' THEN ForecastMW END) AS wz_northcentral,
  SUM(CASE WHEN lt='weather zone' AND loc='farwest'      THEN ForecastMW END) AS wz_farwest,
  SUM(CASE WHEN lt='weather zone' AND loc='north'        THEN ForecastMW END) AS wz_north,
  SUM(CASE WHEN lt='weather zone' AND loc='southern'     THEN ForecastMW END) AS wz_southern,
  SUM(CASE WHEN lt='weather zone' AND loc='coast'        THEN ForecastMW END) AS wz_coast,

  -- Load Zone columns
  SUM(CASE WHEN lt='load zone' AND loc='north'           THEN ForecastMW END) AS lz_north,
  SUM(CASE WHEN lt='load zone' AND loc='west'            THEN ForecastMW END) AS lz_west,
  SUM(CASE WHEN lt='load zone' AND loc='south'           THEN ForecastMW END) AS lz_south,
  SUM(CASE WHEN lt='load zone' AND loc='houston'         THEN ForecastMW END) AS lz_houston

FROM base
GROUP BY 1,2,3,4;
""").fetchdf()

print(df)

con.close()

Empty DataFrame
Columns: [Count]
Index: []


In [ ]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 select * from vw_loadforecast_by_zone order by 1,2 desc Limit 200
""").fetchdf()

print(df)

con.close()

      OperatingDTM  Interval   Month DSTFlag  wz_southcentral    wz_east  \
0       2025-08-23         1  8-2025       N        9345.7148  1915.8475   
1       2025-08-23         4  8-2025       N        7997.4878  1647.8125   
2       2025-08-23        12  8-2025       N       10691.9756  2247.4026   
3       2025-08-23        22  8-2025       N       11418.9746  2320.7075   
4       2025-08-23         9  8-2025       N        8461.2676  1736.9750   
...            ...       ...     ...     ...              ...        ...   
23178   2023-01-01         9  1-2023       N        6065.9102  1427.3000   
23179   2023-01-01        20  1-2023       N        7026.8945  1459.9000   
23180   2023-01-01        22  1-2023       N        6619.1650  1423.3440   
23181   2023-01-01        10  1-2023       N        6097.6401  1406.4800   
23182   2023-01-01        16  1-2023       N        6567.7715  1376.9348   

         wz_west  wz_northcentral  wz_farwest   wz_north  wz_southern  \
0      1368.06

testing out prices_hourly table

In [63]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
-- Wide, one row per OperatingDTM+Interval (and Month/DSTFlag), one column per Location
SELECT Distinct Location          
  FROM "prices_hourly"
;
""").fetchdf()

print(df)

con.close()

     Location
0   hb_hubavg
1     hb_west
2    hb_north
3    hb_south
4  hb_houston
5      hb_pan


In [73]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""CREATE OR REPLACE VIEW vw_prices_by_hub AS
WITH base AS (
  SELECT
      p.OperatingDTM,
      CAST(
        COALESCE(
          TRY_CAST(regexp_extract(p."TIME", 'HE(\\d+)', 1) AS INTEGER),
          CASE
            WHEN TRY_CAST(p."TIME" AS INTEGER) IS NOT NULL
            THEN FLOOR(TRY_CAST(p."TIME" AS INTEGER) / 60)
            ELSE CAST(EXTRACT(hour FROM p.OperatingDTM) AS INTEGER)
          END
        ) AS INTEGER
      ) AS Interval,
      lower(p.Location) AS loc,
      p.Price           AS price
  FROM prices_hourly p
)
SELECT
  OperatingDTM,
  Interval,
  AVG(CASE WHEN loc='hb_hubavg'  THEN price END) AS hb_hubavg,
  AVG(CASE WHEN loc='hb_west'    THEN price END) AS hb_west,
  AVG(CASE WHEN loc='hb_north'   THEN price END) AS hb_north,
  AVG(CASE WHEN loc='hb_south'   THEN price END) AS hb_south,
  AVG(CASE WHEN loc='hb_houston' THEN price END) AS hb_houston,
  AVG(CASE WHEN loc='hb_pan'     THEN price END) AS hb_pan
FROM base
GROUP BY 1,2;
""").fetchdf()

print(df)

con.close()


Empty DataFrame
Columns: [Count]
Index: []


In [74]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 select * from vw_prices_by_hub order by 1,2 desc
""").fetchdf()

print(df)

con.close()

      OperatingDTM  Interval  hb_hubavg  hb_west  hb_north  hb_south  \
0       2023-01-01        24       8.14     5.60     10.37      6.93   
1       2023-01-01        23      12.69    11.20     14.79     11.00   
2       2023-01-01        22      16.36    17.10     17.70     13.90   
3       2023-01-01        21      15.71    16.61     16.95     13.18   
4       2023-01-01        20      17.92    19.06     19.24     15.02   
...            ...       ...        ...      ...       ...       ...   
23058   2025-08-18         5      27.78    30.65     26.70     26.97   
23059   2025-08-18         4      26.86    30.30     25.53     25.91   
23060   2025-08-18         3      28.46    32.31     26.89     27.48   
23061   2025-08-18         2      29.90    34.29     27.98     28.91   
23062   2025-08-18         1      32.21    37.29     29.89     31.13   

       hb_houston  hb_pan  
0            9.65    5.20  
1           13.78   10.86  
2           16.74   16.68  
3           16.09   16.

make a mapping table of lat/lon to city

In [75]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 CREATE OR REPLACE TABLE city_coords (
  city VARCHAR,
  lat  DOUBLE,
  lon  DOUBLE
);

INSERT INTO city_coords (city, lat, lon) VALUES
  ('The_Woodlands_TX', 30.1658, -95.4613),
  ('Katy_TX',           29.7858, -95.8245),
  ('Friendswood_TX',    29.5294, -95.2010),
  ('Baytown_TX',        29.7355, -94.9774),
  ('Houston_TX',        29.7604, -95.3698);

""").fetchdf()

print(df)

con.close()

   Count
0      5


Join the label table and pivot the locations with weather forecast

In [15]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
CREATE OR REPLACE VIEW vw_forecast_weather_by_city AS
WITH labeled AS (
  SELECT
      f.forecast_time,
      f."timestamp",
      f."interval",
      /* OperatingDTM logic:
         - If time = 00:00:00, assign to prior operating day
         - Else assign to same day
      */
      CASE
        WHEN STRFTIME(f."timestamp", '%H:%M:%S') = '00:00:00'
          THEN CAST(DATE_TRUNC('day', f."timestamp") - INTERVAL 1 DAY AS DATE)
        ELSE CAST(DATE_TRUNC('day', f."timestamp") AS DATE)
      END AS OperatingDTM,
      f.lat,
      f.lon,
      COALESCE(c.city, 'unmapped') AS city_name,
      -- slug for flexible naming if needed later
      REGEXP_REPLACE(LOWER(COALESCE(c.city, 'unmapped')), '[^a-z0-9]+', '_') AS city_key,
      f.temperature,
      f.humidity,
      f.windspeed,
      f.precipitation
  FROM forecast_weather f
  LEFT JOIN city_coords c
    ON ROUND(f.lat, 4) = ROUND(c.lat, 4)
   AND ROUND(f.lon, 4) = ROUND(c.lon, 4)
),
latest AS (
  -- keep the most recent forecast issuance per delivery hour & city
  SELECT *
  FROM (
    SELECT
      labeled.*,
      ROW_NUMBER() OVER (
        PARTITION BY "timestamp", "interval", city_name
        ORDER BY forecast_time DESC
      ) AS rn
    FROM labeled
  )
  WHERE rn = 1
)
SELECT
  /* Keys for joining to loadforecast */
  OperatingDTM,
  "timestamp",
  "interval",

  /* The_Woodlands_TX */
  AVG(CASE WHEN city_name='The_Woodlands_TX' THEN temperature   END) AS temp_the_woodlands_tx,
  AVG(CASE WHEN city_name='The_Woodlands_TX' THEN humidity      END) AS hum_the_woodlands_tx,
  AVG(CASE WHEN city_name='The_Woodlands_TX' THEN windspeed     END) AS wind_the_woodlands_tx,
  AVG(CASE WHEN city_name='The_Woodlands_TX' THEN precipitation END) AS precip_the_woodlands_tx,

  /* Katy_TX */
  AVG(CASE WHEN city_name='Katy_TX' THEN temperature   END) AS temp_katy_tx,
  AVG(CASE WHEN city_name='Katy_TX' THEN humidity      END) AS hum_katy_tx,
  AVG(CASE WHEN city_name='Katy_TX' THEN windspeed     END) AS wind_katy_tx,
  AVG(CASE WHEN city_name='Katy_TX' THEN precipitation END) AS precip_katy_tx,

  /* Friendswood_TX */
  AVG(CASE WHEN city_name='Friendswood_TX' THEN temperature   END) AS temp_friendswood_tx,
  AVG(CASE WHEN city_name='Friendswood_TX' THEN humidity      END) AS hum_friendswood_tx,
  AVG(CASE WHEN city_name='Friendswood_TX' THEN windspeed     END) AS wind_friendswood_tx,
  AVG(CASE WHEN city_name='Friendswood_TX' THEN precipitation END) AS precip_friendswood_tx,

  /* Baytown_TX */
  AVG(CASE WHEN city_name='Baytown_TX' THEN temperature   END) AS temp_baytown_tx,
  AVG(CASE WHEN city_name='Baytown_TX' THEN humidity      END) AS hum_baytown_tx,
  AVG(CASE WHEN city_name='Baytown_TX' THEN windspeed     END) AS wind_baytown_tx,
  AVG(CASE WHEN city_name='Baytown_TX' THEN precipitation END) AS precip_baytown_tx,

  /* Houston_TX */
  AVG(CASE WHEN city_name='Houston_TX' THEN temperature   END) AS temp_houston_tx,
  AVG(CASE WHEN city_name='Houston_TX' THEN humidity      END) AS hum_houston_tx,
  AVG(CASE WHEN city_name='Houston_TX' THEN windspeed     END) AS wind_houston_tx,
  AVG(CASE WHEN city_name='Houston_TX' THEN precipitation END) AS precip_houston_tx

FROM latest
GROUP BY OperatingDTM, "timestamp", "interval"
ORDER BY OperatingDTM, "timestamp", "interval";
""").fetchdf()

print(df)

con.close()

Empty DataFrame
Columns: [Count]
Index: []


In [77]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 select * from vw_forecast_weather_by_city 
;
""").fetchdf()

print(df)

con.close()

              timestamp  interval  temp_the_woodlands_tx  \
0   2025-08-17 12:00:00        12                   31.3   
1   2025-08-18 04:00:00         4                   26.6   
2   2025-08-18 12:00:00        12                   33.6   
3   2025-08-18 16:00:00        16                   37.7   
4   2025-08-19 14:00:00        14                   28.2   
..                  ...       ...                    ...   
187 2025-08-24 13:00:00        13                   33.6   
188 2025-08-23 08:00:00         8                   24.1   
189 2025-08-23 23:00:00        23                   30.2   
190 2025-08-23 21:00:00        21                   31.8   
191 2025-08-23 18:00:00        18                   34.4   

     hum_the_woodlands_tx  wind_the_woodlands_tx  precip_the_woodlands_tx  \
0                    74.0                   0.45                      0.0   
1                    79.0                   1.84                      0.0   
2                    54.0                   1.52

So close, just the historic weather to build then we have four views to work across for training.

In [78]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
CREATE OR REPLACE VIEW vw_historical_weather_by_city AS
WITH labeled AS (
  SELECT
    h."timestamp",
    h."interval",
    COALESCE(c.city, 'unmapped') AS city_name,
    h.temperature,
    h.humidity,
    h.windspeed,
    h.precipitation
  FROM historical_weather h
  LEFT JOIN city_coords c
    ON ROUND(h.lat, 4) = ROUND(c.lat, 4)
   AND ROUND(h.lon, 4) = ROUND(c.lon, 4)
)
SELECT
  "timestamp",
  "interval",

  /* The_Woodlands_TX */
  AVG(CASE WHEN city_name='The_Woodlands_TX' THEN temperature   END) AS temp_the_woodlands_tx,
  AVG(CASE WHEN city_name='The_Woodlands_TX' THEN humidity      END) AS hum_the_woodlands_tx,
  AVG(CASE WHEN city_name='The_Woodlands_TX' THEN windspeed     END) AS wind_the_woodlands_tx,
  AVG(CASE WHEN city_name='The_Woodlands_TX' THEN precipitation END) AS precip_the_woodlands_tx,

  /* Katy_TX */
  AVG(CASE WHEN city_name='Katy_TX' THEN temperature   END) AS temp_katy_tx,
  AVG(CASE WHEN city_name='Katy_TX' THEN humidity      END) AS hum_katy_tx,
  AVG(CASE WHEN city_name='Katy_TX' THEN windspeed     END) AS wind_katy_tx,
  AVG(CASE WHEN city_name='Katy_TX' THEN precipitation END) AS precip_katy_tx,

  /* Friendswood_TX */
  AVG(CASE WHEN city_name='Friendswood_TX' THEN temperature   END) AS temp_friendswood_tx,
  AVG(CASE WHEN city_name='Friendswood_TX' THEN humidity      END) AS hum_friendswood_tx,
  AVG(CASE WHEN city_name='Friendswood_TX' THEN windspeed     END) AS wind_friendswood_tx,
  AVG(CASE WHEN city_name='Friendswood_TX' THEN precipitation END) AS precip_friendswood_tx,

  /* Baytown_TX */
  AVG(CASE WHEN city_name='Baytown_TX' THEN temperature   END) AS temp_baytown_tx,
  AVG(CASE WHEN city_name='Baytown_TX' THEN humidity      END) AS hum_baytown_tx,
  AVG(CASE WHEN city_name='Baytown_TX' THEN windspeed     END) AS wind_baytown_tx,
  AVG(CASE WHEN city_name='Baytown_TX' THEN precipitation END) AS precip_baytown_tx,

  /* Houston_TX */
  AVG(CASE WHEN city_name='Houston_TX' THEN temperature   END) AS temp_houston_tx,
  AVG(CASE WHEN city_name='Houston_TX' THEN humidity      END) AS hum_houston_tx,
  AVG(CASE WHEN city_name='Houston_TX' THEN windspeed     END) AS wind_houston_tx,
  AVG(CASE WHEN city_name='Houston_TX' THEN precipitation END) AS precip_houston_tx

FROM labeled
GROUP BY 1,2;
""").fetchdf()

print(df)

con.close()

Empty DataFrame
Columns: [Count]
Index: []


In [79]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 select * from vw_historical_weather_by_city 
;
""").fetchdf()

print(df)

con.close()

                timestamp  interval  temp_the_woodlands_tx  \
0     2023-10-25 12:00:00        12                   28.3   
1     2023-10-25 18:00:00        18                   27.8   
2     2023-10-26 01:00:00         1                   25.6   
3     2023-10-26 16:00:00        16                   26.1   
4     2023-10-26 20:00:00        20                   26.1   
...                   ...       ...                    ...   
23013 2023-10-24 10:00:00        10                   25.0   
23014 2023-10-24 12:00:00        12                   29.4   
23015 2023-10-24 13:00:00        13                   30.6   
23016 2023-10-24 16:00:00        16                   30.6   
23017 2023-10-24 20:00:00        20                   26.7   

       hum_the_woodlands_tx  wind_the_woodlands_tx  precip_the_woodlands_tx  \
0                      70.0               6.694444                      0.0   
1                      74.0               7.694444                      0.3   
2                 

Now we have four views of base data both forecast and historic.

# vw_loadforecast_by_zone,
# vw_forecast_weather_by_city,
# vw_historical_weather_by_city,
# vw_prices_by_hub


In [82]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 select * from vw_loadforecast_by_zone order by 1 desc,2 desc Limit 2000
""").fetchdf()

print(df)

con.close()

     OperatingDTM  Interval   Month DSTFlag  wz_southcentral    wz_east  \
0      2025-08-23        24  8-2025       N       10105.1426  2033.3350   
1      2025-08-23        23  8-2025       N       10797.6992  2187.1775   
2      2025-08-23        22  8-2025       N       11418.9746  2320.7075   
3      2025-08-23        21  8-2025       N       11860.6748  2438.8701   
4      2025-08-23        20  8-2025       N       12319.8994  2547.4299   
...           ...       ...     ...     ...              ...        ...   
1995   2025-06-01        21  6-2025       N       12427.0996  1958.4700   
1996   2025-06-01        20  6-2025       N       12868.0000  2070.0701   
1997   2025-06-01        19  6-2025       N       13119.7002  2131.6799   
1998   2025-06-01        18  6-2025       N       13110.9004  2152.7600   
1999   2025-06-01        17  6-2025       N       12815.2998  2104.6299   

        wz_west  wz_northcentral  wz_farwest   wz_north  wz_southern  \
0     1456.1176       17665

In [83]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 select * from vw_forecast_weather_by_city order by 1 desc,2 desc Limit 2000
""").fetchdf()

print(df)

con.close()

              timestamp  interval  temp_the_woodlands_tx  \
0   2025-08-25 00:00:00        24                   30.0   
1   2025-08-24 23:00:00        23                   30.7   
2   2025-08-24 22:00:00        22                   31.4   
3   2025-08-24 21:00:00        21                   32.3   
4   2025-08-24 20:00:00        20                   33.3   
..                  ...       ...                    ...   
187 2025-08-17 05:00:00         5                   25.1   
188 2025-08-17 04:00:00         4                   25.5   
189 2025-08-17 03:00:00         3                   25.9   
190 2025-08-17 02:00:00         2                   26.3   
191 2025-08-17 01:00:00         1                   26.5   

     hum_the_woodlands_tx  wind_the_woodlands_tx  precip_the_woodlands_tx  \
0                    61.0                   3.70                      0.0   
1                    58.0                   4.22                      0.0   
2                    54.0                   4.18

attempt 1 at join:



In [ ]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""-- One-stop wide table for modeling
CREATE OR REPLACE VIEW vw_spine_load_wx_price AS
WITH spine AS (
  SELECT
    /* normalize to hour if needed; comment out DATE_TRUNC if already hourly */
    DATE_TRUNC('hour', OperatingDTM) AS ts,
    OperatingDTM,
    Interval AS load_interval,
    Month,
    DSTFlag,
    -- keep your load forecast columns (zones & load zones); include * if you prefer
    wz_southcentral, wz_east, wz_west, wz_northcentral, wz_farwest, wz_north, wz_southern, wz_coast,
    lz_north, lz_west, lz_south, lz_houston
  FROM vw_loadforecast_by_zone
),
calendar_feats AS (
  SELECT
    ts,
    EXTRACT(hour  FROM ts)::INT  AS cal_hour,
    EXTRACT(dow   FROM ts)::INT  AS cal_dow,
    EXTRACT(month FROM ts)::INT  AS cal_month_num,
    STRFTIME(ts, '%Y-%m')        AS cal_ym,
    CASE WHEN EXTRACT(dow FROM ts) IN (0,6) THEN 1 ELSE 0 END AS cal_is_weekend
  FROM spine
),
-- Forecast weather already pivoted wide (your sample shows columns like temp_the_woodlands_tx)
wx_forecast AS (
  SELECT
    DATE_TRUNC('hour', "timestamp") AS ts,
    interval AS wx_interval,
    -- include the columns you have; sample starts with temp_* — add wind/hum/precip columns too if present
    *
  FROM vw_forecast_weather_by_city
),
-- Historical weather: if you also have pivoted wide columns, this is fine.
-- If your historical view is LONG (city rows), replace this CTE with an aggregate-pivot (see note below).
wx_historical AS (
  SELECT
    DATE_TRUNC('hour', "timestamp") AS ts,
    interval AS hist_interval,
    *
  FROM vw_historical_weather_by_city
),
prices AS (
  SELECT
    DATE_TRUNC('hour', OperatingDTM) AS ts,
    hb_houston
  FROM vw_prices_by_hub
)
SELECT
  -- spine (load forecast) = source of truth timeline
  s.*,
  -- calendar features
  cf.* EXCLUDE (ts),
  -- forecast weather (left join, future-known)
  wf.* EXCLUDE (ts),
  -- historical weather (left join, past-only)
  wh.* EXCLUDE (ts),
  -- prices (left join; will be NULL for future)
  p.hb_houston AS price_hb_houston
FROM spine s
LEFT JOIN calendar_feats cf USING (ts)
LEFT JOIN wx_forecast   wf USING (ts)
LEFT JOIN wx_historical wh USING (ts)
LEFT JOIN prices        p  USING (ts)
ORDER BY s.ts;

                 """).fetchdf()

print(df)

con.close()

Empty DataFrame
Columns: [Count]
Index: []


Export last X days to csv for adhoc analysis

In [85]:
import duckdb
import pandas as pd
from pathlib import Path

# --- config ---
DB_PATH = "../ProjectMain/db/data.duckdb"   # adjust if different
OUT_PATH = Path("last20_days_dataset.csv")

# --- connect ---
con = duckdb.connect(DB_PATH)

# 1. Find last 20 operating dates in vw_spine_load_wx_price
dates_query = """
SELECT DISTINCT DATE(ts) AS operating_date
FROM vw_spine_load_wx_price
ORDER BY operating_date DESC
LIMIT 20;
"""
last20_dates = con.execute(dates_query).fetchdf()
date_list = last20_dates["operating_date"].tolist()

# 2. Query the full dataset for those dates
# DuckDB accepts a list via VALUES
query = f"""
SELECT *
FROM vw_spine_load_wx_price
WHERE DATE(ts) IN (
    SELECT UNNEST(LIST_VALUE({','.join([f"DATE '{d}'" for d in date_list])}))
)
ORDER BY ts;
"""

df = con.execute(query).fetchdf()

# 3. Export to CSV
df.to_csv(OUT_PATH, index=False)
print(f"Exported {len(df)} rows covering {len(date_list)} days to {OUT_PATH}")

con.close()

Exported 210240 rows covering 20 days to last20_days_dataset.csv


Not a great file lets start over

In [10]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""SELECT
    lf.*,
    -- calendar features from the date
    EXTRACT(year  FROM OperatingDTM)::INT  AS cal_year,
    EXTRACT(month FROM OperatingDTM)::INT  AS cal_month,
    EXTRACT(dow   FROM OperatingDTM)::INT  AS cal_dow,    -- 0=Sunday .. 6=Saturday
    CASE WHEN EXTRACT(dow FROM OperatingDTM) IN (0,6) THEN 1 ELSE 0 END AS cal_is_weekend,

    -- hour features from Interval
    Interval AS cal_hour,
    ((EXTRACT(dow FROM OperatingDTM)::INT) * 24 + (Interval - 1)) AS cal_hour_of_week,

    -- optional cyclic encodings
    SIN(2*PI() * Interval/24.0)                  AS cal_sin_hour,
    COS(2*PI() * Interval/24.0)                  AS cal_cos_hour,
    SIN(2*PI() * EXTRACT(dow FROM OperatingDTM)/7.0) AS cal_sin_dow,
    COS(2*PI() * EXTRACT(dow FROM OperatingDTM)/7.0) AS cal_cos_dow

FROM vw_loadforecast_by_zone lf
WHERE OperatingDTM = DATE '2025-08-18'
ORDER BY Interval;

                 """).fetchdf()
print(df)
#print(df.describe())
#print(df.info)

con.close()

   OperatingDTM  Interval   Month DSTFlag  wz_southcentral    wz_east  \
0    2025-08-18         1  8-2025       N       10345.2998  1994.8000   
1    2025-08-18         2  8-2025       N        9684.5996  1886.0500   
2    2025-08-18         3  8-2025       N        9151.6699  1806.5300   
3    2025-08-18         4  8-2025       N        8843.3496  1754.6300   
4    2025-08-18         5  8-2025       N        8732.6504  1729.5000   
5    2025-08-18         6  8-2025       N        8838.7402  1767.5000   
6    2025-08-18         7  8-2025       N        9106.2305  1845.3700   
7    2025-08-18         8  8-2025       N        9247.9404  1864.5200   
8    2025-08-18         9  8-2025       N        9665.3301  1984.9800   
9    2025-08-18        10  8-2025       N       10330.2002  2191.4299   
10   2025-08-18        11  8-2025       N       11081.2998  2370.3401   
11   2025-08-18        12  8-2025       N       11862.7002  2560.0701   
12   2025-08-18        13  8-2025       N       126

Okay that works for me. Lets add the left join of the vw_forecast_weather_by_city

In [17]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 select * from vw_forecast_weather_by_city  order by 1 desc,2 desc Limit 25
""").fetchdf()

print(df)

con.close()

   OperatingDTM           timestamp  interval  temp_the_woodlands_tx  \
0    2025-08-24 2025-08-25 00:00:00        24                   30.0   
1    2025-08-24 2025-08-24 23:00:00        23                   30.7   
2    2025-08-24 2025-08-24 22:00:00        22                   31.4   
3    2025-08-24 2025-08-24 21:00:00        21                   32.3   
4    2025-08-24 2025-08-24 20:00:00        20                   33.3   
5    2025-08-24 2025-08-24 19:00:00        19                   34.7   
6    2025-08-24 2025-08-24 18:00:00        18                   36.3   
7    2025-08-24 2025-08-24 17:00:00        17                   37.2   
8    2025-08-24 2025-08-24 16:00:00        16                   37.0   
9    2025-08-24 2025-08-24 15:00:00        15                   36.2   
10   2025-08-24 2025-08-24 14:00:00        14                   35.1   
11   2025-08-24 2025-08-24 13:00:00        13                   33.6   
12   2025-08-24 2025-08-24 12:00:00        12                   

Joining forecast weather onto forecast MW + calendar features query:

In [18]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
SELECT
    lf.*,

    -- calendar features from the date
    EXTRACT(year  FROM lf.OperatingDTM)::INT  AS cal_year,
    EXTRACT(month FROM lf.OperatingDTM)::INT  AS cal_month,
    EXTRACT(dow   FROM lf.OperatingDTM)::INT  AS cal_dow,    -- 0=Sunday .. 6=Saturday
    CASE WHEN EXTRACT(dow FROM lf.OperatingDTM) IN (0,6) THEN 1 ELSE 0 END AS cal_is_weekend,

    -- hour features from Interval (since OperatingDTM is a date)
    lf.Interval AS cal_hour,
    ((EXTRACT(dow FROM lf.OperatingDTM)::INT) * 24 + (lf.Interval - 1)) AS cal_hour_of_week,

    -- optional cyclic encodings
    SIN(2*PI() * lf.Interval/24.0)                        AS cal_sin_hour,
    COS(2*PI() * lf.Interval/24.0)                        AS cal_cos_hour,
    SIN(2*PI() * EXTRACT(dow FROM lf.OperatingDTM)/7.0)   AS cal_sin_dow,
    COS(2*PI() * EXTRACT(dow FROM lf.OperatingDTM)/7.0)   AS cal_cos_dow,

    -- forecast weather columns (already pivoted wide). Exclude join keys.
    wfc.* EXCLUDE (OperatingDTM, "timestamp", "interval")

FROM vw_loadforecast_by_zone lf
LEFT JOIN vw_forecast_weather_by_city wfc
  ON wfc.OperatingDTM = lf.OperatingDTM
 AND wfc."interval"   = lf.Interval

WHERE lf.OperatingDTM = DATE '2025-08-18'
ORDER BY lf.Interval;
""").fetchdf()

print(df)
con.close()


   OperatingDTM  Interval   Month DSTFlag  wz_southcentral    wz_east  \
0    2025-08-18         1  8-2025       N       10345.2998  1994.8000   
1    2025-08-18         2  8-2025       N        9684.5996  1886.0500   
2    2025-08-18         3  8-2025       N        9151.6699  1806.5300   
3    2025-08-18         4  8-2025       N        8843.3496  1754.6300   
4    2025-08-18         5  8-2025       N        8732.6504  1729.5000   
5    2025-08-18         6  8-2025       N        8838.7402  1767.5000   
6    2025-08-18         7  8-2025       N        9106.2305  1845.3700   
7    2025-08-18         8  8-2025       N        9247.9404  1864.5200   
8    2025-08-18         9  8-2025       N        9665.3301  1984.9800   
9    2025-08-18        10  8-2025       N       10330.2002  2191.4299   
10   2025-08-18        11  8-2025       N       11081.2998  2370.3401   
11   2025-08-18        12  8-2025       N       11862.7002  2560.0701   
12   2025-08-18        13  8-2025       N       126

good, now lets review historic weather data before joining

In [19]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 select * from vw_historical_weather_by_city order by 1 desc,2 desc Limit 2000
""").fetchdf()

print(df)

con.close()

               timestamp  interval  temp_the_woodlands_tx  \
0    2025-08-16 20:00:00        20                   30.0   
1    2025-08-16 19:00:00        19                   31.1   
2    2025-08-16 18:00:00        18                   31.7   
3    2025-08-16 17:00:00        17                   29.4   
4    2025-08-16 16:00:00        16                   27.8   
...                  ...       ...                    ...   
1995 2025-05-25 17:00:00        17                   33.3   
1996 2025-05-25 16:00:00        16                   33.9   
1997 2025-05-25 15:00:00        15                   33.3   
1998 2025-05-25 14:00:00        14                   32.2   
1999 2025-05-25 13:00:00        13                   32.2   

      hum_the_woodlands_tx  wind_the_woodlands_tx  precip_the_woodlands_tx  \
0                     72.0               1.388889                      0.0   
1                     66.0               1.944444                      0.2   
2                     68.0       

make the same update to historic weather data

In [20]:
import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

con.execute("""
CREATE OR REPLACE VIEW vw_historical_weather_by_city AS
WITH labeled AS (
  SELECT
      h."timestamp",
      h."interval",
      /* OperatingDTM logic:
         - If time = 00:00:00, assign to prior operating day
         - Else assign to same day
      */
      CASE
        WHEN STRFTIME(h."timestamp", '%H:%M:%S') = '00:00:00'
          THEN CAST(DATE_TRUNC('day', h."timestamp") - INTERVAL 1 DAY AS DATE)
        ELSE CAST(DATE_TRUNC('day', h."timestamp") AS DATE)
      END AS OperatingDTM,
      h.lat,
      h.lon,
      COALESCE(c.city, 'unmapped') AS city_name,
      REGEXP_REPLACE(LOWER(COALESCE(c.city, 'unmapped')), '[^a-z0-9]+', '_') AS city_key,
      h.temperature,
      h.humidity,
      h.windspeed,
      h.precipitation
  FROM historical_weather h
  LEFT JOIN city_coords c
    ON ROUND(h.lat, 4) = ROUND(c.lat, 4)
   AND ROUND(h.lon, 4) = ROUND(c.lon, 4)
),
latest AS (
  -- If duplicates exist per delivery hour & city, keep the most recent observation
  SELECT *
  FROM (
    SELECT
      labeled.*,
      ROW_NUMBER() OVER (
        PARTITION BY "timestamp", "interval", city_name
        ORDER BY "timestamp" DESC
      ) AS rn
    FROM labeled
  )
  WHERE rn = 1
)
SELECT
  OperatingDTM,
  "timestamp",
  "interval",

  /* The_Woodlands_TX */
  AVG(CASE WHEN city_name='The_Woodlands_TX' THEN temperature   END) AS hist_temp_the_woodlands_tx,
  AVG(CASE WHEN city_name='The_Woodlands_TX' THEN humidity      END) AS hist_hum_the_woodlands_tx,
  AVG(CASE WHEN city_name='The_Woodlands_TX' THEN windspeed     END) AS hist_wind_the_woodlands_tx,
  AVG(CASE WHEN city_name='The_Woodlands_TX' THEN precipitation END) AS hist_precip_the_woodlands_tx,

  /* Katy_TX */
  AVG(CASE WHEN city_name='Katy_TX' THEN temperature   END) AS hist_temp_katy_tx,
  AVG(CASE WHEN city_name='Katy_TX' THEN humidity      END) AS hist_hum_katy_tx,
  AVG(CASE WHEN city_name='Katy_TX' THEN windspeed     END) AS hist_wind_katy_tx,
  AVG(CASE WHEN city_name='Katy_TX' THEN precipitation END) AS hist_precip_katy_tx,

  /* Friendswood_TX */
  AVG(CASE WHEN city_name='Friendswood_TX' THEN temperature   END) AS hist_temp_friendswood_tx,
  AVG(CASE WHEN city_name='Friendswood_TX' THEN humidity      END) AS hist_hum_friendswood_tx,
  AVG(CASE WHEN city_name='Friendswood_TX' THEN windspeed     END) AS hist_wind_friendswood_tx,
  AVG(CASE WHEN city_name='Friendswood_TX' THEN precipitation END) AS hist_precip_friendswood_tx,

  /* Baytown_TX */
  AVG(CASE WHEN city_name='Baytown_TX' THEN temperature   END) AS hist_temp_baytown_tx,
  AVG(CASE WHEN city_name='Baytown_TX' THEN humidity      END) AS hist_hum_baytown_tx,
  AVG(CASE WHEN city_name='Baytown_TX' THEN windspeed     END) AS hist_wind_baytown_tx,
  AVG(CASE WHEN city_name='Baytown_TX' THEN precipitation END) AS hist_precip_baytown_tx,

  /* Houston_TX */
  AVG(CASE WHEN city_name='Houston_TX' THEN temperature   END) AS hist_temp_houston_tx,
  AVG(CASE WHEN city_name='Houston_TX' THEN humidity      END) AS hist_hum_houston_tx,
  AVG(CASE WHEN city_name='Houston_TX' THEN windspeed     END) AS hist_wind_houston_tx,
  AVG(CASE WHEN city_name='Houston_TX' THEN precipitation END) AS hist_precip_houston_tx

FROM latest
GROUP BY OperatingDTM, "timestamp", "interval"
ORDER BY OperatingDTM, "timestamp", "interval";
""")

print("View vw_historical_weather_by_city updated with OperatingDTM (hist_* columns).")
con.close()


View vw_historical_weather_by_city updated with OperatingDTM (hist_* columns).


In [21]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 select * from vw_historical_weather_by_city  order by 1 desc,2 desc Limit 25
""").fetchdf()

print(df)

con.close()

   OperatingDTM           timestamp  interval  hist_temp_the_woodlands_tx  \
0    2025-08-16 2025-08-16 20:00:00        20                        30.0   
1    2025-08-16 2025-08-16 19:00:00        19                        31.1   
2    2025-08-16 2025-08-16 18:00:00        18                        31.7   
3    2025-08-16 2025-08-16 17:00:00        17                        29.4   
4    2025-08-16 2025-08-16 16:00:00        16                        27.8   
5    2025-08-16 2025-08-16 15:00:00        15                        26.1   
6    2025-08-16 2025-08-16 14:00:00        14                        26.1   
7    2025-08-16 2025-08-16 13:00:00        13                        26.7   
8    2025-08-16 2025-08-16 12:00:00        12                        28.0   
9    2025-08-16 2025-08-16 11:00:00        11                        31.1   
10   2025-08-16 2025-08-16 10:00:00        10                        31.1   
11   2025-08-16 2025-08-16 09:00:00         9                        27.2   

Def a problem with the querying of the historic weather data. Runnign it through the ui is now throwing an error.
 SOlving that in a different workflow while pressing on with the join of this view to the master spine

In [22]:
import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

df = con.execute("""
SELECT
    lf.*,

    -- calendar features from the date
    EXTRACT(year  FROM lf.OperatingDTM)::INT  AS cal_year,
    EXTRACT(month FROM lf.OperatingDTM)::INT  AS cal_month,
    EXTRACT(dow   FROM lf.OperatingDTM)::INT  AS cal_dow,    -- 0=Sunday .. 6=Saturday
    CASE WHEN EXTRACT(dow FROM lf.OperatingDTM) IN (0,6) THEN 1 ELSE 0 END AS cal_is_weekend,

    -- hour features from Interval (since OperatingDTM is a date)
    lf.Interval AS cal_hour,
    ((EXTRACT(dow FROM lf.OperatingDTM)::INT) * 24 + (lf.Interval - 1)) AS cal_hour_of_week,

    -- optional cyclic encodings
    SIN(2*PI() * lf.Interval/24.0)                        AS cal_sin_hour,
    COS(2*PI() * lf.Interval/24.0)                        AS cal_cos_hour,
    SIN(2*PI() * EXTRACT(dow FROM lf.OperatingDTM)/7.0)   AS cal_sin_dow,
    COS(2*PI() * EXTRACT(dow FROM lf.OperatingDTM)/7.0)   AS cal_cos_dow,

    -- forecast weather (future-known), exclude join keys
    wfc.* EXCLUDE (OperatingDTM, "timestamp", "interval"),

    -- historical weather (past-only), exclude join keys
    whc.* EXCLUDE (OperatingDTM, "timestamp", "interval")

FROM vw_loadforecast_by_zone lf
LEFT JOIN vw_forecast_weather_by_city wfc
  ON wfc.OperatingDTM = lf.OperatingDTM
 AND wfc."interval"   = lf.Interval
LEFT JOIN vw_historical_weather_by_city whc
  ON whc.OperatingDTM = lf.OperatingDTM
 AND whc."interval"   = lf.Interval

WHERE lf.OperatingDTM = DATE '2025-08-15'
ORDER BY lf.Interval;
""").fetchdf()

print(df)
con.close()


   OperatingDTM  Interval   Month DSTFlag  wz_southcentral    wz_east  \
0    2025-08-15         1  8-2025       N       10968.5996  2131.9399   
1    2025-08-15         2  8-2025       N       10136.5000  1983.5100   
2    2025-08-15         3  8-2025       N        9746.1602  1866.6100   
3    2025-08-15         4  8-2025       N        9462.0498  1844.2700   
4    2025-08-15         5  8-2025       N        9396.8799  1800.1600   
5    2025-08-15         6  8-2025       N        9425.6602  1830.5500   
6    2025-08-15         7  8-2025       N        9703.2002  1915.6700   
7    2025-08-15         8  8-2025       N        9848.6104  1929.8101   
8    2025-08-15         9  8-2025       N       10091.0000  2020.5000   
9    2025-08-15        10  8-2025       N       11048.2998  2216.1599   
10   2025-08-15        11  8-2025       N       11824.2002  2570.5300   
11   2025-08-15        12  8-2025       N       12548.4004  2846.6499   
12   2025-08-15        13  8-2025       N       131

checking prices one last time before joining

In [23]:
import duckdb

# Connect to your DuckDB file
con = duckdb.connect("../ProjectMain/db/data.duckdb")

# Run query and fetch results
df = con.execute("""
                 select * from vw_prices_by_hub   order by 1 desc,2 desc Limit 25
""").fetchdf()

print(df)

con.close()

   OperatingDTM  Interval  hb_hubavg  hb_west  hb_north  hb_south  hb_houston  \
0    2025-08-18        24      46.10    49.61     44.36     45.52       44.92   
1    2025-08-18        23      75.00    78.87     74.11     73.12       73.90   
2    2025-08-18        22     136.14   141.17    136.00    132.82      134.57   
3    2025-08-18        21     218.71   227.49    221.14    210.66      215.58   
4    2025-08-18        20     228.82   241.16    234.59    215.28      224.23   
5    2025-08-18        19     116.74   121.95    115.00    116.83      113.16   
6    2025-08-18        18      83.97    83.56     74.97     96.08       81.29   
7    2025-08-18        17      67.79    63.56     55.56     85.93       66.11   
8    2025-08-18        16      54.50    48.02     41.33     74.15       54.52   
9    2025-08-18        15      50.26    43.28     39.33     67.55       50.88   
10   2025-08-18        14      42.72    36.68     33.58     57.31       43.30   
11   2025-08-18        13   

great now lets join the prices on to the master spine view

In [26]:
import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

# --- Minimal: add only Houston hub price ---
df = con.execute("""
SELECT
    lf.*,

    -- calendar features from the date
    EXTRACT(year  FROM lf.OperatingDTM)::INT  AS cal_year,
    EXTRACT(month FROM lf.OperatingDTM)::INT  AS cal_month,
    EXTRACT(dow   FROM lf.OperatingDTM)::INT  AS cal_dow,    -- 0=Sun..6=Sat
    CASE WHEN EXTRACT(dow FROM lf.OperatingDTM) IN (0,6) THEN 1 ELSE 0 END AS cal_is_weekend,

    -- hour features from Interval (since OperatingDTM is a date)
    lf.Interval AS cal_hour,
    ((EXTRACT(dow FROM lf.OperatingDTM)::INT) * 24 + (lf.Interval - 1)) AS cal_hour_of_week,

    -- optional cyclic encodings
    SIN(2*PI() * lf.Interval/24.0)                      AS cal_sin_hour,
    COS(2*PI() * lf.Interval/24.0)                      AS cal_cos_hour,
    SIN(2*PI() * EXTRACT(dow FROM lf.OperatingDTM)/7.0) AS cal_sin_dow,
    COS(2*PI() * EXTRACT(dow FROM lf.OperatingDTM)/7.0) AS cal_cos_dow,

    -- forecast weather (exclude join keys)
    wfc.* EXCLUDE (OperatingDTM, "timestamp", "interval"),

    -- historical weather (exclude join keys)
    whc.* EXCLUDE (OperatingDTM, "timestamp", "interval"),

    -- prices (join on OperatingDTM + Interval)
    p.* EXCLUDE (OperatingDTM, Interval)

FROM vw_loadforecast_by_zone lf
LEFT JOIN vw_forecast_weather_by_city    wfc
  ON wfc.OperatingDTM = lf.OperatingDTM
 AND wfc."interval"   = lf.Interval
LEFT JOIN vw_historical_weather_by_city  whc
  ON whc.OperatingDTM = lf.OperatingDTM
 AND whc."interval"   = lf.Interval
LEFT JOIN vw_prices_by_hub               p
  ON p.OperatingDTM   = lf.OperatingDTM
 AND p.Interval       = lf.Interval

WHERE lf.OperatingDTM = DATE '2025-08-16'
ORDER BY lf.Interval;
""").fetchdf()

print(df)
con.close()


   OperatingDTM  Interval   Month DSTFlag  wz_southcentral    wz_east  \
0    2025-08-16         1  8-2025       N       10012.4004  2085.3000   
1    2025-08-16         2  8-2025       N        9431.2402  1985.4600   
2    2025-08-16         3  8-2025       N        8878.2305  1870.9000   
3    2025-08-16         4  8-2025       N        8570.9004  1847.8400   
4    2025-08-16         5  8-2025       N        8444.1797  1770.3000   
5    2025-08-16         6  8-2025       N        8272.6104  1753.5100   
6    2025-08-16         7  8-2025       N        8256.2197  1747.1600   
7    2025-08-16         8  8-2025       N        8603.3496  1798.7300   
8    2025-08-16         9  8-2025       N        8905.4502  1916.1801   
9    2025-08-16        10  8-2025       N        9686.9004  2184.7300   
10   2025-08-16        11  8-2025       N       10357.2002  2301.9700   
11   2025-08-16        12  8-2025       N       10838.9004  2404.2600   
12   2025-08-16        13  8-2025       N       113

Final Query for training: 

In [27]:
import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

con.execute("""
CREATE OR REPLACE VIEW vw_master_spine AS
SELECT
    lf.*,

    -- calendar features from the date
    EXTRACT(year  FROM lf.OperatingDTM)::INT  AS cal_year,
    EXTRACT(month FROM lf.OperatingDTM)::INT  AS cal_month,
    EXTRACT(dow   FROM lf.OperatingDTM)::INT  AS cal_dow,    -- 0=Sun..6=Sat
    CASE WHEN EXTRACT(dow FROM lf.OperatingDTM) IN (0,6) THEN 1 ELSE 0 END AS cal_is_weekend,

    -- hour features from Interval (since OperatingDTM is a date)
    lf.Interval AS cal_hour,
    ((EXTRACT(dow FROM lf.OperatingDTM)::INT) * 24 + (lf.Interval - 1)) AS cal_hour_of_week,

    -- optional cyclic encodings
    SIN(2*PI() * lf.Interval/24.0)                      AS cal_sin_hour,
    COS(2*PI() * lf.Interval/24.0)                      AS cal_cos_hour,
    SIN(2*PI() * EXTRACT(dow FROM lf.OperatingDTM)/7.0) AS cal_sin_dow,
    COS(2*PI() * EXTRACT(dow FROM lf.OperatingDTM)/7.0) AS cal_cos_dow,

    -- forecast weather (exclude join keys)
    wfc.* EXCLUDE (OperatingDTM, "timestamp", "interval"),

    -- historical weather (exclude join keys)
    whc.* EXCLUDE (OperatingDTM, "timestamp", "interval"),

    -- prices (exclude join keys; brings all hub/zone price columns)
    p.* EXCLUDE (OperatingDTM, Interval)

FROM vw_loadforecast_by_zone lf
LEFT JOIN vw_forecast_weather_by_city    wfc
  ON wfc.OperatingDTM = lf.OperatingDTM
 AND wfc."interval"   = lf.Interval
LEFT JOIN vw_historical_weather_by_city  whc
  ON whc.OperatingDTM = lf.OperatingDTM
 AND whc."interval"   = lf.Interval
LEFT JOIN vw_prices_by_hub               p
  ON p.OperatingDTM   = lf.OperatingDTM
 AND p.Interval       = lf.Interval;
""")

print("Created view vw_master_spine")
con.close()


Created view vw_master_spine


single day query

In [29]:
import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

df = con.execute("""
SELECT * FROM vw_master_spine
WHERE OperatingDTM = DATE '2025-08-16'
ORDER BY Interval;

""").fetchdf()

print(df)
con.close()


   OperatingDTM  Interval   Month DSTFlag  wz_southcentral    wz_east  \
0    2025-08-16         1  8-2025       N       10012.4004  2085.3000   
1    2025-08-16         2  8-2025       N        9431.2402  1985.4600   
2    2025-08-16         3  8-2025       N        8878.2305  1870.9000   
3    2025-08-16         4  8-2025       N        8570.9004  1847.8400   
4    2025-08-16         5  8-2025       N        8444.1797  1770.3000   
5    2025-08-16         6  8-2025       N        8272.6104  1753.5100   
6    2025-08-16         7  8-2025       N        8256.2197  1747.1600   
7    2025-08-16         8  8-2025       N        8603.3496  1798.7300   
8    2025-08-16         9  8-2025       N        8905.4502  1916.1801   
9    2025-08-16        10  8-2025       N        9686.9004  2184.7300   
10   2025-08-16        11  8-2025       N       10357.2002  2301.9700   
11   2025-08-16        12  8-2025       N       10838.9004  2404.2600   
12   2025-08-16        13  8-2025       N       113

last X Op Dates

In [30]:


import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

df = con.execute("""
SELECT ms.*
FROM vw_master_spine ms
JOIN (
  SELECT DISTINCT OperatingDTM
  FROM vw_master_spine
  ORDER BY OperatingDTM DESC
  LIMIT 20
) d USING (OperatingDTM)
ORDER BY ms.OperatingDTM, ms.Interval;

""").fetchdf()

print(df)
con.close()



    OperatingDTM  Interval   Month DSTFlag  wz_southcentral    wz_east  \
0     2025-08-04         1  8-2025       N       10777.4004  1983.6000   
1     2025-08-04         2  8-2025       N        9904.2803  1771.8500   
2     2025-08-04         3  8-2025       N        9372.2900  1682.3199   
3     2025-08-04         4  8-2025       N        9024.0596  1620.0400   
4     2025-08-04         5  8-2025       N        8870.4102  1622.1500   
..           ...       ...     ...     ...              ...        ...   
475   2025-08-23        20  8-2025       N       12319.8994  2547.4299   
476   2025-08-23        21  8-2025       N       11860.6748  2438.8701   
477   2025-08-23        22  8-2025       N       11418.9746  2320.7075   
478   2025-08-23        23  8-2025       N       10797.6992  2187.1775   
479   2025-08-23        24  8-2025       N       10105.1426  2033.3350   

       wz_west  wz_northcentral  wz_farwest   wz_north  ...  \
0    1642.8500       16235.0996   7639.3398  201

Now lets make a lagged and rolled price view adding a timestamp back to the master view

In [31]:


import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

con.execute("""
CREATE OR REPLACE VIEW vw_master_spine_ts AS
SELECT
  ms.*,
  CAST(ms.OperatingDTM AS TIMESTAMP) + (ms.Interval - 1) * INTERVAL 1 HOUR AS ts
FROM vw_master_spine ms;

""")

print(df)
con.close()


    OperatingDTM  Interval   Month DSTFlag  wz_southcentral    wz_east  \
0     2025-08-04         1  8-2025       N       10777.4004  1983.6000   
1     2025-08-04         2  8-2025       N        9904.2803  1771.8500   
2     2025-08-04         3  8-2025       N        9372.2900  1682.3199   
3     2025-08-04         4  8-2025       N        9024.0596  1620.0400   
4     2025-08-04         5  8-2025       N        8870.4102  1622.1500   
..           ...       ...     ...     ...              ...        ...   
475   2025-08-23        20  8-2025       N       12319.8994  2547.4299   
476   2025-08-23        21  8-2025       N       11860.6748  2438.8701   
477   2025-08-23        22  8-2025       N       11418.9746  2320.7075   
478   2025-08-23        23  8-2025       N       10797.6992  2187.1775   
479   2025-08-23        24  8-2025       N       10105.1426  2033.3350   

       wz_west  wz_northcentral  wz_farwest   wz_north  ...  \
0    1642.8500       16235.0996   7639.3398  201

In [32]:


import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

con.execute("""
CREATE OR REPLACE VIEW vw_master_spine_lags AS
SELECT
  t.*,
  LAG(hb_houston, 1)   OVER (ORDER BY ts) AS p_lag1,
  LAG(hb_houston, 2)   OVER (ORDER BY ts) AS p_lag2,
  LAG(hb_houston, 3)   OVER (ORDER BY ts) AS p_lag3,
  LAG(hb_houston, 6)   OVER (ORDER BY ts) AS p_lag6,
  LAG(hb_houston, 12)  OVER (ORDER BY ts) AS p_lag12,
  LAG(hb_houston, 24)  OVER (ORDER BY ts) AS p_lag24,
  LAG(hb_houston, 48)  OVER (ORDER BY ts) AS p_lag48,
  LAG(hb_houston, 72)  OVER (ORDER BY ts) AS p_lag72,
  LAG(hb_houston, 168) OVER (ORDER BY ts) AS p_lag168,
  hb_houston - LAG(hb_houston, 1)  OVER (ORDER BY ts) AS dp1,
  hb_houston - LAG(hb_houston, 24) OVER (ORDER BY ts) AS dp24,
  AVG(hb_houston) OVER (ORDER BY ts ROWS BETWEEN 23 PRECEDING AND CURRENT ROW)  AS p_roll24_mean,
  STDDEV_SAMP(hb_houston) OVER (ORDER BY ts ROWS BETWEEN 23 PRECEDING AND CURRENT ROW) AS p_roll24_std,
  AVG(hb_houston) OVER (ORDER BY ts ROWS BETWEEN 71 PRECEDING AND CURRENT ROW)  AS p_roll72_mean,
  AVG(hb_houston) OVER (ORDER BY ts ROWS BETWEEN 167 PRECEDING AND CURRENT ROW) AS p_roll168_mean
FROM vw_master_spine_ts t
ORDER BY ts;
""")

print(df)
con.close()


    OperatingDTM  Interval   Month DSTFlag  wz_southcentral    wz_east  \
0     2025-08-04         1  8-2025       N       10777.4004  1983.6000   
1     2025-08-04         2  8-2025       N        9904.2803  1771.8500   
2     2025-08-04         3  8-2025       N        9372.2900  1682.3199   
3     2025-08-04         4  8-2025       N        9024.0596  1620.0400   
4     2025-08-04         5  8-2025       N        8870.4102  1622.1500   
..           ...       ...     ...     ...              ...        ...   
475   2025-08-23        20  8-2025       N       12319.8994  2547.4299   
476   2025-08-23        21  8-2025       N       11860.6748  2438.8701   
477   2025-08-23        22  8-2025       N       11418.9746  2320.7075   
478   2025-08-23        23  8-2025       N       10797.6992  2187.1775   
479   2025-08-23        24  8-2025       N       10105.1426  2033.3350   

       wz_west  wz_northcentral  wz_farwest   wz_north  ...  \
0    1642.8500       16235.0996   7639.3398  201

In [34]:
#########           sanity checks
import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

df = con.execute("""
-- 24 rows per day?
SELECT OperatingDTM, COUNT(*) AS n
FROM vw_master_spine_ts
GROUP BY 1 ORDER BY 1 DESC;


""").fetchdf()

print(df)
con.close()






    OperatingDTM   n
0     2025-08-23  24
1     2025-08-22  24
2     2025-08-21  24
3     2025-08-20  24
4     2025-08-19  24
..           ...  ..
961   2023-01-05  24
962   2023-01-04  24
963   2023-01-03  24
964   2023-01-02  24
965   2023-01-01  24

[966 rows x 2 columns]


In [35]:
import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

df = con.execute("""
-- 1 row per (OperatingDTM, Interval)?
SELECT OperatingDTM, Interval, COUNT(*) AS n
FROM vw_master_spine_ts
GROUP BY 1,2 HAVING COUNT(*)>1;


""").fetchdf()

print(df)
con.close()


  OperatingDTM  Interval  n
0   2024-11-03         2  2
1   2023-11-05         2  2


In [26]:

import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

df = con.execute("""

-- any null prices inside the historical period?
SELECT COUNT(*) FROM vw_master_spine_ts WHERE hb_houston IS NULL AND ts <= (SELECT MAX(ts) FROM vw_master_spine_ts WHERE hb_houston IS NOT NULL);


""").fetchdf()
print(df)
con.close()


   count_star()
0             0


Now we have two new additons to the master spine view: 
# vw_master_spine_ts 
# vw_master_spine_lags 

Next, lets build horizon-specific datasets (future-known covariates shifted by H):

For each horizon H = 1..24, we’ll:

Use recency features at time t from vw_master_spine_lags (b).

Join future-known features (load forecast + forecast weather + calendar at the delivery hour) from the same view shifted to t+H (f).

Create the label y_hH = LEAD(hb_houston, H).

In [2]:
import duckdb, pandas as pd, numpy as np
from pathlib import Path

DB = "../ProjectMain/db/data.duckdb"

# columns to fetch as FUTURE covariates (from 'f' alias; adjust to your exact columns)
FUTURE_LOAD_COLS = [
    "wz_southcentral","wz_east","wz_west","wz_northcentral","wz_farwest","wz_north","wz_southern","wz_coast",
    "lz_north","lz_west","lz_south","lz_houston"
]
FUTURE_WX_COLS = [
    "temp_the_woodlands_tx","hum_the_woodlands_tx","wind_the_woodlands_tx","precip_the_woodlands_tx",
    "temp_katy_tx","hum_katy_tx","wind_katy_tx","precip_katy_tx",
    "temp_friendswood_tx","hum_friendswood_tx","wind_friendswood_tx","precip_friendswood_tx",
    "temp_baytown_tx","hum_baytown_tx","wind_baytown_tx","precip_baytown_tx",
    "temp_houston_tx","hum_houston_tx","wind_houston_tx","precip_houston_tx"
]
FUTURE_CAL_COLS = ["cal_hour","cal_dow","cal_is_weekend","cal_sin_hour","cal_cos_hour","cal_sin_dow","cal_cos_dow"]

RECENCY_COLS = [
    "p_lag1","p_lag2","p_lag3","p_lag6","p_lag12","p_lag24","p_lag48","p_lag72","p_lag168",
    "dp1","dp24","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean",
    # historical weather recency (at t)
] + [c for c in FUTURE_WX_COLS if c.startswith("hist_")]  # if you created hist_* columns in master

def fetch_horizon_df(h: int) -> pd.DataFrame:
    con = duckdb.connect(DB)
    # Build select lists
    future_select = ",\n       ".join([f"f.{c} AS {c}_f" for c in FUTURE_LOAD_COLS + FUTURE_WX_COLS + FUTURE_CAL_COLS])
    recency_select = ",\n       ".join([f"b.{c}" for c in RECENCY_COLS])
    q = f"""
    WITH base AS (
      SELECT *
      FROM vw_master_spine_lags
    ),
    future AS (
      SELECT *
      FROM vw_master_spine_ts
    )
    SELECT
      b.ts, b.OperatingDTM, b.Interval,
      b.hb_houston,            -- current observed price (for reference)
      {recency_select},
      {future_select},
      LEAD(b.hb_houston, {h}) OVER (ORDER BY b.ts) AS target
    FROM base b
    LEFT JOIN future f
      ON f.ts = b.ts + INTERVAL {h} HOUR
    ORDER BY b.ts;
    """
    df = con.execute(q).df()
    con.close()
    # Drop rows with missing target or missing future features
    df = df.dropna().reset_index(drop=True)
    return df


finally, we're ready to do introductory ML baselining

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score, mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import numpy as np

def eval_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    try:
        rmse = root_mean_squared_error(y_true, y_pred)  # newer sklearn
    except TypeError:
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))        # fallback
    mape = float(np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), 1e-6, None))) * 100)
    r2 = r2_score(y_true, y_pred)
    return {"MAE": mae, "RMSE": rmse, "MAPE%": mape, "R2": r2}

MODELS = {
    "Linear": LinearRegression(),
    "Ridge":  Ridge(alpha=5.0),
    "Lasso":  Lasso(alpha=0.001, max_iter=20000),
    "XGB":    XGBRegressor(n_estimators=800, learning_rate=0.05, max_depth=6,
                           subsample=0.9, colsample_bytree=0.9, random_state=42),
    "LGBM":   LGBMRegressor(n_estimators=1200, learning_rate=0.05, num_leaves=127,
                            subsample=0.9, colsample_bytree=0.9, random_state=42),
}

def train_horizon(h: int):
    df = fetch_horizon_df(h)
    # Feature matrix
    drop_cols = ["ts","OperatingDTM","Interval","hb_houston","target"]
    X = df.drop(columns=drop_cols, errors="ignore")
    y = df["target"].values

    # Walk-forward split (3 folds example)
    tscv = TimeSeriesSplit(n_splits=3)
    results = {}
    for name, mdl in MODELS.items():
        fold_scores = []
        for tr_idx, te_idx in tscv.split(X):
            Xtr, Xte = X.iloc[tr_idx], X.iloc[te_idx]
            ytr, yte = y[tr_idx], y[te_idx]

            if name in ("Linear","Ridge","Lasso"):
                scaler = StandardScaler()
                Xtr2 = scaler.fit_transform(Xtr)
                Xte2 = scaler.transform(Xte)
                mdl.fit(Xtr2, ytr)
                yhat = mdl.predict(Xte2)
            else:
                mdl.fit(Xtr, ytr)
                yhat = mdl.predict(Xte)

            fold_scores.append(eval_metrics(yte, yhat))
        # average across folds
        results[name] = {
            k: float(np.mean([fs[k] for fs in fold_scores])) for k in fold_scores[0].keys()
        }
    return results

# Run for horizons 1..24
all_metrics = {h: train_horizon(h) for h in range(1,25)}
for h in range(1,25):
    best = min(all_metrics[h].items(), key=lambda kv: kv[1]["RMSE"])
    print(f"H{h:02d} best: {best[0]}  →  {best[1]}")


In [6]:
import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

df = con.execute("""
-- For a chosen horizon H (say H=6), verify that future calendar aligns
WITH b AS (
  SELECT ts, OperatingDTM, Interval
  FROM vw_master_spine_lags
  WHERE hb_houston IS NOT NULL
  ORDER BY ts
  LIMIT 200
),
f AS (
  SELECT ts, OperatingDTM AS f_operatingdtm, Interval AS f_interval, cal_hour AS f_cal_hour
  FROM vw_master_spine_ts
)
SELECT
  b.ts,
  b.OperatingDTM, b.Interval,
  f.f_operatingdtm, f.f_interval, f.f_cal_hour
FROM b
LEFT JOIN f
  ON f.ts = b.ts + INTERVAL 6 HOUR
WHERE (b.Interval + 6 - 1) % 24 + 1 <> f.f_cal_hour
LIMIT 5;

""").fetchdf()
print(df)
con.close()

Empty DataFrame
Columns: [ts, OperatingDTM, Interval, f_operatingdtm, f_interval, f_cal_hour]
Index: []


In [7]:
import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

df = con.execute("""

CREATE OR REPLACE VIEW load_fcst_enh AS
WITH z AS (
  SELECT
    OperatingDTM, Interval,
    (wz_southcentral + wz_east + wz_west + wz_northcentral + wz_farwest + wz_north + wz_southern + wz_coast) AS net_wz,
    (lz_north + lz_west + lz_south + lz_houston) AS net_lz,
    GREATEST(wz_southcentral, wz_east, wz_west, wz_northcentral, wz_farwest, wz_north, wz_southern, wz_coast) AS wz_max,
    LEAST(  wz_southcentral, wz_east, wz_west, wz_northcentral, wz_farwest, wz_north, wz_southern, wz_coast) AS wz_min
  FROM vw_loadforecast_by_zone
)
SELECT
  *,
  net_wz - LAG(net_wz) OVER (ORDER BY OperatingDTM, Interval) AS net_wz_ramp1,
  net_lz - LAG(net_lz) OVER (ORDER BY OperatingDTM, Interval) AS net_lz_ramp1,
  (wz_max - wz_min) AS wz_spread
FROM z;
""").fetchdf()
print(df)
con.close()

Empty DataFrame
Columns: [Count]
Index: []


In [8]:
import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

df = con.execute("""

select * from load_fcst_enh
""").fetchdf()
print(df)
con.close()

      OperatingDTM  Interval      net_wz        net_lz      wz_max     wz_min  \
0       2023-01-01         1  35811.1029  35811.102844   9965.3701   758.7730   
1       2023-01-01         2  35059.0604  35059.060486   9831.7500   761.5800   
2       2023-01-01         3  34451.1034  34451.103333   9737.5303   728.3730   
3       2023-01-01         4  34032.5793  34032.579346   9606.1904   724.6090   
4       2023-01-01         5  33818.4683  33818.468384   9510.8096   753.9290   
...            ...       ...         ...           ...         ...        ...   
23178   2025-08-23        20  72789.6773  72789.677368  21937.4258  1751.6400   
23179   2025-08-23        21  70209.0098  70209.009888  20975.9746  1687.8900   
23180   2025-08-23        22  68042.1196  68042.119629  20113.9238  1637.4351   
23181   2025-08-23        23  64982.1933  64982.193359  18937.5488  1535.3601   
23182   2025-08-23        24  61709.0737  61709.073608  17665.5742  1456.1176   

       net_wz_ramp1  net_lz

In [10]:
# --- Day-ahead block backtest (uses your existing fetch_horizon_df) ---
import duckdb, pandas as pd, numpy as np

DB = "../ProjectMain/db/data.duckdb"

def day_ahead_backtest(models_by_h, scalers_by_h=None):
    con = duckdb.connect(DB)
    # Base times = end-of-day hours where we have price and a full next day available
    base_df = con.execute("""
        WITH t AS (
          SELECT ts, OperatingDTM, Interval, hb_houston
          FROM vw_master_spine_lags
          WHERE hb_houston IS NOT NULL
        )
        SELECT *
        FROM t
        WHERE Interval = 24
          AND ts + INTERVAL 24 HOUR <= (SELECT MAX(ts) FROM t)  -- ensure tomorrow exists
        ORDER BY ts
    """).df()
    con.close()

    records = []
    for _, row in base_df.iterrows():
        ts0 = row["ts"]  # end of day
        preds, acts = [], []
        for h in range(1, 25):
            # build one-row features for base ts0 and delivery ts0+h
            # (same logic as in fetch_horizon_df but filtered to ts==ts0)
            q = f"""
            WITH b AS (
              SELECT *
              FROM vw_master_spine_lags
              WHERE ts = TIMESTAMP '{ts0}'
            ),
            f AS (
              SELECT *
              FROM vw_master_spine_ts
              WHERE ts = TIMESTAMP '{ts0}' + INTERVAL {h} HOUR
            )
            SELECT
              b.ts, b.OperatingDTM, b.Interval,
              b.hb_houston,
              -- recency features at t
              p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
              dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean,
              -- (optional) hist_* recency if you added them:
              {", ".join(
                  [c for c in [
                      "hist_temp_the_woodlands_tx","hist_hum_the_woodlands_tx",
                      "hist_wind_the_woodlands_tx","hist_precip_the_woodlands_tx",
                      "hist_temp_katy_tx","hist_hum_katy_tx","hist_wind_katy_tx","hist_precip_katy_tx",
                      "hist_temp_friendswood_tx","hist_hum_friendswood_tx","hist_wind_friendswood_tx","hist_precip_friendswood_tx",
                      "hist_temp_baytown_tx","hist_hum_baytown_tx","hist_wind_baytown_tx","hist_precip_baytown_tx",
                      "hist_temp_houston_tx","hist_hum_houston_tx","hist_wind_houston_tx","hist_precip_houston_tx"
                  ] if c in pd.read_sql("SELECT * FROM vw_master_spine_lags LIMIT 1", duckdb.connect(DB)).columns]
              ) or "/* no hist_* in view */"} 
              ,
              -- future-known covariates at t+h
              f.wz_southcentral AS wz_southcentral_f, f.wz_east AS wz_east_f, f.wz_west AS wz_west_f,
              f.wz_northcentral AS wz_northcentral_f, f.wz_farwest AS wz_farwest_f, f.wz_north AS wz_north_f,
              f.wz_southern AS wz_southern_f, f.wz_coast AS wz_coast_f,
              f.lz_north AS lz_north_f, f.lz_west AS lz_west_f, f.lz_south AS lz_south_f, f.lz_houston AS lz_houston_f,
              f.temp_the_woodlands_tx AS temp_the_woodlands_tx_f, f.hum_the_woodlands_tx AS hum_the_woodlands_tx_f,
              f.wind_the_woodlands_tx AS wind_the_woodlands_tx_f, f.precip_the_woodlands_tx AS precip_the_woodlands_tx_f,
              f.temp_katy_tx AS temp_katy_tx_f, f.hum_katy_tx AS hum_katy_tx_f, f.wind_katy_tx AS wind_katy_tx_f, f.precip_katy_tx AS precip_katy_tx_f,
              f.temp_friendswood_tx AS temp_friendswood_tx_f, f.hum_friendswood_tx AS hum_friendswood_tx_f,
              f.wind_friendswood_tx AS wind_friendswood_tx_f, f.precip_friendswood_tx AS precip_friendswood_tx_f,
              f.temp_baytown_tx AS temp_baytown_tx_f, f.hum_baytown_tx AS hum_baytown_tx_f, f.wind_baytown_tx AS wind_baytown_tx_f, f.precip_baytown_tx AS precip_baytown_tx_f,
              f.temp_houston_tx AS temp_houston_tx_f, f.hum_houston_tx AS hum_houston_tx_f, f.wind_houston_tx AS wind_houston_tx_f, f.precip_houston_tx AS precip_houston_tx_f,
              f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
              f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f, f.cal_sin_dow AS cal_sin_dow_f, f.cal_cos_dow AS cal_cos_dow_f,
              -- label: actual price at t+h
              (SELECT hb_houston FROM vw_master_spine_ts WHERE ts = TIMESTAMP '{ts0}' + INTERVAL {h} HOUR) AS target
            FROM b
            LEFT JOIN f ON TRUE
            """
            con = duckdb.connect(DB)
            feat = con.execute(q).df()
            con.close()

            # prune non-feature columns
            X = feat.drop(columns=[c for c in ["ts","OperatingDTM","Interval","hb_houston","target"] if c in feat.columns], errors="ignore")
            mdl = models_by_h[h]["model"]
            scaler = models_by_h[h].get("scaler")

            if scaler is not None:
                yhat = mdl.predict(scaler.transform(X))
            else:
                yhat = mdl.predict(X)

            preds.append(float(yhat[0]))
            acts.append(float(feat["target"].iloc[0]))

        # day metrics
        mae = float(np.mean(np.abs(np.array(acts) - np.array(preds))))
        rmse = float(np.sqrt(np.mean((np.array(acts) - np.array(preds))**2)))
        records.append({"base_ts": ts0, "day_mae": mae, "day_rmse": rmse})

    return pd.DataFrame(records)


In [11]:
def predict_tomorrow(models_by_h):
    con = duckdb.connect(DB)
    # last hour with a known price (end of today in your data)
    t0 = con.execute("""
        SELECT MAX(ts) AS t0
        FROM vw_master_spine_ts
        WHERE hb_houston IS NOT NULL
    """).fetchone()[0]

    rows = []
    for h in range(1, 25):
        q = f"""
        WITH b AS (SELECT * FROM vw_master_spine_lags WHERE ts = TIMESTAMP '{t0}'),
             f AS (SELECT * FROM vw_master_spine_ts   WHERE ts = TIMESTAMP '{t0}' + INTERVAL {h} HOUR)
        SELECT
          b.OperatingDTM AS base_operatingdtm, b.Interval AS base_interval,
          f.OperatingDTM AS delivery_operatingdtm, f.Interval AS delivery_interval,
          -- features
          p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
          dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean,
          f.wz_southcentral AS wz_southcentral_f, f.wz_east AS wz_east_f, f.wz_west AS wz_west_f,
          f.wz_northcentral AS wz_northcentral_f, f.wz_farwest AS wz_farwest_f, f.wz_north AS wz_north_f,
          f.wz_southern AS wz_southern_f, f.wz_coast AS wz_coast_f,
          f.lz_north AS lz_north_f, f.lz_west AS lz_west_f, f.lz_south AS lz_south_f, f.lz_houston AS lz_houston_f,
          f.temp_the_woodlands_tx AS temp_the_woodlands_tx_f, f.hum_the_woodlands_tx AS hum_the_woodlands_tx_f,
          f.wind_the_woodlands_tx AS wind_the_woodlands_tx_f, f.precip_the_woodlands_tx AS precip_the_woodlands_tx_f,
          f.temp_katy_tx AS temp_katy_tx_f, f.hum_katy_tx AS hum_katy_tx_f, f.wind_katy_tx AS wind_katy_tx_f, f.precip_katy_tx AS precip_katy_tx_f,
          f.temp_friendswood_tx AS temp_friendswood_tx_f, f.hum_friendswood_tx AS hum_friendswood_tx_f,
          f.wind_friendswood_tx AS wind_friendswood_tx_f, f.precip_friendswood_tx AS precip_friendswood_tx_f,
          f.temp_baytown_tx AS temp_baytown_tx_f, f.hum_baytown_tx AS hum_baytown_tx_f, f.wind_baytown_tx AS wind_baytown_tx_f, f.precip_baytown_tx AS precip_baytown_tx_f,
          f.temp_houston_tx AS temp_houston_tx_f, f.hum_houston_tx AS hum_houston_tx_f, f.wind_houston_tx AS wind_houston_tx_f, f.precip_houston_tx AS precip_houston_tx_f,
          f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
          f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f, f.cal_sin_dow AS cal_sin_dow_f, f.cal_cos_dow AS cal_cos_dow_f
        FROM b LEFT JOIN f ON TRUE
        """
        feat = con.execute(q).df()
        X = feat.drop(columns=[c for c in feat.columns if c.endswith("_operatingdtm") or c.endswith("_interval")], errors="ignore")
        mdl = models_by_h[h]["model"]
        scaler = models_by_h[h].get("scaler")
        yhat = mdl.predict(scaler.transform(X) if scaler is not None else X)[0]
        rows.append({
            "OperatingDTM": feat["delivery_operatingdtm"].iloc[0],
            "Interval": int(feat["delivery_interval"].iloc[0]),
            "hb_houston_pred": float(yhat)
        })
    con.close()
    out = pd.DataFrame(rows).sort_values(["OperatingDTM","Interval"]).reset_index(drop=True)
    return out


In [13]:

import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

df = con.execute("""
SELECT COUNT(*) AS n_rows
FROM vw_master_spine_ts
WHERE hb_houston IS NOT NULL;
    """).fetchdf()       

df1 = con.execute("""      
SELECT COUNT(*) AS n_eod
FROM vw_master_spine_ts
WHERE hb_houston IS NOT NULL
  AND Interval = 24;

""").fetchdf()

df2 = con.execute("""      
SELECT MAX(ts) AS last_priced_ts
FROM vw_master_spine_ts
WHERE hb_houston IS NOT NULL;


""").fetchdf()
df3 = con.execute("""      
WITH last AS (
  SELECT DATE(MAX(ts)) AS d
  FROM vw_master_spine_ts
  WHERE hb_houston IS NOT NULL
)
SELECT COUNT(*) AS tomorrow_hours
FROM vw_master_spine_ts f
JOIN last l ON f.OperatingDTM = l.d + INTERVAL 1 DAY;


""").fetchdf()

print(df)
print(df1)
print(df2)
print(df3)
con.close()

   n_rows
0   23063
   n_eod
0    961
       last_priced_ts
0 2025-08-18 23:00:00
   tomorrow_hours
0              24


In [15]:

import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

df = con.execute("""
-- Inspect: day-ahead pairs (base end-of-day -> next day 24 hours)
WITH base AS (
  SELECT ts AS base_ts, OperatingDTM AS base_date, Interval
  FROM vw_master_spine_lags
  WHERE hb_houston IS NOT NULL
    AND Interval = 24
),
future AS (
  SELECT *
  FROM vw_master_spine_ts
)
SELECT
  b.base_ts,
  b.base_date,
  f.OperatingDTM   AS delivery_date,       -- should be base_date + 1 day
  f.Interval       AS delivery_interval,   -- 1..24
  f.hb_houston     AS target_price,        -- for evaluation
  -- recency at base time (add whatever you computed in *_lags)
  -- example:
  -- lags
  -- (You can include more recency cols from vw_master_spine_lags by joining b->lags on base_ts)
  -- future-known covariates at delivery hour (already in future view)
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.temp_the_woodlands_tx, f.hum_the_woodlands_tx, f.wind_the_woodlands_tx, f.precip_the_woodlands_tx,
  f.temp_katy_tx,          f.hum_katy_tx,          f.wind_katy_tx,          f.precip_katy_tx,
  f.temp_friendswood_tx,   f.hum_friendswood_tx,   f.wind_friendswood_tx,   f.precip_friendswood_tx,
  f.temp_baytown_tx,       f.hum_baytown_tx,       f.wind_baytown_tx,       f.precip_baytown_tx,
  f.temp_houston_tx,       f.hum_houston_tx,       f.wind_houston_tx,       f.precip_houston_tx,
  f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f
FROM base b
JOIN future f
  ON f.OperatingDTM = b.base_date + INTERVAL 1 DAY
ORDER BY b.base_ts, f.Interval
LIMIT 120;  -- just to preview

""").fetchdf()
print(df)
con.close()

                base_ts  base_date delivery_date  delivery_interval  \
0   2023-01-01 23:00:00 2023-01-01    2023-01-02                  1   
1   2023-01-01 23:00:00 2023-01-01    2023-01-02                  2   
2   2023-01-01 23:00:00 2023-01-01    2023-01-02                  3   
3   2023-01-01 23:00:00 2023-01-01    2023-01-02                  4   
4   2023-01-01 23:00:00 2023-01-01    2023-01-02                  5   
..                  ...        ...           ...                ...   
115 2023-01-05 23:00:00 2023-01-05    2023-01-06                 20   
116 2023-01-05 23:00:00 2023-01-05    2023-01-06                 21   
117 2023-01-05 23:00:00 2023-01-05    2023-01-06                 22   
118 2023-01-05 23:00:00 2023-01-05    2023-01-06                 23   
119 2023-01-05 23:00:00 2023-01-05    2023-01-06                 24   

     target_price  wz_southcentral  wz_east    wz_west  wz_northcentral  \
0            9.31        5729.4399  1212.96  1037.0500        9694.4102 

In [ ]:
import duckdb, pandas as pd
from datetime import timedelta

DB = "../ProjectMain/db/data.duckdb"

def predict_tomorrow_simple(models_by_h):
    con = duckdb.connect(DB)

    # Last priced hour (base time)
    t0 = con.execute("""
        SELECT MAX(ts) FROM vw_master_spine_ts
        WHERE hb_houston IS NOT NULL
    """).fetchone()[0]
    if t0 is None:
        con.close()
        raise RuntimeError("No priced rows found in vw_master_spine_ts.")

    # Base recency features at t0 (one row)
    b = con.execute("""
        SELECT *
        FROM vw_master_spine_lags
        WHERE ts = ?
    """, [t0]).df()

    # Delivery day = DATE(t0) + 1 day
    base_date = pd.to_datetime(t0).date()
    delivery_date = pd.Timestamp(base_date) + pd.Timedelta(days=1)

    # Future-known features for the 24 delivery hours
    f = con.execute("""
        SELECT *
        FROM vw_master_spine_ts
        WHERE OperatingDTM = ?
        ORDER BY Interval
    """, [delivery_date]).df()

    con.close()

    if f.empty:
        raise RuntimeError(f"No future rows for delivery_date={delivery_date}. "
                           "Check that vw_master_spine_ts has tomorrow's spine rows.")

    rows = []
    for h in range(1, 25):
        feat = pd.concat([b.reset_index(drop=True), f.iloc[[h-1]].reset_index(drop=True)], axis=1)
        X = feat.drop(columns=[c for c in feat.columns
                               if c in ("ts","OperatingDTM","Interval","hb_houston")
                               or c.endswith("_x") or c.endswith("_y")], errors="ignore")
        mdl = models_by_h[h]["model"]
        scaler = models_by_h[h].get("scaler")
        yhat = mdl.predict(scaler.transform(X) if scaler is not None else X)[0]
        rows.append({"OperatingDTM": delivery_date.date(), "Interval": h, "hb_houston_pred": float(yhat)})

    return pd.DataFrame(rows)


<class 'pandas.core.frame.DataFrame'>


In [20]:
import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

df = con.execute("""
WITH last AS (
  SELECT DATE(MAX(ts)) AS d
  FROM vw_master_spine_ts
  WHERE hb_houston IS NOT NULL
)
SELECT
  (SELECT COUNT(*) FROM vw_master_spine_ts)                                                     AS total_rows,
  (SELECT COUNT(*) FROM vw_master_spine_ts WHERE Interval = 24)                                 AS eod_rows,
  (SELECT COUNT(*) FROM vw_master_spine_ts WHERE hb_houston IS NOT NULL)                        AS priced_rows,
  (SELECT COUNT(*) FROM vw_master_spine_ts WHERE hb_houston IS NOT NULL AND Interval = 24)      AS priced_eod_rows,
  (SELECT COUNT(*) FROM vw_master_spine_ts f, last l
     WHERE f.OperatingDTM = l.d + INTERVAL 1 DAY)                                               AS tomorrow_spine_rows;
""").fetchdf()

con.close()

df


,total_rows,eod_rows,priced_rows,priced_eod_rows,tomorrow_spine_rows
0,23183,966,23063,961,24


In [21]:
import duckdb
import pandas as pd

DB = "../ProjectMain/db/data.duckdb"
con = duckdb.connect(DB)

# Last priced timestamp and date
last_priced = con.execute("""
SELECT
  MAX(ts) AS last_priced_ts,
  DATE(MAX(ts)) AS last_priced_date
FROM vw_master_spine_ts
WHERE hb_houston IS NOT NULL
""").fetchdf()

# How many tomorrow spine rows exist after that date?
tomorrow_cnt = con.execute("""
WITH last AS (
  SELECT DATE(MAX(ts)) AS d
  FROM vw_master_spine_ts
  WHERE hb_houston IS NOT NULL
)
SELECT COUNT(*) AS tomorrow_spine_rows
FROM vw_master_spine_ts f, last l
WHERE f.OperatingDTM = l.d + INTERVAL 1 DAY
""").fetchdf()

# Build a list of base days that have Interval=24 price AND the next day has 24 actual prices
day_pairs = con.execute("""
WITH base AS (
  SELECT DATE(ts) AS base_date, ts AS base_ts
  FROM vw_master_spine_ts
  WHERE hb_houston IS NOT NULL AND Interval = 24
),
nextday_prices AS (
  SELECT OperatingDTM AS delivery_date, COUNT(*) AS n_price_hours
  FROM vw_master_spine_ts
  WHERE hb_houston IS NOT NULL
  GROUP BY OperatingDTM
)
SELECT
  b.base_date,
  b.base_ts,
  (b.base_date + INTERVAL 1 DAY) AS delivery_date,
  COALESCE(n.n_price_hours, 0)   AS nextday_price_hours
FROM base b
LEFT JOIN nextday_prices n
  ON n.delivery_date = b.base_date + INTERVAL 1 DAY
ORDER BY b.base_date DESC
""").fetchdf()

con.close()

print("Last priced:", last_priced.iloc[0].to_dict())
print("Tomorrow spine rows after last priced day:", int(tomorrow_cnt.iloc[0,0]))
print("Top 5 day-ahead pairs (for evaluation):")
display(day_pairs.head())

valid_eval_days = day_pairs[day_pairs["nextday_price_hours"] == 24]
print("Valid day-ahead evaluation days (full next-day actuals):", len(valid_eval_days))


Last priced: {'last_priced_ts': Timestamp('2025-08-18 23:00:00'), 'last_priced_date': Timestamp('2025-08-18 00:00:00')}
Tomorrow spine rows after last priced day: 24
Top 5 day-ahead pairs (for evaluation):


,base_date,base_ts,delivery_date,nextday_price_hours
0,2025-08-18,2025-08-18 23:00:00,2025-08-19,0
1,2025-08-17,2025-08-17 23:00:00,2025-08-18,24
2,2025-08-16,2025-08-16 23:00:00,2025-08-17,24
3,2025-08-15,2025-08-15 23:00:00,2025-08-16,24
4,2025-08-14,2025-08-14 23:00:00,2025-08-15,24


Valid day-ahead evaluation days (full next-day actuals): 955


In [23]:
# Quick refit per horizon on all available rows (dropna) — Ridge baseline
import duckdb, pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

DB = "../ProjectMain/db/data.duckdb"

def refit_ridge_models():
    models_by_h = {}
    con = duckdb.connect(DB)
    for h in range(1, 25):
        # Build horizon dataset: recency at t + future-known covariates at t+h + label y_h=h
        df = con.execute(f"""
            WITH b AS (
              SELECT *,
                     LEAD(hb_houston, {h}) OVER (ORDER BY ts) AS target
              FROM vw_master_spine_lags
            ),
            f AS (
              SELECT *
              FROM vw_master_spine_ts
            )
            SELECT
              b.*,  -- base recency & current price (hb_houston)
              -- future-known covariates at t+h (join by ts + h)
              f.wz_southcentral AS wz_southcentral_f, f.wz_east AS wz_east_f, f.wz_west AS wz_west_f,
              f.wz_northcentral AS wz_northcentral_f, f.wz_farwest AS wz_farwest_f, f.wz_north AS wz_north_f,
              f.wz_southern AS wz_southern_f, f.wz_coast AS wz_coast_f,
              f.lz_north AS lz_north_f, f.lz_west AS lz_west_f, f.lz_south AS lz_south_f, f.lz_houston AS lz_houston_f,
              f.temp_the_woodlands_tx AS temp_the_woodlands_tx_f, f.hum_the_woodlands_tx AS hum_the_woodlands_tx_f,
              f.wind_the_woodlands_tx AS wind_the_woodlands_tx_f, f.precip_the_woodlands_tx AS precip_the_woodlands_tx_f,
              f.temp_katy_tx AS temp_katy_tx_f, f.hum_katy_tx AS hum_katy_tx_f, f.wind_katy_tx AS wind_katy_tx_f, f.precip_katy_tx AS precip_katy_tx_f,
              f.temp_friendswood_tx AS temp_friendswood_tx_f, f.hum_friendswood_tx AS hum_friendswood_tx_f,
              f.wind_friendswood_tx AS wind_friendswood_tx_f, f.precip_friendswood_tx AS precip_friendswood_tx_f,
              f.temp_baytown_tx AS temp_baytown_tx_f, f.hum_baytown_tx AS hum_baytown_tx_f, f.wind_baytown_tx AS wind_baytown_tx_f, f.precip_baytown_tx AS precip_baytown_tx_f,
              f.temp_houston_tx AS temp_houston_tx_f, f.hum_houston_tx AS hum_houston_tx_f, f.wind_houston_tx AS wind_houston_tx_f, f.precip_houston_tx AS precip_houston_tx_f,
              f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
              f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f, f.cal_sin_dow AS cal_sin_dow_f, f.cal_cos_dow AS cal_cos_dow_f
            FROM b
            LEFT JOIN f ON f.ts = b.ts + INTERVAL {h} HOUR
            ORDER BY b.ts
        """).fetchdf()
        con.close() if h == 24 else None

        df = df.dropna().reset_index(drop=True)
        y = df["target"].values
        X = df.drop(columns=[c for c in ["ts","OperatingDTM","Interval","hb_houston","target"] if c in df.columns], errors="ignore")

        scaler = StandardScaler()
        Xs = scaler.fit_transform(X)
        model = Ridge(alpha=5.0)
        model.fit(Xs, y)
        models_by_h[h] = {"model": model, "scaler": scaler}
        con = duckdb.connect(DB)  # reopen for next loop

    con.close()
    return models_by_h

# models_by_h = refit_ridge_models()
# forecast_df = predict_tomorrow(models_by_h)
# display(forecast_df)


In [ ]:
import duckdb, pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

DB = "../ProjectMain/db/data.duckdb"

def refit_ridge_models():
    models_by_h = {}
    con = duckdb.connect(DB)
    for h in range(1, 25):
        df = con.execute(f"""
            WITH b AS (
              SELECT *,
                     LEAD(hb_houston, {h}) OVER (ORDER BY ts) AS target
              FROM vw_master_spine_lags
            ),
            f AS (
              SELECT ts, OperatingDTM, Interval,
                     wz_southcentral, wz_east, wz_west, wz_northcentral, wz_farwest, wz_north, wz_southern, wz_coast,
                     lz_north, lz_west, lz_south, lz_houston,
                     temp_the_woodlands_tx, hum_the_woodlands_tx, wind_the_woodlands_tx, precip_the_woodlands_tx,
                     temp_katy_tx,          hum_katy_tx,          wind_katy_tx,          precip_katy_tx,
                     temp_friendswood_tx,   hum_friendswood_tx,   wind_friendswood_tx,   precip_friendswood_tx,
                     temp_baytown_tx,       hum_baytown_tx,       wind_baytown_tx,       precip_baytown_tx,
                     temp_houston_tx,       hum_houston_tx,       wind_houston_tx,       precip_houston_tx,
                     cal_hour, cal_dow, cal_is_weekend, cal_sin_hour, cal_cos_hour, cal_sin_dow, cal_cos_dow
              FROM vw_master_spine_ts
            )
            SELECT
              b.*,  -- recency + target
              f.wz_southcentral AS wz_southcentral_f, f.wz_east AS wz_east_f, f.wz_west AS wz_west_f,
              f.wz_northcentral AS wz_northcentral_f, f.wz_farwest AS wz_farwest_f, f.wz_north AS wz_north_f,
              f.wz_southern AS wz_southern_f, f.wz_coast AS wz_coast_f,
              f.lz_north AS lz_north_f, f.lz_west AS lz_west_f, f.lz_south AS lz_south_f, f.lz_houston AS lz_houston_f,
              f.temp_the_woodlands_tx AS temp_the_woodlands_tx_f, f.hum_the_woodlands_tx AS hum_the_woodlands_tx_f,
              f.wind_the_woodlands_tx AS wind_the_woodlands_tx_f, f.precip_the_woodlands_tx AS precip_the_woodlands_tx_f,
              f.temp_katy_tx AS temp_katy_tx_f, f.hum_katy_tx AS hum_katy_tx_f, f.wind_katy_tx AS wind_katy_tx_f, f.precip_katy_tx AS precip_katy_tx_f,
              f.temp_friendswood_tx AS temp_friendswood_tx_f, f.hum_friendswood_tx AS hum_friendswood_tx_f,
              f.wind_friendswood_tx AS wind_friendswood_tx_f, f.precip_friendswood_tx AS precip_friendswood_tx_f,
              f.temp_baytown_tx AS temp_baytown_tx_f, f.hum_baytown_tx AS hum_baytown_tx_f, f.wind_baytown_tx AS wind_baytown_tx_f, f.precip_baytown_tx AS precip_baytown_tx_f,
              f.temp_houston_tx AS temp_houston_tx_f, f.hum_houston_tx AS hum_houston_tx_f, f.wind_houston_tx AS wind_houston_tx_f, f.precip_houston_tx AS precip_houston_tx_f,
              f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
              f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f, f.cal_sin_dow AS cal_sin_dow_f, f.cal_cos_dow AS cal_cos_dow_f
            FROM b
            LEFT JOIN f ON f.ts = b.ts + INTERVAL {h} HOUR
            ORDER BY b.ts
        """).fetchdf()
        df = df.dropna().reset_index(drop=True)
        y = df["target"].values
        X = df.drop(columns=[c for c in ["ts","OperatingDTM","Interval","hb_houston","target"] if c in df.columns], errors="ignore")
        scaler, model = StandardScaler(), Ridge(alpha=5.0)
        Xs = scaler.fit_transform(X); model.fit(Xs, y)
        models_by_h[h] = {"model": model, "scaler": scaler}
    con.close()
    return models_by_h

def predict_tomorrow(models_by_h):
    con = duckdb.connect(DB)
    t0 = con.execute("""
        SELECT MAX(ts) FROM vw_master_spine_ts
        WHERE hb_houston IS NOT NULL
    """).fetchone()[0]
    b = con.execute("SELECT * FROM vw_master_spine_lags WHERE ts = ?", [t0]).fetchdf()
    f = con.execute("""
        SELECT * FROM vw_master_spine_ts
        WHERE OperatingDTM = DATE(?::TIMESTAMP) + INTERVAL 1 DAY
        ORDER BY Interval
    """, [t0]).fetchdf()
    con.close()
    if f.empty or len(f) < 24:
        raise RuntimeError("Tomorrow is not fully present in the spine (need 24 rows).")
    rows = []
    for h in range(1, 25):
        feat = pd.concat([b.reset_index(drop=True), f.iloc[[h-1]].reset_index(drop=True)], axis=1)
        X = feat.drop(columns=[c for c in ["ts","OperatingDTM","Interval","hb_houston"] if c in feat.columns], errors="ignore")
        mdl = models_by_h[h]["model"]; scaler = models_by_h[h].get("scaler")
        yhat = mdl.predict(scaler.transform(X) if scaler is not None else X)[0]
        rows.append({"OperatingDTM": f.loc[h-1, "OperatingDTM"], "Interval": int(f.loc[h-1, "Interval"]), "hb_houston_pred": float(yhat)})
    return pd.DataFrame(rows).sort_values(["OperatingDTM","Interval"]).reset_index(drop=True)

# Use existing models_by_h if you already trained; else auto-fit Ridge baselines:
if 'models_by_h' not in globals():
    models_by_h = refit_ridge_models()

forecast_df = predict_tomorrow(models_by_h)
forecast_df


In [3]:
import duckdb, pandas as pd, numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, r2_score

DB = "../ProjectMain/db/data.duckdb"
DELIVERY_DATE = pd.Timestamp("2025-08-18").date()  # <- change if needed

con = duckdb.connect(DB)

# Day-ahead dataset: base=end-of-day with price/recency, future=next day 24h exogenous + label (if known)
day_ahead_sql = """
WITH base_days AS (
  SELECT DATE(ts) AS base_date, ts AS base_ts
  FROM vw_master_spine_lags
  WHERE hb_houston IS NOT NULL AND Interval = 24
),
base_feats AS (
  SELECT
    ts AS base_ts,
    hb_houston                                  AS price_t,
    p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
    dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean
  FROM vw_master_spine_lags
),
future_24 AS (
  SELECT
    OperatingDTM    AS delivery_date,
    Interval        AS delivery_interval,
    hb_houston      AS target,                           -- may be NULL for the newest day
    wz_southcentral, wz_east, wz_west, wz_northcentral, wz_farwest, wz_north, wz_southern, wz_coast,
    lz_north, lz_west, lz_south, lz_houston,
    temp_the_woodlands_tx, hum_the_woodlands_tx, wind_the_woodlands_tx, precip_the_woodlands_tx,
    temp_katy_tx,          hum_katy_tx,          wind_katy_tx,          precip_katy_tx,
    temp_friendswood_tx,   hum_friendswood_tx,   wind_friendswood_tx,   precip_friendswood_tx,
    temp_baytown_tx,       hum_baytown_tx,       wind_baytown_tx,       precip_baytown_tx,
    temp_houston_tx,       hum_houston_tx,       wind_houston_tx,       precip_houston_tx,
    cal_hour AS cal_hour_f, cal_dow AS cal_dow_f, cal_is_weekend AS cal_is_weekend_f,
    cal_sin_hour AS cal_sin_hour_f, cal_cos_hour AS cal_cos_hour_f,
    cal_sin_dow  AS cal_sin_dow_f,  cal_cos_dow  AS cal_cos_dow_f
  FROM vw_master_spine_ts
)
SELECT
  b.base_date,
  b.base_date + INTERVAL 1 DAY        AS delivery_date,
  f.delivery_interval,
  bf.price_t,
  bf.p_lag1,bf.p_lag2,bf.p_lag3,bf.p_lag6,bf.p_lag12,bf.p_lag24,bf.p_lag48,bf.p_lag72,bf.p_lag168,
  bf.dp1,bf.dp24,bf.p_roll24_mean,bf.p_roll24_std,bf.p_roll72_mean,bf.p_roll168_mean,
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.temp_the_woodlands_tx, f.hum_the_woodlands_tx, f.wind_the_woodlands_tx, f.precip_the_woodlands_tx,
  f.temp_katy_tx,          f.hum_katy_tx,          f.wind_katy_tx,          f.precip_katy_tx,
  f.temp_friendswood_tx,   f.hum_friendswood_tx,   f.wind_friendswood_tx,   f.precip_friendswood_tx,
  f.temp_baytown_tx,       f.hum_baytown_tx,       f.wind_baytown_tx,       f.precip_baytown_tx,
  f.temp_houston_tx,       f.hum_houston_tx,       f.wind_houston_tx,       f.precip_houston_tx,
  f.cal_hour_f, f.cal_dow_f, f.cal_is_weekend_f, f.cal_sin_hour_f, f.cal_cos_hour_f, f.cal_sin_dow_f, f.cal_cos_dow_f,
  f.target
FROM base_days b
JOIN base_feats bf
  ON bf.base_ts = b.base_ts
JOIN future_24 f
  ON f.delivery_date = b.base_date + INTERVAL 1 DAY
ORDER BY b.base_date, f.delivery_interval;
"""

data = con.execute(day_ahead_sql).fetchdf()
con.close()

# Ensure types
data["delivery_date"] = pd.to_datetime(data["delivery_date"]).dt.date
data["delivery_interval"] = data["delivery_interval"].astype(int)

# --- Warm-up filter: keep rows with sufficient history to avoid NaNs in recency features
required_recency = ["p_lag24","p_lag72","p_lag168","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean"]
mask_warmup = data[required_recency].notna().all(axis=1)
data = data[mask_warmup].reset_index(drop=True)

# Train rows = where target is known
train_data = data[~data["target"].isna()].copy()

# Hold out last 60 delivery days (if available) for evaluation
if not train_data.empty:
    last_day = max(train_data["delivery_date"])
    recent_cut = pd.Timestamp(last_day) - pd.Timedelta(days=60)
    train_mask = pd.to_datetime(train_data["delivery_date"]) <= recent_cut
    has_eval = (~train_mask).any()
else:
    raise RuntimeError("No rows with known targets found after warm-up. Expand history or relax filters.")

# Features/target
drop_non_features = {"base_date","delivery_date","target"}
X_cols = [c for c in train_data.columns if c not in drop_non_features]

pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",  StandardScaler()),
    ("model",   Ridge(alpha=5.0))
])

# Fit
X_train = train_data.loc[train_mask, X_cols]
y_train = train_data.loc[train_mask, "target"].values
if X_train.shape[0] == 0:
    # fallback: if the 60-day holdout leaves no training, train on all
    X_train = train_data[X_cols]
    y_train = train_data["target"].values
    has_eval = False

pipe.fit(X_train, y_train)

# Evaluate (optional)
if has_eval:
    X_eval = train_data.loc[~train_mask, X_cols]
    y_eval = train_data.loc[~train_mask, "target"].values
    y_pred = pipe.predict(X_eval)
    mae = float(mean_absolute_error(y_eval, y_pred))
    rmse = float(np.sqrt(np.mean((y_eval - y_pred)**2)))
    r2   = float(r2_score(y_eval, y_pred))
    print({"eval_days": int(pd.Series(train_data.loc[~train_mask, "delivery_date"]).nunique()),
           "rows": int(len(y_eval)), "MAE": mae, "RMSE": rmse, "R2": r2})
else:
    print("No eval holdout (trained on all rows).")

# Predict the 24 prices for the requested delivery date
pred_rows = data[(data["delivery_date"] == DELIVERY_DATE)].copy()
if pred_rows.empty or pred_rows["delivery_interval"].nunique() < 24:
    raise RuntimeError(f"Spine missing for {DELIVERY_DATE} (need 24 intervals).")

X_pred = pred_rows[X_cols]
yhat = pipe.predict(X_pred)

forecast = pd.DataFrame({
    "OperatingDTM": [DELIVERY_DATE]*24,
    "Interval": pred_rows["delivery_interval"].tolist(),
    "hb_houston_pred": yhat.tolist()
}).sort_values("Interval").reset_index(drop=True)

forecast


{'eval_days': 60, 'rows': 1440, 'MAE': 32.765872300955444, 'RMSE': 44.48798502742817, 'R2': -1.525670042704765}


c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\sklearn\impute\_base.py:637: UserWarning: Skipping features without any observed values: ['temp_the_woodlands_tx' 'hum_the_woodlands_tx' 'wind_the_woodlands_tx'
 'precip_the_woodlands_tx' 'temp_katy_tx' 'hum_katy_tx' 'wind_katy_tx'
 'precip_katy_tx' 'temp_friendswood_tx' 'hum_friendswood_tx'
 'wind_friendswood_tx' 'precip_friendswood_tx' 'temp_baytown_tx'
 'hum_baytown_tx' 'wind_baytown_tx' 'precip_baytown_tx' 'temp_houston_tx'
 'hum_houston_tx' 'wind_houston_tx' 'precip_houston_tx']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\sklearn\impute\_base.py:637: UserWarning: Skipping features without any observed values: ['temp_the_woodlands_tx' 'hum_the_woodlands_tx' 'wind_the_woodlands_tx'
 'precip_the_woodlands_tx' 'temp_katy_tx' 'hum_katy_tx' 'wind_katy_tx'
 'precip_katy_tx' 'temp_friendswood_tx' 'hum_friendswood_tx'
 'wind_friendswood_t

,OperatingDTM,Interval,hb_houston_pred
0,2025-08-18,1,70.918419
1,2025-08-18,2,56.588500
2,2025-08-18,3,42.603458
3,2025-08-18,4,34.488922
4,2025-08-18,5,30.233206
5,2025-08-18,6,31.682249
6,2025-08-18,7,34.279342
7,2025-08-18,8,35.460637
8,2025-08-18,9,27.451566
9,2025-08-18,10,29.284216


Okay, finally some results after better defining the 24 hour set of expected results and confirming data was properly aligned.

# LETS ENHANCE

In [5]:
# === ONE-CELL DAY-AHEAD GBM WITH RAMP/SPREAD FEATURES ===
import duckdb, pandas as pd, numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Prefer LightGBM; fallback to Ridge if not present
try:
    from lightgbm import LGBMRegressor
    USE_LGBM = True
except Exception:
    from sklearn.linear_model import Ridge
    from sklearn.preprocessing import StandardScaler
    USE_LGBM = False

from sklearn.metrics import mean_absolute_error, r2_score

DB = "../ProjectMain/db/data.duckdb"
DELIVERY_DATE = pd.Timestamp("2025-08-17").date()   # <--- change target date here if needed

con = duckdb.connect(DB)

# --- 1) Enhanced feature views: load ramps/spreads + weather ramps/spreads (future-known) ---
con.execute("""
CREATE OR REPLACE VIEW load_fcst_enh AS
WITH z AS (
  SELECT
    OperatingDTM, Interval,
    -- system-level aggregates
    (wz_southcentral + wz_east + wz_west + wz_northcentral + wz_farwest + wz_north + wz_southern + wz_coast) AS net_wz,
    (lz_north + lz_west + lz_south + lz_houston) AS net_lz,
    -- spatial spreads
    GREATEST(wz_southcentral, wz_east, wz_west, wz_northcentral, wz_farwest, wz_north, wz_southern, wz_coast) AS wz_max,
    LEAST(  wz_southcentral, wz_east, wz_west, wz_northcentral, wz_farwest, wz_north, wz_southern, wz_coast) AS wz_min,
    GREATEST(lz_north, lz_west, lz_south, lz_houston) AS lz_max,
    LEAST(  lz_north, lz_west, lz_south, lz_houston) AS lz_min,
    lz_houston
  FROM vw_loadforecast_by_zone
)
SELECT
  *,
  -- 1h ramps
  net_wz - LAG(net_wz) OVER (ORDER BY OperatingDTM, Interval) AS net_wz_ramp1,
  net_lz - LAG(net_lz) OVER (ORDER BY OperatingDTM, Interval) AS net_lz_ramp1,
  lz_houston - LAG(lz_houston) OVER (ORDER BY OperatingDTM, Interval) AS lz_houston_ramp1,
  -- spreads
  (wz_max - wz_min) AS wz_spread,
  (lz_max - lz_min) AS lz_spread
FROM z;
""")

con.execute("""
CREATE OR REPLACE VIEW wx_fcst_enh AS
WITH w AS (
  SELECT
    OperatingDTM, "interval" AS Interval,
    -- temperatures & humidity for your 5 cities
    temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx, temp_baytown_tx, temp_houston_tx,
    hum_the_woodlands_tx,  hum_katy_tx,  hum_friendswood_tx,  hum_baytown_tx,  hum_houston_tx
  FROM vw_forecast_weather_by_city
),
agg AS (
  SELECT
    OperatingDTM, Interval,
    -- averages
    (temp_the_woodlands_tx + temp_katy_tx + temp_friendswood_tx + temp_baytown_tx + temp_houston_tx)/5.0 AS temp_avg,
    (hum_the_woodlands_tx  + hum_katy_tx  + hum_friendswood_tx  + hum_baytown_tx  + hum_houston_tx)/5.0  AS hum_avg,
    -- spreads
    GREATEST(temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx, temp_baytown_tx, temp_houston_tx)
      - LEAST(temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx, temp_baytown_tx, temp_houston_tx) AS temp_spread,
    GREATEST(hum_the_woodlands_tx, hum_katy_tx, hum_friendswood_tx, hum_baytown_tx, hum_houston_tx)
      - LEAST(hum_the_woodlands_tx, hum_katy_tx, hum_friendswood_tx, hum_baytown_tx, hum_houston_tx) AS hum_spread
  FROM w
)
SELECT
  a.*,
  -- 1h ramps on aggregates
  a.temp_avg - LAG(a.temp_avg) OVER (ORDER BY OperatingDTM, Interval) AS temp_avg_ramp1,
  a.hum_avg  - LAG(a.hum_avg)  OVER (ORDER BY OperatingDTM, Interval) AS hum_avg_ramp1
FROM agg a;
""")

# --- 2) Day-ahead dataset (base=end-of-day recency; future=next-day 24h features + enhanced joins) ---
day_ahead_sql = """
WITH base_days AS (
  SELECT DATE(ts) AS base_date, ts AS base_ts
  FROM vw_master_spine_lags
  WHERE hb_houston IS NOT NULL AND Interval = 24
),
base_feats AS (
  SELECT
    ts AS base_ts,
    hb_houston                                  AS price_t,
    p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
    dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean
  FROM vw_master_spine_lags
),
future_24 AS (
  SELECT
    OperatingDTM    AS delivery_date,
    Interval        AS delivery_interval,
    hb_houston      AS target,                           -- may be NULL for the newest day
    -- base future-known features
    wz_southcentral, wz_east, wz_west, wz_northcentral, wz_farwest, wz_north, wz_southern, wz_coast,
    lz_north, lz_west, lz_south, lz_houston,
    temp_the_woodlands_tx, hum_the_woodlands_tx, wind_the_woodlands_tx, precip_the_woodlands_tx,
    temp_katy_tx,          hum_katy_tx,          wind_katy_tx,          precip_katy_tx,
    temp_friendswood_tx,   hum_friendswood_tx,   wind_friendswood_tx,   precip_friendswood_tx,
    temp_baytown_tx,       hum_baytown_tx,       wind_baytown_tx,       precip_baytown_tx,
    temp_houston_tx,       hum_houston_tx,       wind_houston_tx,       precip_houston_tx,
    cal_hour AS cal_hour_f, cal_dow AS cal_dow_f, cal_is_weekend AS cal_is_weekend_f,
    cal_sin_hour AS cal_sin_hour_f, cal_cos_hour AS cal_cos_hour_f,
    cal_sin_dow  AS cal_sin_dow_f,  cal_cos_dow  AS cal_cos_dow_f
  FROM vw_master_spine_ts
)
SELECT
  b.base_date,
  b.base_date + INTERVAL 1 DAY        AS delivery_date,
  f.delivery_interval,

  -- recency at base EOD
  bf.price_t,
  bf.p_lag1,bf.p_lag2,bf.p_lag3,bf.p_lag6,bf.p_lag12,bf.p_lag24,bf.p_lag48,bf.p_lag72,bf.p_lag168,
  bf.dp1,bf.dp24,bf.p_roll24_mean,bf.p_roll24_std,bf.p_roll72_mean,bf.p_roll168_mean,

  -- future-known baseline features at delivery hour
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.temp_the_woodlands_tx, f.hum_the_woodlands_tx, f.wind_the_woodlands_tx, f.precip_the_woodlands_tx,
  f.temp_katy_tx,          f.hum_katy_tx,          f.wind_katy_tx,          f.precip_katy_tx,
  f.temp_friendswood_tx,   f.hum_friendswood_tx,   f.wind_friendswood_tx,   f.precip_friendswood_tx,
  f.temp_baytown_tx,       f.hum_baytown_tx,       f.wind_baytown_tx,       f.precip_baytown_tx,
  f.temp_houston_tx,       f.hum_houston_tx,       f.wind_houston_tx,       f.precip_houston_tx,
  f.cal_hour_f, f.cal_dow_f, f.cal_is_weekend_f, f.cal_sin_hour_f, f.cal_cos_hour_f, f.cal_sin_dow_f, f.cal_cos_dow_f,

  -- enhanced load features at delivery hour
  lfe.net_wz, lfe.net_lz, lfe.net_wz_ramp1, lfe.net_lz_ramp1, lfe.lz_houston_ramp1, lfe.wz_spread, lfe.lz_spread,

  -- enhanced weather features at delivery hour
  wfe.temp_avg, wfe.temp_spread, wfe.temp_avg_ramp1,
  wfe.hum_avg,  wfe.hum_spread,  wfe.hum_avg_ramp1,

  -- target (actual hb_houston at delivery hour; may be NULL for newest day)
  f.target

FROM base_days b
JOIN base_feats bf
  ON bf.base_ts = b.base_ts
JOIN future_24 f
  ON f.delivery_date = b.base_date + INTERVAL 1 DAY
LEFT JOIN load_fcst_enh lfe
  ON lfe.OperatingDTM = f.delivery_date AND lfe.Interval = f.delivery_interval
LEFT JOIN wx_fcst_enh wfe
  ON wfe.OperatingDTM = f.delivery_date AND wfe.Interval = f.delivery_interval
ORDER BY b.base_date, f.delivery_interval;
"""
data = con.execute(day_ahead_sql).fetchdf()
con.close()

# --- 3) Prep data: warm-up filter (avoid lag NaNs), train/eval split ---
data["delivery_date"] = pd.to_datetime(data["delivery_date"]).dt.date
data["delivery_interval"] = data["delivery_interval"].astype(int)

required_recency = ["p_lag24","p_lag72","p_lag168","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean"]
data = data[data[required_recency].notna().all(axis=1)].reset_index(drop=True)

train_data = data[~data["target"].isna()].copy()
if train_data.empty:
    raise RuntimeError("No rows with known targets after warm-up. Expand history or relax filters.")

# Hold out the last 60 delivery days for eval (if available)
last_day = max(train_data["delivery_date"])
recent_cut = pd.Timestamp(last_day) - pd.Timedelta(days=60)
train_mask = pd.to_datetime(train_data["delivery_date"]) <= recent_cut
has_eval = (~train_mask).any()

# Features/target
DROP_NON_FEATURES = {"base_date","delivery_date","target"}
X_cols = [c for c in train_data.columns if c not in DROP_NON_FEATURES]

if USE_LGBM:
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", LGBMRegressor(
            objective="mae",     # robust to spikes
            n_estimators=1200,
            learning_rate=0.03,
            num_leaves=127,
            subsample=0.9,
            colsample_bytree=0.9,
            random_state=42
        ))
    ])
else:
    # Fallback to Ridge if LightGBM isn't available
    from sklearn.linear_model import Ridge
    from sklearn.preprocessing import StandardScaler
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler",  StandardScaler()),
        ("model",   Ridge(alpha=5.0))
    ])

# Train
X_train = train_data.loc[train_mask, X_cols] if has_eval else train_data[X_cols]
y_train = train_data.loc[train_mask, "target"].values if has_eval else train_data["target"].values
pipe.fit(X_train, y_train)

# Evaluate on the holdout (optional)
if has_eval:
    X_eval = train_data.loc[~train_mask, X_cols]
    y_eval = train_data.loc[~train_mask, "target"].values
    y_pred = pipe.predict(X_eval)
    mae = float(mean_absolute_error(y_eval, y_pred))
    rmse = float(np.sqrt(np.mean((y_eval - y_pred)**2)))
    r2   = float(r2_score(y_eval, y_pred))
    print({"model": "LGBM" if USE_LGBM else "Ridge", "eval_days": int(pd.Series(train_data.loc[~train_mask, "delivery_date"]).nunique()),
           "rows": int(len(y_eval)), "MAE": mae, "RMSE": rmse, "R2": r2})

# --- 4) Predict the 24 prices for DELIVERY_DATE ---
pred_rows = data[(data["delivery_date"] == DELIVERY_DATE)].copy()
if pred_rows.empty or pred_rows["delivery_interval"].nunique() < 24:
    raise RuntimeError(f"Spine missing for {DELIVERY_DATE} (need 24 intervals).")

X_pred = pred_rows[X_cols]
yhat = pipe.predict(X_pred)

forecast = pd.DataFrame({
    "OperatingDTM": [DELIVERY_DATE]*24,
    "Interval": pred_rows["delivery_interval"].tolist(),
    "hb_houston_pred": yhat.tolist()
}).sort_values("Interval").reset_index(drop=True)

forecast


c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\sklearn\impute\_base.py:637: UserWarning: Skipping features without any observed values: ['temp_the_woodlands_tx' 'hum_the_woodlands_tx' 'wind_the_woodlands_tx'
 'precip_the_woodlands_tx' 'temp_katy_tx' 'hum_katy_tx' 'wind_katy_tx'
 'precip_katy_tx' 'temp_friendswood_tx' 'hum_friendswood_tx'
 'wind_friendswood_tx' 'precip_friendswood_tx' 'temp_baytown_tx'
 'hum_baytown_tx' 'wind_baytown_tx' 'precip_baytown_tx' 'temp_houston_tx'
 'hum_houston_tx' 'wind_houston_tx' 'precip_houston_tx' 'temp_avg'
 'temp_spread' 'temp_avg_ramp1' 'hum_avg' 'hum_spread' 'hum_avg_ramp1']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004345 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9038
[LightGBM] [Info] Number of data points in the train set: 21435, number of used features: 43
[LightGBM] [Info] Start training from score 22.680000
{'model': 'LGBM', 'eval_days': 60, 'rows': 1440, 'MAE': 15.082037261885308, 'RMSE': 37.02211994018373, 'R2': -0.7490962479080749}


c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\sklearn\impute\_base.py:637: UserWarning: Skipping features without any observed values: ['temp_the_woodlands_tx' 'hum_the_woodlands_tx' 'wind_the_woodlands_tx'
 'precip_the_woodlands_tx' 'temp_katy_tx' 'hum_katy_tx' 'wind_katy_tx'
 'precip_katy_tx' 'temp_friendswood_tx' 'hum_friendswood_tx'
 'wind_friendswood_tx' 'precip_friendswood_tx' 'temp_baytown_tx'
 'hum_baytown_tx' 'wind_baytown_tx' 'precip_baytown_tx' 'temp_houston_tx'
 'hum_houston_tx' 'wind_houston_tx' 'precip_houston_tx' 'temp_avg'
 'temp_spread' 'temp_avg_ramp1' 'hum_avg' 'hum_spread' 'hum_avg_ramp1']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\sklearn\impute\_base.py:637: Use

,OperatingDTM,Interval,hb_houston_pred
0,2025-08-17,1,28.807174
1,2025-08-17,2,28.189162
2,2025-08-17,3,26.260620
3,2025-08-17,4,26.062219
4,2025-08-17,5,25.447546
5,2025-08-17,6,26.150197
6,2025-08-17,7,26.055271
7,2025-08-17,8,25.386501
8,2025-08-17,9,20.661650
9,2025-08-17,10,22.233936


In [9]:
# ==== ONE-CELL PYTORCH SEQ2SEQ (ENCODER: past W hours, DECODER: next 24 hours) ====
import duckdb, pandas as pd, numpy as np, math, datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from typing import List, Tuple

# ---- Config ----
DB = "../ProjectMain/db/data.duckdb"
DELIVERY_DATE = pd.Timestamp("2025-08-18").date()  # change target date if you want
W_PAST = 168          # encoder window (hours)
H_FUT = 24            # decoder steps (hours)
HOLDOUT_DAYS = 60     # last N delivery days for evaluation
HIDDEN = 128
EPOCHS = 20
LR = 1e-3
DEVICE = "cuda" if __import__("torch").cuda.is_available() else "cpu"

import torch
import torch.nn as nn
torch.manual_seed(42)

# ---- Ensure enhanced views exist (idempotent) ----
con = duckdb.connect(DB)
con.execute("""
CREATE OR REPLACE VIEW load_fcst_enh AS
WITH z AS (
  SELECT
    OperatingDTM, Interval,
    -- system aggregates
    (wz_southcentral + wz_east + wz_west + wz_northcentral + wz_farwest + wz_north + wz_southern + wz_coast) AS net_wz,
    (lz_north + lz_west + lz_south + lz_houston) AS net_lz,
    -- spreads (wz + lz)
    GREATEST(wz_southcentral, wz_east, wz_west, wz_northcentral, wz_farwest, wz_north, wz_southern, wz_coast) AS wz_max,
    LEAST(  wz_southcentral, wz_east, wz_west, wz_northcentral, wz_farwest, wz_north, wz_southern, wz_coast) AS wz_min,
    GREATEST(lz_north, lz_west, lz_south, lz_houston) AS lz_max,
    LEAST(  lz_north, lz_west, lz_south, lz_houston) AS lz_min,
    -- keep lz_houston for its own ramp
    lz_houston
  FROM vw_loadforecast_by_zone
)
SELECT
  z.*,
  -- 1h ramps
  net_wz      - LAG(net_wz)      OVER (ORDER BY OperatingDTM, Interval) AS net_wz_ramp1,
  net_lz      - LAG(net_lz)      OVER (ORDER BY OperatingDTM, Interval) AS net_lz_ramp1,
  lz_houston  - LAG(lz_houston)  OVER (ORDER BY OperatingDTM, Interval) AS lz_houston_ramp1,
  -- spreads
  (wz_max - wz_min) AS wz_spread,
  (lz_max - lz_min) AS lz_spread
FROM z;
""")

# Weather enhanced view (unchanged)
con.execute("""
CREATE OR REPLACE VIEW wx_fcst_enh AS
WITH w AS (
  SELECT
    OperatingDTM, "interval" AS Interval,
    temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx, temp_baytown_tx, temp_houston_tx,
    hum_the_woodlands_tx,  hum_katy_tx,  hum_friendswood_tx,  hum_baytown_tx,  hum_houston_tx
  FROM vw_forecast_weather_by_city
),
agg AS (
  SELECT
    OperatingDTM, Interval,
    (temp_the_woodlands_tx + temp_katy_tx + temp_friendswood_tx + temp_baytown_tx + temp_houston_tx)/5.0 AS temp_avg,
    (hum_the_woodlands_tx  + hum_katy_tx  + hum_friendswood_tx  + hum_baytown_tx  + hum_houston_tx)/5.0  AS hum_avg,
    GREATEST(temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx, temp_baytown_tx, temp_houston_tx)
      - LEAST(temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx, temp_baytown_tx, temp_houston_tx) AS temp_spread,
    GREATEST(hum_the_woodlands_tx, hum_katy_tx, hum_friendswood_tx, hum_baytown_tx, hum_houston_tx)
      - LEAST(hum_the_woodlands_tx, hum_katy_tx, hum_friendswood_tx, hum_baytown_tx, hum_houston_tx) AS hum_spread
  FROM w
)
SELECT
  a.*,
  a.temp_avg - LAG(a.temp_avg) OVER (ORDER BY OperatingDTM, Interval) AS temp_avg_ramp1,
  a.hum_avg  - LAG(a.hum_avg)  OVER (ORDER BY OperatingDTM, Interval) AS hum_avg_ramp1
FROM agg a;
""")

# quick smoke test (optional)
print(con.execute("SELECT COUNT(*) FROM load_fcst_enh").fetchone())
print(con.execute("SELECT COUNT(*) FROM wx_fcst_enh").fetchone())


# ---- Pull core tables to pandas (small enough to fit in memory) ----
df_lags = con.execute("""
SELECT
  ts, OperatingDTM, Interval, hb_houston,
  p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
  dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean
FROM vw_master_spine_lags
ORDER BY ts
""").df()
df_lags["ts"] = pd.to_datetime(df_lags["ts"])

df_future = con.execute("""
SELECT
  f.OperatingDTM AS delivery_date,
  f.Interval     AS delivery_interval,
  f.hb_houston   AS target,
  -- base future features
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.temp_the_woodlands_tx, f.hum_the_woodlands_tx, f.wind_the_woodlands_tx, f.precip_the_woodlands_tx,
  f.temp_katy_tx, f.hum_katy_tx, f.wind_katy_tx, f.precip_katy_tx,
  f.temp_friendswood_tx, f.hum_friendswood_tx, f.wind_friendswood_tx, f.precip_friendswood_tx,
  f.temp_baytown_tx, f.hum_baytown_tx, f.wind_baytown_tx, f.precip_baytown_tx,
  f.temp_houston_tx, f.hum_houston_tx, f.wind_houston_tx, f.precip_houston_tx,
  f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
  f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f,
  f.cal_sin_dow  AS cal_sin_dow_f,  f.cal_cos_dow  AS cal_cos_dow_f,
  -- enhanced joins
  l.net_wz, l.net_lz, l.net_wz_ramp1, l.net_lz_ramp1, l.lz_houston_ramp1, l.wz_spread, l.lz_spread,
  w.temp_avg, w.temp_spread, w.temp_avg_ramp1, w.hum_avg, w.hum_spread, w.hum_avg_ramp1
FROM vw_master_spine_ts f
LEFT JOIN load_fcst_enh l
  ON l.OperatingDTM = f.OperatingDTM AND l.Interval = f.Interval
LEFT JOIN wx_fcst_enh w
  ON w.OperatingDTM = f.OperatingDTM AND w.Interval = f.Interval
ORDER BY f.OperatingDTM, f.Interval
""").df()
con.close()

df_future["delivery_date"] = pd.to_datetime(df_future["delivery_date"]).dt.date
df_future["delivery_interval"] = df_future["delivery_interval"].astype(int)

# ---- Build list of training samples: base = EOD with price AND next-day has 24 actuals ----
# base_ts are end-of-day rows (Interval=24) with price
base_rows = df_lags[(df_lags["hb_houston"].notna()) & (df_lags["Interval"]==24)][["ts","OperatingDTM","hb_houston"]].copy()
base_rows["base_date"] = pd.to_datetime(base_rows["OperatingDTM"]).dt.date

# find delivery days with 24 actuals
has24 = df_future.groupby("delivery_date")["target"].apply(lambda s: s.notna().sum()==24)
valid_delivery_dates = set(has24[has24].index)

# keep those base rows whose next day has 24 targets
base_rows["delivery_date"] = base_rows["base_date"] + pd.Timedelta(days=1)
base_rows = base_rows[base_rows["delivery_date"].isin(valid_delivery_dates)].reset_index(drop=True)

# ---- Assemble sequences ----
enc_cols = [
    "hb_houston","p_lag1","p_lag2","p_lag3","p_lag6","p_lag12","p_lag24","p_lag48","p_lag72","p_lag168",
    "dp1","dp24","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean"
]
dec_future_cols = [
    # calendar & exogenous for each delivery hour (future-known)
    "cal_hour_f","cal_dow_f","cal_is_weekend_f","cal_sin_hour_f","cal_cos_hour_f","cal_sin_dow_f","cal_cos_dow_f",
    "net_wz","net_lz","net_wz_ramp1","net_lz_ramp1","lz_houston_ramp1","wz_spread","lz_spread",
    "temp_avg","temp_spread","temp_avg_ramp1","hum_avg","hum_spread","hum_avg_ramp1",
    # (you can add specific city-level features too)
]

# index df_lags by ts for fast slicing
df_lags_idx = df_lags.set_index("ts").sort_index()

samples = []
for _, r in base_rows.iterrows():
    ts0 = r["ts"]  # base end-of-day timestamp
    # Encoder window
    start = ts0 - pd.Timedelta(hours=W_PAST-1)
    enc_df = df_lags_idx.loc[start:ts0][enc_cols]
    if enc_df.shape[0] != W_PAST or enc_df.isna().any().any():
        continue  # skip incomplete windows
    # Decoder future 24 rows
    fdf = df_future[(df_future["delivery_date"]==r["delivery_date"])].sort_values("delivery_interval")
    if fdf.shape[0] != H_FUT:
        continue
    X_enc = enc_df.values.astype(np.float32)                               # (W, E)
    X_dec = fdf[dec_future_cols].values.astype(np.float32)                 # (24, D)
    y     = fdf["target"].values.astype(np.float32)                        # (24,)
    # previous observed price at base (to start decoding)
    y0    = np.float32(r["hb_houston"])
    samples.append((X_enc, X_dec, y, y0, r["delivery_date"]))

if len(samples) == 0:
    raise RuntimeError("No training samples assembled. Check warm-up length W_PAST and data availability.")

# ---- Train/val split by delivery_date (hold out last HOLDOUT_DAYS) ----
dates = sorted({s[4] for s in samples})
cut_date = dates[-1] - dt.timedelta(days=HOLDOUT_DAYS) if len(dates)>HOLDOUT_DAYS else dates[0]
train_idx = [i for i,s in enumerate(samples) if s[4] <= cut_date]
val_idx   = [i for i,s in enumerate(samples) if s[4] >  cut_date]

# stack into arrays
def stack(idxs):
    Xe = np.stack([samples[i][0] for i in idxs], axis=0)   # (N, W, E)
    Xd = np.stack([samples[i][1] for i in idxs], axis=0)   # (N, 24, D)
    Y  = np.stack([samples[i][2] for i in idxs], axis=0)   # (N, 24)
    Y0 = np.stack([samples[i][3] for i in idxs], axis=0)   # (N,)
    return Xe, Xd, Y, Y0

Xe_tr, Xd_tr, Y_tr, Y0_tr = stack(train_idx)
Xe_va, Xd_va, Y_va, Y0_va = stack(val_idx) if len(val_idx)>0 else (None, None, None, None)

# ---- Normalize features (standardize enc & dec features; standardize target for stability) ----
enc_scaler = StandardScaler().fit(Xe_tr.reshape(len(train_idx)*W_PAST, -1))
dec_scaler = StandardScaler().fit(Xd_tr.reshape(len(train_idx)*H_FUT, -1))
y_scaler   = StandardScaler().fit(Y_tr.reshape(-1,1))

Xe_tr_n = enc_scaler.transform(Xe_tr.reshape(len(train_idx)*W_PAST, -1)).reshape(Xe_tr.shape)
Xd_tr_n = dec_scaler.transform(Xd_tr.reshape(len(train_idx)*H_FUT, -1)).reshape(Xd_tr.shape)
Y_tr_n  = y_scaler.transform(Y_tr.reshape(-1,1)).reshape(Y_tr.shape)
Y0_tr_n = y_scaler.transform(Y0_tr.reshape(-1,1)).reshape(-1)

if Xe_va is not None:
    Xe_va_n = enc_scaler.transform(Xe_va.reshape(len(val_idx)*W_PAST, -1)).reshape(Xe_va.shape)
    Xd_va_n = dec_scaler.transform(Xd_va.reshape(len(val_idx)*H_FUT, -1)).reshape(Xd_va.shape)
    Y_va_n  = y_scaler.transform(Y_va.reshape(-1,1)).reshape(Y_va.shape)
    Y0_va_n = y_scaler.transform(Y0_va.reshape(-1,1)).reshape(-1)

# ---- PyTorch Dataset/Dataloader ----
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self, Xe, Xd, Y, Y0):
        self.Xe = torch.from_numpy(Xe).to(torch.float32)
        self.Xd = torch.from_numpy(Xd).to(torch.float32)
        self.Y  = torch.from_numpy(Y).to(torch.float32)
        self.Y0 = torch.from_numpy(Y0).to(torch.float32)
    def __len__(self): return self.Xe.shape[0]
    def __getitem__(self, i): return self.Xe[i], self.Xd[i], self.Y[i], self.Y0[i]

bs = 32
train_loader = torch.utils.data.DataLoader(Seq2SeqDataset(Xe_tr_n, Xd_tr_n, Y_tr_n, Y0_tr_n), batch_size=bs, shuffle=True)
val_loader   = None if Xe_va is None else torch.utils.data.DataLoader(Seq2SeqDataset(Xe_va_n, Xd_va_n, Y_va_n, Y0_va_n), batch_size=bs, shuffle=False)

# ---- Model: GRU encoder-decoder with autoregressive decoding (teacher forcing p=0.5) ----
class Encoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(input_size=in_dim, hidden_size=hidden, batch_first=True)
    def forward(self, x):
        _, h = self.rnn(x)   # h: (1, B, H)
        return h

class Decoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(input_size=in_dim+1, hidden_size=hidden, batch_first=True)  # +1 for prev y
        self.out = nn.Linear(hidden, 1)
    def forward(self, future_feats, h0, y0, teacher=None, tf_prob=0.5):
        """
        future_feats: (B, 24, D)
        y0: (B,) normalized last price at base time
        teacher: (B, 24) normalized targets (optional, for teacher forcing)
        """
        B, T, D = future_feats.shape
        y_prev = y0.view(B,1)           # (B,1)
        outputs = []
        h = h0
        for t in range(T):
            x_t = torch.cat([future_feats[:,t,:], y_prev], dim=1).unsqueeze(1)  # (B,1,D+1)
            out, h = self.rnn(x_t, h)
            y_t = self.out(out[:, -1, :]).squeeze(1)   # (B,)
            outputs.append(y_t)
            if (teacher is not None) and (np.random.rand() < tf_prob):
                y_prev = teacher[:,t].view(B,1)
            else:
                y_prev = y_t.view(B,1)
        return torch.stack(outputs, dim=1)   # (B, 24)

class Seq2Seq(nn.Module):
    def __init__(self, enc_in, dec_in, hidden):
        super().__init__()
        self.enc = Encoder(enc_in, hidden)
        self.dec = Decoder(dec_in, hidden)
    def forward(self, x_enc, x_dec, y0, y_teacher=None, tf_prob=0.5):
        h = self.enc(x_enc)
        y_hat = self.dec(x_dec, h, y0, y_teacher, tf_prob)
        return y_hat

enc_in = Xe_tr_n.shape[2]
dec_in = Xd_tr_n.shape[2]
model = Seq2Seq(enc_in, dec_in, HIDDEN).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.HuberLoss()  # robust to spikes

# ---- Train ----
def train_epoch():
    model.train()
    tot = 0.0
    for xenc, xdec, y, y0 in train_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        optimizer.zero_grad()
        yhat = model(xenc, xdec, y0, y_teacher=y, tf_prob=0.5)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        tot += loss.item() * xenc.size(0)
    return tot / len(train_loader.dataset)

@torch.no_grad()
def eval_epoch():
    if val_loader is None: return None
    model.eval()
    tot = 0.0
    for xenc, xdec, y, y0 in val_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        yhat = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0)
        loss = loss_fn(yhat, y)
        tot += loss.item() * xenc.size(0)
    return tot / len(val_loader.dataset)

for ep in range(1, EPOCHS+1):
    tr = train_epoch()
    va = eval_epoch()
    if va is None:
        print(f"Epoch {ep:02d}  train_loss={tr:.4f}")
    else:
        print(f"Epoch {ep:02d}  train_loss={tr:.4f}  val_loss={va:.4f}")

# ---- Evaluate on holdout in real units (MAE) ----
if val_loader is not None:
    model.eval()
    preds, trues = [], []
    for xenc, xdec, y, y0 in val_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        yhat_n = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0).cpu().numpy()
        y_n = y.cpu().numpy()
        # invert scale
        yhat = y_scaler.inverse_transform(yhat_n.reshape(-1,1)).reshape(yhat_n.shape)
        yt    = y_scaler.inverse_transform(y_n.reshape(-1,1)).reshape(y_n.shape)
        preds.append(yhat); trues.append(yt)
    preds = np.vstack(preds); trues = np.vstack(trues)
    print({"holdout_MAE": float(mean_absolute_error(trues.flatten(), preds.flatten()))})

# ---- Predict the 24 prices for DELIVERY_DATE ----
# Find the base_ts (end of previous day) for this delivery date
base_row = base_rows[base_rows["delivery_date"]==DELIVERY_DATE]
if base_row.empty:
    raise RuntimeError(f"No base EOD found for delivery_date={DELIVERY_DATE} (check that previous day has Interval=24 price).")
ts0 = base_row.iloc[0]["ts"]
y0  = np.float32(base_row.iloc[0]["hb_houston"])

# Encoder window
df_win = df_lags_idx.loc[ts0 - pd.Timedelta(hours=W_PAST-1): ts0][enc_cols]
if df_win.shape[0] != W_PAST:
    raise RuntimeError("Insufficient history for encoder window.")
X_enc_pred = df_win.values.astype(np.float32)[None, ...]                 # (1,W,E)
# Decoder future features (24 rows for delivery date)
fpred = df_future[(df_future["delivery_date"]==DELIVERY_DATE)].sort_values("delivery_interval")
if fpred.shape[0] != H_FUT:
    raise RuntimeError("Spine missing 24 future rows for delivery date.")
X_dec_pred = fpred[dec_future_cols].values.astype(np.float32)[None, ...] # (1,24,D)

# Normalize
X_enc_pred_n = enc_scaler.transform(X_enc_pred.reshape(W_PAST, -1)).reshape(1, W_PAST, -1)
X_dec_pred_n = dec_scaler.transform(X_dec_pred.reshape(H_FUT,  -1)).reshape(1, H_FUT,  -1)
y0_n = y_scaler.transform(np.array([[y0]], dtype=np.float32)).reshape(-1)[0]

# Run model
model.eval()
with torch.no_grad():
    yhat_n = model(torch.from_numpy(X_enc_pred_n).to(DEVICE),
                   torch.from_numpy(X_dec_pred_n).to(DEVICE),
                   torch.tensor([y0_n], dtype=torch.float32, device=DEVICE),
                   y_teacher=None, tf_prob=0.0).cpu().numpy()[0]
yhat = y_scaler.inverse_transform(yhat_n.reshape(-1,1)).reshape(-1)

forecast = pd.DataFrame({
    "OperatingDTM": [DELIVERY_DATE]*H_FUT,
    "Interval": list(range(1, H_FUT+1)),
    "hb_houston_pred": yhat.tolist()
})
forecast


(23183,)
(192,)


c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\sklearn\utils\extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\sklearn\utils\extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\sklearn\utils\extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Epoch 01  train_loss=nan  val_loss=nan
Epoch 02  train_loss=nan  val_loss=nan
Epoch 03  train_loss=nan  val_loss=nan
Epoch 04  train_loss=nan  val_loss=nan
Epoch 05  train_loss=nan  val_loss=nan
Epoch 06  train_loss=nan  val_loss=nan
Epoch 07  train_loss=nan  val_loss=nan
Epoch 08  train_loss=nan  val_loss=nan
Epoch 09  train_loss=nan  val_loss=nan
Epoch 10  train_loss=nan  val_loss=nan
Epoch 11  train_loss=nan  val_loss=nan
Epoch 12  train_loss=nan  val_loss=nan
Epoch 13  train_loss=nan  val_loss=nan
Epoch 14  train_loss=nan  val_loss=nan
Epoch 15  train_loss=nan  val_loss=nan
Epoch 16  train_loss=nan  val_loss=nan
Epoch 17  train_loss=nan  val_loss=nan
Epoch 18  train_loss=nan  val_loss=nan
Epoch 19  train_loss=nan  val_loss=nan
Epoch 20  train_loss=nan  val_loss=nan


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [10]:
# ==== PyTorch seq2seq with robust preprocessing (median impute + safe standardize) ====
import duckdb, pandas as pd, numpy as np, datetime as dt, math
import torch, torch.nn as nn
from sklearn.metrics import mean_absolute_error

DB = "../ProjectMain/db/data.duckdb"
DELIVERY_DATE = pd.Timestamp("2025-08-18").date()   # change target date if needed
W_PAST = 168
H_FUT = 24
HOLDOUT_DAYS = 60
HIDDEN = 128
EPOCHS = 20
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
rng = np.random.default_rng(42)
torch.manual_seed(42)

# ---------- helpers ----------
def fit_standardizer(X2d):
    """Return (mean,std) with std guarded (==0 -> 1). X2d: (N, F)"""
    mu = np.nanmean(X2d, axis=0)
    sd = np.nanstd(X2d, axis=0)
    sd = np.where(sd <= 1e-8, 1.0, sd)
    return mu, sd

def apply_standardizer(X, mu, sd):
    return (X - mu) / sd

def col_median(X2d):
    """Median per feature, ignoring NaNs"""
    return np.nanmedian(X2d, axis=0)

# ---------- ensure enhanced views exist (already fixed earlier); skip if you ran that cell ----------

# ---------- pull data ----------
con = duckdb.connect(DB)
df_lags = con.execute("""
SELECT
  ts, OperatingDTM, Interval, hb_houston,
  p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
  dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean
FROM vw_master_spine_lags
ORDER BY ts
""").df()
df_lags["ts"] = pd.to_datetime(df_lags["ts"])

df_future = con.execute("""
SELECT
  f.OperatingDTM AS delivery_date,
  f.Interval     AS delivery_interval,
  f.hb_houston   AS target,
  -- base future features
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.temp_the_woodlands_tx, f.hum_the_woodlands_tx, f.wind_the_woodlands_tx, f.precip_the_woodlands_tx,
  f.temp_katy_tx, f.hum_katy_tx, f.wind_katy_tx, f.precip_katy_tx,
  f.temp_friendswood_tx, f.hum_friendswood_tx, f.wind_friendswood_tx, f.precip_friendswood_tx,
  f.temp_baytown_tx, f.hum_baytown_tx, f.wind_baytown_tx, f.precip_baytown_tx,
  f.temp_houston_tx, f.hum_houston_tx, f.wind_houston_tx, f.precip_houston_tx,
  f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
  f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f,
  f.cal_sin_dow  AS cal_sin_dow_f,  f.cal_cos_dow  AS cal_cos_dow_f,
  -- enhanced joins
  l.net_wz, l.net_lz, l.net_wz_ramp1, l.net_lz_ramp1, l.lz_houston_ramp1, l.wz_spread, l.lz_spread,
  w.temp_avg, w.temp_spread, w.temp_avg_ramp1, w.hum_avg, w.hum_spread, w.hum_avg_ramp1
FROM vw_master_spine_ts f
LEFT JOIN load_fcst_enh l
  ON l.OperatingDTM = f.OperatingDTM AND l.Interval = f.Interval
LEFT JOIN wx_fcst_enh w
  ON w.OperatingDTM = f.OperatingDTM AND w.Interval = f.Interval
ORDER BY f.OperatingDTM, f.Interval
""").df()
con.close()

df_future["delivery_date"] = pd.to_datetime(df_future["delivery_date"]).dt.date
df_future["delivery_interval"] = df_future["delivery_interval"].astype(int)

# ---------- assemble samples (day-ahead) ----------
enc_cols = [
    "hb_houston","p_lag1","p_lag2","p_lag3","p_lag6","p_lag12","p_lag24","p_lag48","p_lag72","p_lag168",
    "dp1","dp24","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean"
]
dec_future_cols = [
    "cal_hour_f","cal_dow_f","cal_is_weekend_f","cal_sin_hour_f","cal_cos_hour_f","cal_sin_dow_f","cal_cos_dow_f",
    "net_wz","net_lz","net_wz_ramp1","net_lz_ramp1","lz_houston_ramp1","wz_spread","lz_spread",
    "temp_avg","temp_spread","temp_avg_ramp1","hum_avg","hum_spread","hum_avg_ramp1"
]

# base rows = end-of-day with price
base_rows = df_lags[(df_lags["hb_houston"].notna()) & (df_lags["Interval"]==24)][["ts","OperatingDTM","hb_houston"]].copy()
base_rows["base_date"] = pd.to_datetime(base_rows["OperatingDTM"]).dt.date
base_rows["delivery_date"] = base_rows["base_date"] + dt.timedelta(days=1)

# only use delivery days with 24 actuals for training/val
has24 = df_future.groupby("delivery_date")["target"].apply(lambda s: s.notna().sum()==24)
valid_delivery_dates = set(has24[has24].index)

df_lags_idx = df_lags.set_index("ts").sort_index()

samples = []
for _, r in base_rows.iterrows():
    ddate = r["delivery_date"]
    ts0 = r["ts"]
    # encoder window
    start = ts0 - pd.Timedelta(hours=W_PAST-1)
    enc_df = df_lags_idx.loc[start:ts0][enc_cols]
    if enc_df.shape[0] != W_PAST: 
        continue
    if enc_df.isna().any().any():
        continue  # strict: skip any encoder NaNs
    # decoder future rows
    fdf = df_future[(df_future["delivery_date"]==ddate)].sort_values("delivery_interval")
    if fdf.shape[0] != H_FUT:
        continue
    X_enc = enc_df.values.astype(np.float32)
    X_dec = fdf[dec_future_cols].values.astype(np.float32)
    y     = fdf["target"].values.astype(np.float32)
    y0    = np.float32(r["hb_houston"])
    samples.append((X_enc, X_dec, y, y0, ddate))

if len(samples) == 0:
    raise RuntimeError("No samples assembled; check data coverage or reduce W_PAST.")

# split by date
dates = sorted({s[4] for s in samples})
cut_date = dates[-1] - dt.timedelta(days=HOLDOUT_DAYS) if len(dates)>HOLDOUT_DAYS else dates[0]
train_idx = [i for i,s in enumerate(samples) if s[4] <= cut_date and s[4] in valid_delivery_dates]
val_idx   = [i for i,s in enumerate(samples) if s[4] >  cut_date and s[4] in valid_delivery_dates]

def stack(idxs):
    Xe = np.stack([samples[i][0] for i in idxs], axis=0)
    Xd = np.stack([samples[i][1] for i in idxs], axis=0)
    Y  = np.stack([samples[i][2] for i in idxs], axis=0)
    Y0 = np.stack([samples[i][3] for i in idxs], axis=0)
    return Xe, Xd, Y, Y0

Xe_tr, Xd_tr, Y_tr, Y0_tr = stack(train_idx)
Xe_va, Xd_va, Y_va, Y0_va = (None, None, None, None)
if len(val_idx) > 0:
    Xe_va, Xd_va, Y_va, Y0_va = stack(val_idx)

# ---------- impute decoder NaNs with training medians ----------
# (encoder already filtered for NaNs above; still guard just in case)
enc_med = np.nanmedian(Xe_tr.reshape(-1, Xe_tr.shape[2]), axis=0)
dec_med = np.nanmedian(Xd_tr.reshape(-1, Xd_tr.shape[2]), axis=0)

def impute_inplace(X, med):
    m = np.isnan(X)
    if m.any():
        X[m] = np.take(med, np.where(m)[2])

impute_inplace(Xe_tr, enc_med)
impute_inplace(Xd_tr, dec_med)
if Xe_va is not None:
    impute_inplace(Xe_va, enc_med)
    impute_inplace(Xd_va, dec_med)

# Targets should be complete for training/val; no imputation for y

# ---------- safe standardization ----------
enc_mu, enc_sd = fit_standardizer(Xe_tr.reshape(-1, Xe_tr.shape[2]))
dec_mu, dec_sd = fit_standardizer(Xd_tr.reshape(-1, Xd_tr.shape[2]))
y_mu,  y_sd    = fit_standardizer(Y_tr.reshape(-1,1))

Xe_tr_n = apply_standardizer(Xe_tr, enc_mu, enc_sd)
Xd_tr_n = apply_standardizer(Xd_tr, dec_mu, dec_sd)
Y_tr_n  = apply_standardizer(Y_tr,  y_mu,  y_sd).reshape(Y_tr.shape)
Y0_tr_n = apply_standardizer(Y0_tr.reshape(-1,1), y_mu, y_sd).reshape(-1)

if Xe_va is not None:
    Xe_va_n = apply_standardizer(Xe_va, enc_mu, enc_sd)
    Xd_va_n = apply_standardizer(Xd_va, dec_mu, dec_sd)
    Y_va_n  = apply_standardizer(Y_va,  y_mu,  y_sd).reshape(Y_va.shape)
    Y0_va_n = apply_standardizer(Y0_va.reshape(-1,1), y_mu, y_sd).reshape(-1)

# ---------- torch datasets ----------
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self, Xe, Xd, Y, Y0):
        self.Xe = torch.from_numpy(Xe).float()
        self.Xd = torch.from_numpy(Xd).float()
        self.Y  = torch.from_numpy(Y).float()
        self.Y0 = torch.from_numpy(Y0).float()
    def __len__(self): return self.Xe.shape[0]
    def __getitem__(self, i): return self.Xe[i], self.Xd[i], self.Y[i], self.Y0[i]

bs = 32
train_loader = torch.utils.data.DataLoader(Seq2SeqDataset(Xe_tr_n, Xd_tr_n, Y_tr_n, Y0_tr_n), batch_size=bs, shuffle=True)
val_loader = None if Xe_va is None else torch.utils.data.DataLoader(Seq2SeqDataset(Xe_va_n, Xd_va_n, Y_va_n, Y0_va_n), batch_size=bs, shuffle=False)

# ---------- model ----------
class Encoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim, hidden, batch_first=True)
    def forward(self, x):  # x: (B,W,E)
        _, h = self.rnn(x)
        return h  # (1,B,H)

class Decoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim + 1, hidden, batch_first=True)  # + prev y
        self.out = nn.Linear(hidden, 1)
    def forward(self, future_feats, h0, y0, teacher=None, tf_prob=0.5):
        B, T, D = future_feats.shape
        y_prev = y0.view(B,1)
        h = h0
        outs = []
        for t in range(T):
            x_t = torch.cat([future_feats[:,t,:], y_prev], dim=1).unsqueeze(1)  # (B,1,D+1)
            o, h = self.rnn(x_t, h)
            y_t = self.out(o[:, -1, :]).squeeze(1)
            outs.append(y_t)
            if (teacher is not None) and (rng.random() < tf_prob):
                y_prev = teacher[:,t].view(B,1)
            else:
                y_prev = y_t.view(B,1)
        return torch.stack(outs, dim=1)  # (B,24)

class Seq2Seq(nn.Module):
    def __init__(self, enc_in, dec_in, hidden):
        super().__init__()
        self.enc = Encoder(enc_in, hidden)
        self.dec = Decoder(dec_in, hidden)
    def forward(self, x_enc, x_dec, y0, y_teacher=None, tf_prob=0.5):
        h = self.enc(x_enc)
        return self.dec(x_dec, h, y0, y_teacher, tf_prob)

enc_in = Xe_tr_n.shape[2]
dec_in = Xd_tr_n.shape[2]
model = Seq2Seq(enc_in, dec_in, HIDDEN).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.HuberLoss()

def train_epoch():
    model.train()
    tot = 0.0
    for xenc, xdec, y, y0 in train_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        opt.zero_grad()
        yhat = model(xenc, xdec, y0, y_teacher=y, tf_prob=0.5)
        loss = loss_fn(yhat, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        tot += loss.item() * xenc.size(0)
    return tot / len(train_loader.dataset)

@torch.no_grad()
def eval_epoch():
    if val_loader is None: return None
    model.eval()
    tot = 0.0
    for xenc, xdec, y, y0 in val_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        yhat = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0)
        loss = loss_fn(yhat, y)
        tot += loss.item() * xenc.size(0)
    return tot / len(val_loader.dataset)

for ep in range(1, EPOCHS+1):
    tr = train_epoch()
    va = eval_epoch()
    print(f"Epoch {ep:02d}  train_loss={tr:.4f}" + ("" if va is None else f"  val_loss={va:.4f}"))

# ---- evaluate holdout MAE in real units ----
if val_loader is not None:
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xenc, xdec, y, y0 in val_loader:
            xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
            yhat_n = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0).detach().cpu().numpy()
            y_n    = y.detach().cpu().numpy()
            # invert scaling
            yhat = (yhat_n * y_sd + y_mu).reshape(-1)
            yt   = (y_n    * y_sd + y_mu).reshape(-1)
            preds.append(yhat); trues.append(yt)
    preds = np.concatenate(preds); trues = np.concatenate(trues)
    print({"holdout_MAE": float(mean_absolute_error(trues, preds))})

# ---- predict requested DELIVERY_DATE ----
# find base row (previous day EOD with price)
prev_day = DELIVERY_DATE - dt.timedelta(days=1)
base_row = df_lags[(pd.to_datetime(df_lags["OperatingDTM"]).dt.date == prev_day) & (df_lags["Interval"]==24) & df_lags["hb_houston"].notna()]
if base_row.empty:
    raise RuntimeError(f"No base EOD price for {prev_day}")
ts0 = pd.to_datetime(base_row.iloc[0]["ts"])
y0  = np.float32(base_row.iloc[0]["hb_houston"])

# encoder window
win = df_lags.set_index("ts").loc[ts0 - pd.Timedelta(hours=W_PAST-1): ts0][enc_cols]
if win.shape[0] != W_PAST or win.isna().any().any():
    raise RuntimeError("Insufficient/NaN history for encoder window.")
Xe_pred = win.values.astype(np.float32)[None, ...]

# decoder future features (24 rows)
frows = df_future[(df_future["delivery_date"]==DELIVERY_DATE)].sort_values("delivery_interval")
if frows.shape[0] != H_FUT:
    raise RuntimeError("Spine missing 24 rows for delivery date.")
Xd_pred = frows[dec_future_cols].values.astype(np.float32)[None, ...]

# impute + standardize pred
# impute decoder NaNs
m = np.isnan(Xd_pred)
if m.any():
    Xd_pred[m] = np.take(dec_med, np.where(m)[2])
Xe_pred_n = apply_standardizer(Xe_pred, enc_mu, enc_sd)
Xd_pred_n = apply_standardizer(Xd_pred, dec_mu, dec_sd)
y0_n = ((y0 - y_mu) / y_sd).reshape(()).astype(np.float32)

model.eval()
with torch.no_grad():
    yhat_n = model(torch.from_numpy(Xe_pred_n).to(DEVICE),
                   torch.from_numpy(Xd_pred_n).to(DEVICE),
                   torch.tensor([y0_n], dtype=torch.float32, device=DEVICE),
                   y_teacher=None, tf_prob=0.0).detach().cpu().numpy()[0]
yhat = (yhat_n * y_sd + y_mu).reshape(-1)

forecast = pd.DataFrame({
    "OperatingDTM": [DELIVERY_DATE]*H_FUT,
    "Interval": list(range(1, H_FUT+1)),
    "hb_houston_pred": yhat.tolist()
})
forecast


c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1216: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a, func=_nanmedian, keepdims=keepdims,
C:\Users\db\AppData\Local\Temp\ipykernel_6672\893039283.py:21: RuntimeWarning: Mean of empty slice
  mu = np.nanmean(X2d, axis=0)
c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Epoch 01  train_loss=nan  val_loss=nan
Epoch 02  train_loss=nan  val_loss=nan
Epoch 03  train_loss=nan  val_loss=nan
Epoch 04  train_loss=nan  val_loss=nan
Epoch 05  train_loss=nan  val_loss=nan
Epoch 06  train_loss=nan  val_loss=nan
Epoch 07  train_loss=nan  val_loss=nan
Epoch 08  train_loss=nan  val_loss=nan
Epoch 09  train_loss=nan  val_loss=nan
Epoch 10  train_loss=nan  val_loss=nan
Epoch 11  train_loss=nan  val_loss=nan
Epoch 12  train_loss=nan  val_loss=nan
Epoch 13  train_loss=nan  val_loss=nan
Epoch 14  train_loss=nan  val_loss=nan
Epoch 15  train_loss=nan  val_loss=nan
Epoch 16  train_loss=nan  val_loss=nan
Epoch 17  train_loss=nan  val_loss=nan
Epoch 18  train_loss=nan  val_loss=nan
Epoch 19  train_loss=nan  val_loss=nan
Epoch 20  train_loss=nan  val_loss=nan


ValueError: Input contains NaN.

In [11]:
# ==== PyTorch seq2seq with ROBUST preprocessing (median impute + safe standardize + safe MAE) ====
import duckdb, pandas as pd, numpy as np, datetime as dt, math
import torch, torch.nn as nn
from sklearn.metrics import mean_absolute_error

DB = "../ProjectMain/db/data.duckdb"
DELIVERY_DATE = pd.Timestamp("2025-08-18").date()   # change target date if needed
W_PAST = 168
H_FUT = 24
HOLDOUT_DAYS = 60
HIDDEN = 128
EPOCHS = 20
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
rng = np.random.default_rng(42)
torch.manual_seed(42)

# ---------- helpers ----------
def fit_standardizer(X2d):
    """Return (mean,std) with std guarded (==0 -> 1). X2d: (N, F)"""
    mu = np.nanmean(X2d, axis=0)
    sd = np.nanstd(X2d, axis=0)
    sd = np.where(sd <= 1e-8, 1.0, sd)
    return mu, sd

def apply_standardizer(X, mu, sd):
    return (X - mu) / sd

# robust 3D imputer
def impute_inplace_3d(X, med_vec):
    """X: (N, T, F) or (N, W, F); med_vec: (F,). Fills NaNs in-place with per-feature medians."""
    mask = np.isnan(X)
    if mask.any():
        med_broadcast = np.broadcast_to(med_vec, X.shape)
        X[mask] = med_broadcast[mask]

# sanity checker
def assert_finite(name, arr):
    if not np.isfinite(arr).all():
        bad = int(np.isnan(arr).sum() + np.isinf(arr).sum())
        raise RuntimeError(f"{name} has {bad} non-finite values")

# ---------- pull data ----------
con = duckdb.connect(DB)
df_lags = con.execute("""
SELECT
  ts, OperatingDTM, Interval, hb_houston,
  p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
  dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean
FROM vw_master_spine_lags
ORDER BY ts
""").df()
df_lags["ts"] = pd.to_datetime(df_lags["ts"])

df_future = con.execute("""
SELECT
  f.OperatingDTM AS delivery_date,
  f.Interval     AS delivery_interval,
  f.hb_houston   AS target,
  -- base future features
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.temp_the_woodlands_tx, f.hum_the_woodlands_tx, f.wind_the_woodlands_tx, f.precip_the_woodlands_tx,
  f.temp_katy_tx, f.hum_katy_tx, f.wind_katy_tx, f.precip_katy_tx,
  f.temp_friendswood_tx, f.hum_friendswood_tx, f.wind_friendswood_tx, f.precip_friendswood_tx,
  f.temp_baytown_tx, f.hum_baytown_tx, f.wind_baytown_tx, f.precip_baytown_tx,
  f.temp_houston_tx, f.hum_houston_tx, f.wind_houston_tx, f.precip_houston_tx,
  f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
  f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f,
  f.cal_sin_dow  AS cal_sin_dow_f,  f.cal_cos_dow  AS cal_cos_dow_f,
  -- enhanced joins
  l.net_wz, l.net_lz, l.net_wz_ramp1, l.net_lz_ramp1, l.lz_houston_ramp1, l.wz_spread, l.lz_spread,
  w.temp_avg, w.temp_spread, w.temp_avg_ramp1, w.hum_avg, w.hum_spread, w.hum_avg_ramp1
FROM vw_master_spine_ts f
LEFT JOIN load_fcst_enh l
  ON l.OperatingDTM = f.OperatingDTM AND l.Interval = f.Interval
LEFT JOIN wx_fcst_enh w
  ON w.OperatingDTM = f.OperatingDTM AND w.Interval = f.Interval
ORDER BY f.OperatingDTM, f.Interval
""").df()
con.close()

df_future["delivery_date"] = pd.to_datetime(df_future["delivery_date"]).dt.date
df_future["delivery_interval"] = df_future["delivery_interval"].astype(int)

# ---------- assemble samples (day-ahead) ----------
enc_cols = [
    "hb_houston","p_lag1","p_lag2","p_lag3","p_lag6","p_lag12","p_lag24","p_lag48","p_lag72","p_lag168",
    "dp1","dp24","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean"
]
dec_future_cols = [
    "cal_hour_f","cal_dow_f","cal_is_weekend_f","cal_sin_hour_f","cal_cos_hour_f","cal_sin_dow_f","cal_cos_dow_f",
    "net_wz","net_lz","net_wz_ramp1","net_lz_ramp1","lz_houston_ramp1","wz_spread","lz_spread",
    "temp_avg","temp_spread","temp_avg_ramp1","hum_avg","hum_spread","hum_avg_ramp1"
]

# base rows = end-of-day with price
base_rows = df_lags[(df_lags["hb_houston"].notna()) & (df_lags["Interval"]==24)][["ts","OperatingDTM","hb_houston"]].copy()
base_rows["base_date"] = pd.to_datetime(base_rows["OperatingDTM"]).dt.date
base_rows["delivery_date"] = base_rows["base_date"] + dt.timedelta(days=1)

# only use delivery days with 24 actuals for training/val
has24 = df_future.groupby("delivery_date")["target"].apply(lambda s: s.notna().sum()==24)
valid_delivery_dates = set(has24[has24].index)

df_lags_idx = df_lags.set_index("ts").sortindex() if hasattr(pd.DataFrame, 'sortindex') else df_lags.set_index("ts").sort_index()

samples = []
for _, r in base_rows.iterrows():
    ddate = r["delivery_date"]
    ts0 = r["ts"]
    # encoder window
    start = ts0 - pd.Timedelta(hours=W_PAST-1)
    enc_df = df_lags_idx.loc[start:ts0][enc_cols]
    if enc_df.shape[0] != W_PAST: 
        continue
    if enc_df.isna().any().any():
        continue  # strict: skip any encoder NaNs
    # decoder future rows
    fdf = df_future[(df_future["delivery_date"]==ddate)].sort_values("delivery_interval")
    if fdf.shape[0] != H_FUT:
        continue
    X_enc = enc_df.values.astype(np.float32)
    X_dec = fdf[dec_future_cols].values.astype(np.float32)
    y     = fdf["target"].values.astype(np.float32)
    y0    = np.float32(r["hb_houston"])
    samples.append((X_enc, X_dec, y, y0, ddate))

if len(samples) == 0:
    raise RuntimeError("No samples assembled; check data coverage or reduce W_PAST.")

# split by date
dates = sorted({s[4] for s in samples})
cut_date = dates[-1] - dt.timedelta(days=HOLDOUT_DAYS) if len(dates)>HOLDOUT_DAYS else dates[0]
train_idx = [i for i,s in enumerate(samples) if s[4] <= cut_date and s[4] in valid_delivery_dates]
val_idx   = [i for i,s in enumerate(samples) if s[4] >  cut_date and s[4] in valid_delivery_dates]

def stack(idxs):
    Xe = np.stack([samples[i][0] for i in idxs], axis=0)
    Xd = np.stack([samples[i][1] for i in idxs], axis=0)
    Y  = np.stack([samples[i][2] for i in idxs], axis=0)
    Y0 = np.stack([samples[i][3] for i in idxs], axis=0)
    return Xe, Xd, Y, Y0

Xe_tr, Xd_tr, Y_tr, Y0_tr = stack(train_idx)
Xe_va, Xd_va, Y_va, Y0_va = (None, None, None, None)
if len(val_idx) > 0:
    Xe_va, Xd_va, Y_va, Y0_va = stack(val_idx)

# ---------- impute NaNs (robust) ----------
enc_med = np.nanmedian(Xe_tr.reshape(-1, Xe_tr.shape[2]), axis=0)
dec_med = np.nanmedian(Xd_tr.reshape(-1, Xd_tr.shape[2]), axis=0)

impute_inplace_3d(Xe_tr, enc_med)
impute_inplace_3d(Xd_tr, dec_med)
if Xe_va is not None:
    impute_inplace_3d(Xe_va, enc_med)
    impute_inplace_3d(Xd_va, dec_med)

# Targets should be complete for training/val; assert
assert np.isfinite(Y_tr).all(), "Y_tr has non-finite values"
if Y_va is not None:
    assert np.isfinite(Y_va).all(), "Y_va has non-finite values"

# ---------- safe standardization ----------
enc_mu, enc_sd = fit_standardizer(Xe_tr.reshape(-1, Xe_tr.shape[2]))
dec_mu, dec_sd = fit_standardizer(Xd_tr.reshape(-1, Xd_tr.shape[2]))
y_mu,  y_sd    = fit_standardizer(Y_tr.reshape(-1,1))

Xe_tr_n = apply_standardizer(Xe_tr, enc_mu, enc_sd)
Xd_tr_n = apply_standardizer(Xd_tr, dec_mu, dec_sd)
Y_tr_n  = apply_standardizer(Y_tr,  y_mu,  y_sd).reshape(Y_tr.shape)
Y0_tr_n = apply_standardizer(Y0_tr.reshape(-1,1), y_mu, y_sd).reshape(-1)

if Xe_va is not None:
    Xe_va_n = apply_standardizer(Xe_va, enc_mu, enc_sd)
    Xd_va_n = apply_standardizer(Xd_va, dec_mu, dec_sd)
    Y_va_n  = apply_standardizer(Y_va,  y_mu,  y_sd).reshape(Y_va.shape)
    Y0_va_n = apply_standardizer(Y0_va.reshape(-1,1), y_mu, y_sd).reshape(-1)

# hard sanity checks
assert_finite("Xe_tr_n", Xe_tr_n)
assert_finite("Xd_tr_n", Xd_tr_n)
assert_finite("Y_tr_n",  Y_tr_n)
assert_finite("Y0_tr_n", Y0_tr_n)
if Xe_va is not None:
    assert_finite("Xe_va_n", Xe_va_n)
    assert_finite("Xd_va_n", Xd_va_n)
    assert_finite("Y_va_n",  Y_va_n)
    assert_finite("Y0_va_n", Y0_va_n)

# ---------- torch datasets ----------
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self, Xe, Xd, Y, Y0):
        self.Xe = torch.from_numpy(Xe).float()
        self.Xd = torch.from_numpy(Xd).float()
        self.Y  = torch.from_numpy(Y).float()
        self.Y0 = torch.from_numpy(Y0).float()
    def __len__(self): return self.Xe.shape[0]
    def __getitem__(self, i): return self.Xe[i], self.Xd[i], self.Y[i], self.Y0[i]

bs = 32
train_loader = torch.utils.data.DataLoader(Seq2SeqDataset(Xe_tr_n, Xd_tr_n, Y_tr_n, Y0_tr_n), batch_size=bs, shuffle=True)
val_loader = None if Xe_va is None else torch.utils.data.DataLoader(Seq2SeqDataset(Xe_va_n, Xd_va_n, Y_va_n, Y0_va_n), batch_size=bs, shuffle=False)

# ---------- model ----------
class Encoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim, hidden, batch_first=True)
    def forward(self, x):  # x: (B,W,E)
        _, h = self.rnn(x)
        return h  # (1,B,H)

class Decoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim + 1, hidden, batch_first=True)  # + prev y
        self.out = nn.Linear(hidden, 1)
    def forward(self, future_feats, h0, y0, teacher=None, tf_prob=0.5):
        B, T, D = future_feats.shape
        y_prev = y0.view(B,1)
        h = h0
        outs = []
        for t in range(T):
            x_t = torch.cat([future_feats[:,t,:], y_prev], dim=1).unsqueeze(1)  # (B,1,D+1)
            o, h = self.rnn(x_t, h)
            y_t = self.out(o[:, -1, :]).squeeze(1)
            outs.append(y_t)
            if (teacher is not None) and (rng.random() < tf_prob):
                y_prev = teacher[:,t].view(B,1)
            else:
                y_prev = y_t.view(B,1)
        return torch.stack(outs, dim=1)  # (B,24)

class Seq2Seq(nn.Module):
    def __init__(self, enc_in, dec_in, hidden):
        super().__init__()
        self.enc = Encoder(enc_in, hidden)
        self.dec = Decoder(dec_in, hidden)
    def forward(self, x_enc, x_dec, y0, y_teacher=None, tf_prob=0.5):
        h = self.enc(x_enc)
        return self.dec(x_dec, h, y0, y_teacher, tf_prob)

enc_in = Xe_tr_n.shape[2]
dec_in = Xd_tr_n.shape[2]
model = Seq2Seq(enc_in, dec_in, HIDDEN).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.HuberLoss()

def train_epoch():
    model.train()
    tot = 0.0
    for xenc, xdec, y, y0 in train_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        opt.zero_grad()
        yhat = model(xenc, xdec, y0, y_teacher=y, tf_prob=0.5)
        loss = loss_fn(yhat, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        tot += loss.item() * xenc.size(0)
    return tot / len(train_loader.dataset)

@torch.no_grad()
def eval_epoch():
    if val_loader is None: return None
    model.eval()
    tot = 0.0
    for xenc, xdec, y, y0 in val_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        yhat = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0)
        loss = loss_fn(yhat, y)
        tot += loss.item() * xenc.size(0)
    return tot / len(val_loader.dataset)

for ep in range(1, EPOCHS+1):
    tr = train_epoch()
    va = eval_epoch()
    print(f"Epoch {ep:02d}  train_loss={tr:.4f}" + ("" if va is None else f"  val_loss={va:.4f}"))

# ---- evaluate holdout MAE in real units (SAFE) ----
if val_loader is not None:
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xenc, xdec, y, y0 in val_loader:
            xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
            yhat_n = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0).detach().cpu().numpy()
            y_n    = y.detach().cpu().numpy()
            # invert scaling
            yhat = (yhat_n * y_sd + y_mu)
            yt   = (y_n    * y_sd + y_mu)
            preds.append(yhat.reshape(-1))
            trues.append(yt.reshape(-1))
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    mask = np.isfinite(preds) & np.isfinite(trues)
    dropped = int((~mask).sum())
    if dropped:
        print(f"Dropped {dropped} non-finite pairs from holdout scoring.")
    print({"holdout_MAE": float(mean_absolute_error(trues[mask], preds[mask]))})

# ---- predict requested DELIVERY_DATE ----
# find base row (previous day EOD with price)
prev_day = DELIVERY_DATE - dt.timedelta(days=1)
base_row = df_lags[(pd.to_datetime(df_lags["OperatingDTM"]).dt.date == prev_day) & (df_lags["Interval"]==24) & df_lags["hb_houston"].notna()]
if base_row.empty:
    raise RuntimeError(f"No base EOD price for {prev_day}")
ts0 = pd.to_datetime(base_row.iloc[0]["ts"])
y0  = np.float32(base_row.iloc[0]["hb_houston"])

# encoder window
win = df_lags.set_index("ts").loc[ts0 - pd.Timedelta(hours=W_PAST-1): ts0][enc_cols]
if win.shape[0] != W_PAST or win.isna().any().any():
    raise RuntimeError("Insufficient/NaN history for encoder window.")
Xe_pred = win.values.astype(np.float32)[None, ...]

# decoder future features (24 rows)
frows = df_future[(df_future["delivery_date"]==DELIVERY_DATE)].sort_values("delivery_interval")
if frows.shape[0] != H_FUT:
    raise RuntimeError("Spine missing 24 rows for delivery date.")
Xd_pred = frows[dec_future_cols].values.astype(np.float32)[None, ...]

# impute + standardize pred
m = np.isnan(Xd_pred)
if m.any():
    # dec_med computed on train; reuse here
    Xd_pred[m] = np.broadcast_to(dec_med, Xd_pred.shape)[m]
Xe_pred_n = apply_standardizer(Xe_pred, enc_mu, enc_sd)
Xd_pred_n = apply_standardizer(Xd_pred, dec_mu, dec_sd)
y0_n = ((y0 - y_mu) / y_sd).reshape(()).astype(np.float32)

model.eval()
with torch.no_grad():
    yhat_n = model(torch.from_numpy(Xe_pred_n).to(DEVICE),
                   torch.from_numpy(Xd_pred_n).to(DEVICE),
                   torch.tensor([y0_n], dtype=torch.float32, device=DEVICE),
                   y_teacher=None, tf_prob=0.0).detach().cpu().numpy()[0]
yhat = (yhat_n * y_sd + y_mu).reshape(-1)
# replace any residual NaN with the day's median
if np.isnan(yhat).any():
    day_med = float(np.nanmedian(yhat))
    yhat = np.nan_to_num(yhat, nan=day_med)

forecast = pd.DataFrame({
    "OperatingDTM": [DELIVERY_DATE]*H_FUT,
    "Interval": list(range(1, H_FUT+1)),
    "hb_houston_pred": yhat.tolist()
})
forecast


c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1216: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a, func=_nanmedian, keepdims=keepdims,
C:\Users\db\AppData\Local\Temp\ipykernel_6672\3780957588.py:21: RuntimeWarning: Mean of empty slice
  mu = np.nanmean(X2d, axis=0)
c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


RuntimeError: Xd_tr_n has 122112 non-finite values

In [12]:
# ==== PyTorch seq2seq with ROBUST preprocessing (median impute + safe standardize + safe MAE) ====
import duckdb, pandas as pd, numpy as np, datetime as dt, math
import torch, torch.nn as nn
from sklearn.metrics import mean_absolute_error

DB = "../ProjectMain/db/data.duckdb"
DELIVERY_DATE = pd.Timestamp("2025-08-18").date()   # change target date if needed
W_PAST = 168
H_FUT = 24
HOLDOUT_DAYS = 60
HIDDEN = 128
EPOCHS = 20
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
rng = np.random.default_rng(42)
torch.manual_seed(42)

# ---------- helpers ----------
def fit_standardizer(X2d):
    """Return (mean,std) with std guarded (==0 -> 1). X2d: (N, F)"""
    mu = np.nanmean(X2d, axis=0)
    sd = np.nanstd(X2d, axis=0)
    sd = np.where(sd <= 1e-8, 1.0, sd)
    return mu, sd

def apply_standardizer(X, mu, sd):
    return (X - mu) / sd

# robust 3D imputer
def impute_inplace_3d(X, med_vec):
    """X: (N, T, F) or (N, W, F); med_vec: (F,). Fills NaNs in-place with per-feature medians."""
    mask = np.isnan(X)
    if mask.any():
        med_broadcast = np.broadcast_to(med_vec, X.shape)
        X[mask] = med_broadcast[mask]

# sanity checker
def assert_finite(name, arr):
    if not np.isfinite(arr).all():
        bad = int(np.isnan(arr).sum() + np.isinf(arr).sum())
        raise RuntimeError(f"{name} has {bad} non-finite values")

# ---------- pull data ----------
con = duckdb.connect(DB)
df_lags = con.execute("""
SELECT
  ts, OperatingDTM, Interval, hb_houston,
  p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
  dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean
FROM vw_master_spine_lags
ORDER BY ts
""").df()
df_lags["ts"] = pd.to_datetime(df_lags["ts"])

df_future = con.execute("""
SELECT
  f.OperatingDTM AS delivery_date,
  f.Interval     AS delivery_interval,
  f.hb_houston   AS target,
  -- base future features
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.temp_the_woodlands_tx, f.hum_the_woodlands_tx, f.wind_the_woodlands_tx, f.precip_the_woodlands_tx,
  f.temp_katy_tx, f.hum_katy_tx, f.wind_katy_tx, f.precip_katy_tx,
  f.temp_friendswood_tx, f.hum_friendswood_tx, f.wind_friendswood_tx, f.precip_friendswood_tx,
  f.temp_baytown_tx, f.hum_baytown_tx, f.wind_baytown_tx, f.precip_baytown_tx,
  f.temp_houston_tx, f.hum_houston_tx, f.wind_houston_tx, f.precip_houston_tx,
  f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
  f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f,
  f.cal_sin_dow  AS cal_sin_dow_f,  f.cal_cos_dow  AS cal_cos_dow_f,
  -- enhanced joins
  l.net_wz, l.net_lz, l.net_wz_ramp1, l.net_lz_ramp1, l.lz_houston_ramp1, l.wz_spread, l.lz_spread,
  w.temp_avg, w.temp_spread, w.temp_avg_ramp1, w.hum_avg, w.hum_spread, w.hum_avg_ramp1
FROM vw_master_spine_ts f
LEFT JOIN load_fcst_enh l
  ON l.OperatingDTM = f.OperatingDTM AND l.Interval = f.Interval
LEFT JOIN wx_fcst_enh w
  ON w.OperatingDTM = f.OperatingDTM AND w.Interval = f.Interval
ORDER BY f.OperatingDTM, f.Interval
""").df()
con.close()

df_future["delivery_date"] = pd.to_datetime(df_future["delivery_date"]).dt.date
df_future["delivery_interval"] = df_future["delivery_interval"].astype(int)

# ---------- assemble samples (day-ahead) ----------
enc_cols = [
    "hb_houston","p_lag1","p_lag2","p_lag3","p_lag6","p_lag12","p_lag24","p_lag48","p_lag72","p_lag168",
    "dp1","dp24","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean"
]
dec_future_cols = [
    "cal_hour_f","cal_dow_f","cal_is_weekend_f","cal_sin_hour_f","cal_cos_hour_f","cal_sin_dow_f","cal_cos_dow_f",
    "net_wz","net_lz","net_wz_ramp1","net_lz_ramp1","lz_houston_ramp1","wz_spread","lz_spread",
    "temp_avg","temp_spread","temp_avg_ramp1","hum_avg","hum_spread","hum_avg_ramp1"
]

# base rows = end-of-day with price
base_rows = df_lags[(df_lags["hb_houston"].notna()) & (df_lags["Interval"]==24)][["ts","OperatingDTM","hb_houston"]].copy()
base_rows["base_date"] = pd.to_datetime(base_rows["OperatingDTM"]).dt.date
base_rows["delivery_date"] = base_rows["base_date"] + dt.timedelta(days=1)

# only use delivery days with 24 actuals for training/val
has24 = df_future.groupby("delivery_date")["target"].apply(lambda s: s.notna().sum()==24)
valid_delivery_dates = set(has24[has24].index)

df_lags_idx = df_lags.set_index("ts").sortindex() if hasattr(pd.DataFrame, 'sortindex') else df_lags.set_index("ts").sort_index()

samples = []
for _, r in base_rows.iterrows():
    ddate = r["delivery_date"]
    ts0 = r["ts"]
    # encoder window
    start = ts0 - pd.Timedelta(hours=W_PAST-1)
    enc_df = df_lags_idx.loc[start:ts0][enc_cols]
    if enc_df.shape[0] != W_PAST: 
        continue
    if enc_df.isna().any().any():
        continue  # strict: skip any encoder NaNs
    # decoder future rows
    fdf = df_future[(df_future["delivery_date"]==ddate)].sort_values("delivery_interval")
    if fdf.shape[0] != H_FUT:
        continue
    X_enc = enc_df.values.astype(np.float32)
    X_dec = fdf[dec_future_cols].values.astype(np.float32)
    y     = fdf["target"].values.astype(np.float32)
    y0    = np.float32(r["hb_houston"])
    samples.append((X_enc, X_dec, y, y0, ddate))

if len(samples) == 0:
    raise RuntimeError("No samples assembled; check data coverage or reduce W_PAST.")

# split by date
dates = sorted({s[4] for s in samples})
cut_date = dates[-1] - dt.timedelta(days=HOLDOUT_DAYS) if len(dates)>HOLDOUT_DAYS else dates[0]
train_idx = [i for i,s in enumerate(samples) if s[4] <= cut_date and s[4] in valid_delivery_dates]
val_idx   = [i for i,s in enumerate(samples) if s[4] >  cut_date and s[4] in valid_delivery_dates]

def stack(idxs):
    Xe = np.stack([samples[i][0] for i in idxs], axis=0)
    Xd = np.stack([samples[i][1] for i in idxs], axis=0)
    Y  = np.stack([samples[i][2] for i in idxs], axis=0)
    Y0 = np.stack([samples[i][3] for i in idxs], axis=0)
    return Xe, Xd, Y, Y0

Xe_tr, Xd_tr, Y_tr, Y0_tr = stack(train_idx)
Xe_va, Xd_va, Y_va, Y0_va = (None, None, None, None)
if len(val_idx) > 0:
    Xe_va, Xd_va, Y_va, Y0_va = stack(val_idx)

# ---------- diagnose + drop all-NaN decoder features, then impute (robust) ----------
# Flatten decoder train to 2D to inspect per-feature missingness
F_orig = Xd_tr.shape[2]
Xd_tr_2d_raw = Xd_tr.reshape(-1, F_orig)
nan_counts = np.isnan(Xd_tr_2d_raw).sum(axis=0)
all_nan_cols = nan_counts == Xd_tr_2d_raw.shape[0]

if all_nan_cols.any():
    dropped_cols = [dec_future_cols[i] for i,flag in enumerate(all_nan_cols) if flag]
    print(f"Dropping {int(all_nan_cols.sum())} decoder feature(s) with all-NaN in TRAIN: {dropped_cols}")
else:
    dropped_cols = []

# Keep mask to be reused at prediction time
dec_keep_mask = ~all_nan_cols

# Apply mask to decoder tensors
Xd_tr = Xd_tr[:, :, dec_keep_mask]
if Xe_va is not None:
    Xd_va = Xd_va[:, :, dec_keep_mask]

# Recompute medians on KEPT features
dec_med = np.nanmedian(Xd_tr.reshape(-1, Xd_tr.shape[2]), axis=0)
# Guard: if a median is still NaN for some reason, set to 0
dec_med = np.where(np.isnan(dec_med), 0.0, dec_med)

# Encoder median (rare but safe)
enc_med = np.nanmedian(Xe_tr.reshape(-1, Xe_tr.shape[2]), axis=0)
enc_med = np.where(np.isnan(enc_med), 0.0, enc_med)

# Impute in-place (3D-safe)
impute_inplace_3d(Xe_tr, enc_med)
impute_inplace_3d(Xd_tr, dec_med)
if Xe_va is not None:
    impute_inplace_3d(Xe_va, enc_med)
    impute_inplace_3d(Xd_va, dec_med)

# Targets should be complete for training/val; assert
assert np.isfinite(Y_tr).all(), "Y_tr has non-finite values"
if Y_va is not None:
    assert np.isfinite(Y_va).all(), "Y_va has non-finite values"

# ---------- safe standardization ----------
enc_mu, enc_sd = fit_standardizer(Xe_tr.reshape(-1, Xe_tr.shape[2]))
dec_mu, dec_sd = fit_standardizer(Xd_tr.reshape(-1, Xd_tr.shape[2]))
y_mu,  y_sd    = fit_standardizer(Y_tr.reshape(-1,1))

Xe_tr_n = apply_standardizer(Xe_tr, enc_mu, enc_sd)
Xd_tr_n = apply_standardizer(Xd_tr, dec_mu, dec_sd)
Y_tr_n  = apply_standardizer(Y_tr,  y_mu,  y_sd).reshape(Y_tr.shape)
Y0_tr_n = apply_standardizer(Y0_tr.reshape(-1,1), y_mu, y_sd).reshape(-1)

if Xe_va is not None:
    Xe_va_n = apply_standardizer(Xe_va, enc_mu, enc_sd)
    Xd_va_n = apply_standardizer(Xd_va, dec_mu, dec_sd)
    Y_va_n  = apply_standardizer(Y_va,  y_mu,  y_sd).reshape(Y_va.shape)
    Y0_va_n = apply_standardizer(Y0_va.reshape(-1,1), y_mu, y_sd).reshape(-1)

# Final safety: ensure everything is finite
assert_finite("Xe_tr_n", Xe_tr_n)
assert_finite("Xd_tr_n", Xd_tr_n)
assert_finite("Y_tr_n",  Y_tr_n)
assert_finite("Y0_tr_n", Y0_tr_n)
if Xe_va is not None:
    assert_finite("Xe_va_n", Xe_va_n)
    assert_finite("Xd_va_n", Xd_va_n)
    assert_finite("Y_va_n",  Y_va_n)
    assert_finite("Y0_va_n", Y0_va_n)

# ---------- torch datasets ----------
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self, Xe, Xd, Y, Y0):
        self.Xe = torch.from_numpy(Xe).float()
        self.Xd = torch.from_numpy(Xd).float()
        self.Y  = torch.from_numpy(Y).float()
        self.Y0 = torch.from_numpy(Y0).float()
    def __len__(self): return self.Xe.shape[0]
    def __getitem__(self, i): return self.Xe[i], self.Xd[i], self.Y[i], self.Y0[i]

bs = 32
train_loader = torch.utils.data.DataLoader(Seq2SeqDataset(Xe_tr_n, Xd_tr_n, Y_tr_n, Y0_tr_n), batch_size=bs, shuffle=True)
val_loader = None if Xe_va is None else torch.utils.data.DataLoader(Seq2SeqDataset(Xe_va_n, Xd_va_n, Y_va_n, Y0_va_n), batch_size=bs, shuffle=False)

# ---------- model ----------
class Encoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim, hidden, batch_first=True)
    def forward(self, x):  # x: (B,W,E)
        _, h = self.rnn(x)
        return h  # (1,B,H)

class Decoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim + 1, hidden, batch_first=True)  # + prev y
        self.out = nn.Linear(hidden, 1)
    def forward(self, future_feats, h0, y0, teacher=None, tf_prob=0.5):
        B, T, D = future_feats.shape
        y_prev = y0.view(B,1)
        h = h0
        outs = []
        for t in range(T):
            x_t = torch.cat([future_feats[:,t,:], y_prev], dim=1).unsqueeze(1)  # (B,1,D+1)
            o, h = self.rnn(x_t, h)
            y_t = self.out(o[:, -1, :]).squeeze(1)
            outs.append(y_t)
            if (teacher is not None) and (rng.random() < tf_prob):
                y_prev = teacher[:,t].view(B,1)
            else:
                y_prev = y_t.view(B,1)
        return torch.stack(outs, dim=1)  # (B,24)

class Seq2Seq(nn.Module):
    def __init__(self, enc_in, dec_in, hidden):
        super().__init__()
        self.enc = Encoder(enc_in, hidden)
        self.dec = Decoder(dec_in, hidden)
    def forward(self, x_enc, x_dec, y0, y_teacher=None, tf_prob=0.5):
        h = self.enc(x_enc)
        return self.dec(x_dec, h, y0, y_teacher, tf_prob)

enc_in = Xe_tr_n.shape[2]
dec_in = Xd_tr_n.shape[2]
model = Seq2Seq(enc_in, dec_in, HIDDEN).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.HuberLoss()

def train_epoch():
    model.train()
    tot = 0.0
    for xenc, xdec, y, y0 in train_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        opt.zero_grad()
        yhat = model(xenc, xdec, y0, y_teacher=y, tf_prob=0.5)
        loss = loss_fn(yhat, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        tot += loss.item() * xenc.size(0)
    return tot / len(train_loader.dataset)

@torch.no_grad()
def eval_epoch():
    if val_loader is None: return None
    model.eval()
    tot = 0.0
    for xenc, xdec, y, y0 in val_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        yhat = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0)
        loss = loss_fn(yhat, y)
        tot += loss.item() * xenc.size(0)
    return tot / len(val_loader.dataset)

for ep in range(1, EPOCHS+1):
    tr = train_epoch()
    va = eval_epoch()
    print(f"Epoch {ep:02d}  train_loss={tr:.4f}" + ("" if va is None else f"  val_loss={va:.4f}"))

# ---- evaluate holdout MAE in real units (SAFE) ----
if val_loader is not None:
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xenc, xdec, y, y0 in val_loader:
            xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
            yhat_n = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0).detach().cpu().numpy()
            y_n    = y.detach().cpu().numpy()
            # invert scaling
            yhat = (yhat_n * y_sd + y_mu)
            yt   = (y_n    * y_sd + y_mu)
            preds.append(yhat.reshape(-1))
            trues.append(yt.reshape(-1))
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    mask = np.isfinite(preds) & np.isfinite(trues)
    dropped = int((~mask).sum())
    if dropped:
        print(f"Dropped {dropped} non-finite pairs from holdout scoring.")
    print({"holdout_MAE": float(mean_absolute_error(trues[mask], preds[mask]))})

# ---- predict requested DELIVERY_DATE ----
# find base row (previous day EOD with price)
prev_day = DELIVERY_DATE - dt.timedelta(days=1)
base_row = df_lags[(pd.to_datetime(df_lags["OperatingDTM"]).dt.date == prev_day) & (df_lags["Interval"]==24) & df_lags["hb_houston"].notna()]
if base_row.empty:
    raise RuntimeError(f"No base EOD price for {prev_day}")
ts0 = pd.to_datetime(base_row.iloc[0]["ts"])
y0  = np.float32(base_row.iloc[0]["hb_houston"])

# encoder window
win = df_lags.set_index("ts").loc[ts0 - pd.Timedelta(hours=W_PAST-1): ts0][enc_cols]
if win.shape[0] != W_PAST or win.isna().any().any():
    raise RuntimeError("Insufficient/NaN history for encoder window.")
Xe_pred = win.values.astype(np.float32)[None, ...]

# decoder future features (24 rows)
frows = df_future[(df_future["delivery_date"]==DELIVERY_DATE)].sort_values("delivery_interval")
if frows.shape[0] != H_FUT:
    raise RuntimeError("Spine missing 24 rows for delivery date.")
Xd_pred_full = frows[dec_future_cols].values.astype(np.float32)[None, ...]

# Apply the same keep mask used in training (drop columns that were all-NaN in train)
if 'dec_keep_mask' in globals():
    Xd_pred = Xd_pred_full[:, :, dec_keep_mask]
else:
    Xd_pred = Xd_pred_full

# impute + standardize pred using TRAIN stats
m = np.isnan(Xd_pred)
if m.any():
    Xd_pred[m] = np.broadcast_to(dec_med, Xd_pred.shape)[m]
Xe_pred_n = apply_standardizer(Xe_pred, enc_mu, enc_sd)
Xd_pred_n = apply_standardizer(Xd_pred, dec_mu, dec_sd)
y0_n = ((y0 - y_mu) / y_sd).reshape(()).astype(np.float32)

model.eval()
with torch.no_grad():
    yhat_n = model(torch.from_numpy(Xe_pred_n).to(DEVICE),
                   torch.from_numpy(Xd_pred_n).to(DEVICE),
                   torch.tensor([y0_n], dtype=torch.float32, device=DEVICE),
                   y_teacher=None, tf_prob=0.0).detach().cpu().numpy()[0]
yhat = (yhat_n * y_sd + y_mu).reshape(-1)
# replace any residual NaN with the day's median
if np.isnan(yhat).any():
    day_med = float(np.nanmedian(yhat))
    yhat = np.nan_to_num(yhat, nan=day_med)

forecast = pd.DataFrame({
    "OperatingDTM": [DELIVERY_DATE]*H_FUT,
    "Interval": list(range(1, H_FUT+1)),
    "hb_houston_pred": yhat.tolist()
})
forecast


Dropping 6 decoder feature(s) with all-NaN in TRAIN: ['temp_avg', 'temp_spread', 'temp_avg_ramp1', 'hum_avg', 'hum_spread', 'hum_avg_ramp1']
Epoch 01  train_loss=0.0845  val_loss=0.0419
Epoch 02  train_loss=0.0758  val_loss=0.0196
Epoch 03  train_loss=0.0704  val_loss=0.0355
Epoch 04  train_loss=0.0656  val_loss=0.0257
Epoch 05  train_loss=0.0615  val_loss=0.0164
Epoch 06  train_loss=0.0610  val_loss=0.2036
Epoch 07  train_loss=0.0598  val_loss=0.0212
Epoch 08  train_loss=0.0539  val_loss=0.1410
Epoch 09  train_loss=0.0515  val_loss=0.0525
Epoch 10  train_loss=0.0513  val_loss=0.0224
Epoch 11  train_loss=0.0505  val_loss=0.0124
Epoch 12  train_loss=0.0496  val_loss=0.0114
Epoch 13  train_loss=0.0513  val_loss=0.0164
Epoch 14  train_loss=0.0484  val_loss=0.0120
Epoch 15  train_loss=0.0462  val_loss=0.0585
Epoch 16  train_loss=0.0439  val_loss=0.0155
Epoch 17  train_loss=0.0452  val_loss=0.0752
Epoch 18  train_loss=0.0440  val_loss=0.0272
Epoch 19  train_loss=0.0417  val_loss=0.0271
Epoc

TypeError: len() of unsized object

In [13]:
# ==== PyTorch seq2seq with ROBUST preprocessing (median impute + safe standardize + safe MAE) ====
import duckdb, pandas as pd, numpy as np, datetime as dt, math
import torch, torch.nn as nn
from sklearn.metrics import mean_absolute_error

DB = "../ProjectMain/db/data.duckdb"
DELIVERY_DATE = pd.Timestamp("2025-08-18").date()   # change target date if needed
W_PAST = 168
H_FUT = 24
HOLDOUT_DAYS = 60
HIDDEN = 128
EPOCHS = 20
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
rng = np.random.default_rng(42)
torch.manual_seed(42)

# ---------- helpers ----------
def fit_standardizer(X2d):
    """Return (mean,std) with std guarded (==0 -> 1). X2d: (N, F)"""
    mu = np.nanmean(X2d, axis=0)
    sd = np.nanstd(X2d, axis=0)
    sd = np.where(sd <= 1e-8, 1.0, sd)
    return mu, sd

def apply_standardizer(X, mu, sd):
    return (X - mu) / sd

# robust 3D imputer
def impute_inplace_3d(X, med_vec):
    """X: (N, T, F) or (N, W, F); med_vec: (F,). Fills NaNs in-place with per-feature medians."""
    mask = np.isnan(X)
    if mask.any():
        med_broadcast = np.broadcast_to(med_vec, X.shape)
        X[mask] = med_broadcast[mask]

# sanity checker
def assert_finite(name, arr):
    if not np.isfinite(arr).all():
        bad = int(np.isnan(arr).sum() + np.isinf(arr).sum())
        raise RuntimeError(f"{name} has {bad} non-finite values")

# ---------- pull data ----------
con = duckdb.connect(DB)
df_lags = con.execute("""
SELECT
  ts, OperatingDTM, Interval, hb_houston,
  p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
  dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean
FROM vw_master_spine_lags
ORDER BY ts
""").df()
df_lags["ts"] = pd.to_datetime(df_lags["ts"])

df_future = con.execute("""
SELECT
  f.OperatingDTM AS delivery_date,
  f.Interval     AS delivery_interval,
  f.hb_houston   AS target,
  -- base future features
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.temp_the_woodlands_tx, f.hum_the_woodlands_tx, f.wind_the_woodlands_tx, f.precip_the_woodlands_tx,
  f.temp_katy_tx, f.hum_katy_tx, f.wind_katy_tx, f.precip_katy_tx,
  f.temp_friendswood_tx, f.hum_friendswood_tx, f.wind_friendswood_tx, f.precip_friendswood_tx,
  f.temp_baytown_tx, f.hum_baytown_tx, f.wind_baytown_tx, f.precip_baytown_tx,
  f.temp_houston_tx, f.hum_houston_tx, f.wind_houston_tx, f.precip_houston_tx,
  f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
  f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f,
  f.cal_sin_dow  AS cal_sin_dow_f,  f.cal_cos_dow  AS cal_cos_dow_f,
  -- enhanced joins
  l.net_wz, l.net_lz, l.net_wz_ramp1, l.net_lz_ramp1, l.lz_houston_ramp1, l.wz_spread, l.lz_spread,
  w.temp_avg, w.temp_spread, w.temp_avg_ramp1, w.hum_avg, w.hum_spread, w.hum_avg_ramp1
FROM vw_master_spine_ts f
LEFT JOIN load_fcst_enh l
  ON l.OperatingDTM = f.OperatingDTM AND l.Interval = f.Interval
LEFT JOIN wx_fcst_enh w
  ON w.OperatingDTM = f.OperatingDTM AND w.Interval = f.Interval
ORDER BY f.OperatingDTM, f.Interval
""").df()
con.close()

df_future["delivery_date"] = pd.to_datetime(df_future["delivery_date"]).dt.date
df_future["delivery_interval"] = df_future["delivery_interval"].astype(int)

# ---------- assemble samples (day-ahead) ----------
enc_cols = [
    "hb_houston","p_lag1","p_lag2","p_lag3","p_lag6","p_lag12","p_lag24","p_lag48","p_lag72","p_lag168",
    "dp1","dp24","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean"
]
dec_future_cols = [
    "cal_hour_f","cal_dow_f","cal_is_weekend_f","cal_sin_hour_f","cal_cos_hour_f","cal_sin_dow_f","cal_cos_dow_f",
    "net_wz","net_lz","net_wz_ramp1","net_lz_ramp1","lz_houston_ramp1","wz_spread","lz_spread",
    "temp_avg","temp_spread","temp_avg_ramp1","hum_avg","hum_spread","hum_avg_ramp1"
]

# base rows = end-of-day with price
base_rows = df_lags[(df_lags["hb_houston"].notna()) & (df_lags["Interval"]==24)][["ts","OperatingDTM","hb_houston"]].copy()
base_rows["base_date"] = pd.to_datetime(base_rows["OperatingDTM"]).dt.date
base_rows["delivery_date"] = base_rows["base_date"] + dt.timedelta(days=1)

# only use delivery days with 24 actuals for training/val
has24 = df_future.groupby("delivery_date")["target"].apply(lambda s: s.notna().sum()==24)
valid_delivery_dates = set(has24[has24].index)

df_lags_idx = df_lags.set_index("ts").sortindex() if hasattr(pd.DataFrame, 'sortindex') else df_lags.set_index("ts").sort_index()

samples = []
for _, r in base_rows.iterrows():
    ddate = r["delivery_date"]
    ts0 = r["ts"]
    # encoder window
    start = ts0 - pd.Timedelta(hours=W_PAST-1)
    enc_df = df_lags_idx.loc[start:ts0][enc_cols]
    if enc_df.shape[0] != W_PAST: 
        continue
    if enc_df.isna().any().any():
        continue  # strict: skip any encoder NaNs
    # decoder future rows
    fdf = df_future[(df_future["delivery_date"]==ddate)].sort_values("delivery_interval")
    if fdf.shape[0] != H_FUT:
        continue
    X_enc = enc_df.values.astype(np.float32)
    X_dec = fdf[dec_future_cols].values.astype(np.float32)
    y     = fdf["target"].values.astype(np.float32)
    y0    = np.float32(r["hb_houston"])
    samples.append((X_enc, X_dec, y, y0, ddate))

if len(samples) == 0:
    raise RuntimeError("No samples assembled; check data coverage or reduce W_PAST.")

# split by date
dates = sorted({s[4] for s in samples})
cut_date = dates[-1] - dt.timedelta(days=HOLDOUT_DAYS) if len(dates)>HOLDOUT_DAYS else dates[0]
train_idx = [i for i,s in enumerate(samples) if s[4] <= cut_date and s[4] in valid_delivery_dates]
val_idx   = [i for i,s in enumerate(samples) if s[4] >  cut_date and s[4] in valid_delivery_dates]

def stack(idxs):
    Xe = np.stack([samples[i][0] for i in idxs], axis=0)
    Xd = np.stack([samples[i][1] for i in idxs], axis=0)
    Y  = np.stack([samples[i][2] for i in idxs], axis=0)
    Y0 = np.stack([samples[i][3] for i in idxs], axis=0)
    return Xe, Xd, Y, Y0

Xe_tr, Xd_tr, Y_tr, Y0_tr = stack(train_idx)
Xe_va, Xd_va, Y_va, Y0_va = (None, None, None, None)
if len(val_idx) > 0:
    Xe_va, Xd_va, Y_va, Y0_va = stack(val_idx)

# ---------- diagnose + drop all-NaN decoder features, then impute (robust) ----------
# Flatten decoder train to 2D to inspect per-feature missingness
F_orig = Xd_tr.shape[2]
Xd_tr_2d_raw = Xd_tr.reshape(-1, F_orig)
nan_counts = np.isnan(Xd_tr_2d_raw).sum(axis=0)
all_nan_cols = nan_counts == Xd_tr_2d_raw.shape[0]

if all_nan_cols.any():
    dropped_cols = [dec_future_cols[i] for i,flag in enumerate(all_nan_cols) if flag]
    print(f"Dropping {int(all_nan_cols.sum())} decoder feature(s) with all-NaN in TRAIN: {dropped_cols}")
else:
    dropped_cols = []

# Keep mask to be reused at prediction time
dec_keep_mask = ~all_nan_cols

# Apply mask to decoder tensors
Xd_tr = Xd_tr[:, :, dec_keep_mask]
if Xe_va is not None:
    Xd_va = Xd_va[:, :, dec_keep_mask]

# Recompute medians on KEPT features
dec_med = np.nanmedian(Xd_tr.reshape(-1, Xd_tr.shape[2]), axis=0)
# Guard: if a median is still NaN for some reason, set to 0
dec_med = np.where(np.isnan(dec_med), 0.0, dec_med)

# Encoder median (rare but safe)
enc_med = np.nanmedian(Xe_tr.reshape(-1, Xe_tr.shape[2]), axis=0)
enc_med = np.where(np.isnan(enc_med), 0.0, enc_med)

# Impute in-place (3D-safe)
impute_inplace_3d(Xe_tr, enc_med)
impute_inplace_3d(Xd_tr, dec_med)
if Xe_va is not None:
    impute_inplace_3d(Xe_va, enc_med)
    impute_inplace_3d(Xd_va, dec_med)

# Targets should be complete for training/val; assert
assert np.isfinite(Y_tr).all(), "Y_tr has non-finite values"
if Y_va is not None:
    assert np.isfinite(Y_va).all(), "Y_va has non-finite values"

# ---------- safe standardization ----------
enc_mu, enc_sd = fit_standardizer(Xe_tr.reshape(-1, Xe_tr.shape[2]))
dec_mu, dec_sd = fit_standardizer(Xd_tr.reshape(-1, Xd_tr.shape[2]))
y_mu,  y_sd    = fit_standardizer(Y_tr.reshape(-1,1))
# Cast to Python floats to avoid 0-d array issues in scalar ops
y_mu = float(np.atleast_1d(y_mu)[0]); y_sd = float(np.atleast_1d(y_sd)[0])

Xe_tr_n = apply_standardizer(Xe_tr, enc_mu, enc_sd)
Xd_tr_n = apply_standardizer(Xd_tr, dec_mu, dec_sd)
Y_tr_n  = apply_standardizer(Y_tr,  y_mu,  y_sd).reshape(Y_tr.shape)
Y0_tr_n = apply_standardizer(Y0_tr.reshape(-1,1), y_mu, y_sd).reshape(-1)

if Xe_va is not None:
    Xe_va_n = apply_standardizer(Xe_va, enc_mu, enc_sd)
    Xd_va_n = apply_standardizer(Xd_va, dec_mu, dec_sd)
    Y_va_n  = apply_standardizer(Y_va,  y_mu,  y_sd).reshape(Y_va.shape)
    Y0_va_n = apply_standardizer(Y0_va.reshape(-1,1), y_mu, y_sd).reshape(-1)

# Final safety: ensure everything is finite
assert_finite("Xe_tr_n", Xe_tr_n)
assert_finite("Xd_tr_n", Xd_tr_n)
assert_finite("Y_tr_n",  Y_tr_n)
assert_finite("Y0_tr_n", Y0_tr_n)
if Xe_va is not None:
    assert_finite("Xe_va_n", Xe_va_n)
    assert_finite("Xd_va_n", Xd_va_n)
    assert_finite("Y_va_n",  Y_va_n)
    assert_finite("Y0_va_n", Y0_va_n)

# ---------- torch datasets ----------
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self, Xe, Xd, Y, Y0):
        self.Xe = torch.from_numpy(Xe).float()
        self.Xd = torch.from_numpy(Xd).float()
        self.Y  = torch.from_numpy(Y).float()
        self.Y0 = torch.from_numpy(Y0).float()
    def __len__(self): return self.Xe.shape[0]
    def __getitem__(self, i): return self.Xe[i], self.Xd[i], self.Y[i], self.Y0[i]

bs = 32
train_loader = torch.utils.data.DataLoader(Seq2SeqDataset(Xe_tr_n, Xd_tr_n, Y_tr_n, Y0_tr_n), batch_size=bs, shuffle=True)
val_loader = None if Xe_va is None else torch.utils.data.DataLoader(Seq2SeqDataset(Xe_va_n, Xd_va_n, Y_va_n, Y0_va_n), batch_size=bs, shuffle=False)

# ---------- model ----------
class Encoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim, hidden, batch_first=True)
    def forward(self, x):  # x: (B,W,E)
        _, h = self.rnn(x)
        return h  # (1,B,H)

class Decoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim + 1, hidden, batch_first=True)  # + prev y
        self.out = nn.Linear(hidden, 1)
    def forward(self, future_feats, h0, y0, teacher=None, tf_prob=0.5):
        B, T, D = future_feats.shape
        y_prev = y0.view(B,1)
        h = h0
        outs = []
        for t in range(T):
            x_t = torch.cat([future_feats[:,t,:], y_prev], dim=1).unsqueeze(1)  # (B,1,D+1)
            o, h = self.rnn(x_t, h)
            y_t = self.out(o[:, -1, :]).squeeze(1)
            outs.append(y_t)
            if (teacher is not None) and (rng.random() < tf_prob):
                y_prev = teacher[:,t].view(B,1)
            else:
                y_prev = y_t.view(B,1)
        return torch.stack(outs, dim=1)  # (B,24)

class Seq2Seq(nn.Module):
    def __init__(self, enc_in, dec_in, hidden):
        super().__init__()
        self.enc = Encoder(enc_in, hidden)
        self.dec = Decoder(dec_in, hidden)
    def forward(self, x_enc, x_dec, y0, y_teacher=None, tf_prob=0.5):
        h = self.enc(x_enc)
        return self.dec(x_dec, h, y0, y_teacher, tf_prob)

enc_in = Xe_tr_n.shape[2]
dec_in = Xd_tr_n.shape[2]
model = Seq2Seq(enc_in, dec_in, HIDDEN).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.HuberLoss()

def train_epoch():
    model.train()
    tot = 0.0
    for xenc, xdec, y, y0 in train_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        opt.zero_grad()
        yhat = model(xenc, xdec, y0, y_teacher=y, tf_prob=0.5)
        loss = loss_fn(yhat, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        tot += loss.item() * xenc.size(0)
    return tot / len(train_loader.dataset)

@torch.no_grad()
def eval_epoch():
    if val_loader is None: return None
    model.eval()
    tot = 0.0
    for xenc, xdec, y, y0 in val_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        yhat = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0)
        loss = loss_fn(yhat, y)
        tot += loss.item() * xenc.size(0)
    return tot / len(val_loader.dataset)

for ep in range(1, EPOCHS+1):
    tr = train_epoch()
    va = eval_epoch()
    print(f"Epoch {ep:02d}  train_loss={tr:.4f}" + ("" if va is None else f"  val_loss={va:.4f}"))

# ---- evaluate holdout MAE in real units (SAFE) ----
if val_loader is not None:
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xenc, xdec, y, y0 in val_loader:
            xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
            yhat_n = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0).detach().cpu().numpy()
            y_n    = y.detach().cpu().numpy()
            # invert scaling
            yhat = (yhat_n * y_sd + y_mu)
            yt   = (y_n    * y_sd + y_mu)
            preds.append(yhat.reshape(-1))
            trues.append(yt.reshape(-1))
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    mask = np.isfinite(preds) & np.isfinite(trues)
    dropped = int((~mask).sum())
    if dropped:
        print(f"Dropped {dropped} non-finite pairs from holdout scoring.")
    print({"holdout_MAE": float(mean_absolute_error(trues[mask], preds[mask]))})

# ---- predict requested DELIVERY_DATE ----
# find base row (previous day EOD with price)
prev_day = DELIVERY_DATE - dt.timedelta(days=1)
base_row = df_lags[(pd.to_datetime(df_lags["OperatingDTM"]).dt.date == prev_day) & (df_lags["Interval"]==24) & df_lags["hb_houston"].notna()]
if base_row.empty:
    raise RuntimeError(f"No base EOD price for {prev_day}")
ts0 = pd.to_datetime(base_row.iloc[0]["ts"])
y0  = np.float32(base_row.iloc[0]["hb_houston"])

# encoder window
win = df_lags.set_index("ts").loc[ts0 - pd.Timedelta(hours=W_PAST-1): ts0][enc_cols]
if win.shape[0] != W_PAST or win.isna().any().any():
    raise RuntimeError("Insufficient/NaN history for encoder window.")
Xe_pred = win.values.astype(np.float32)[None, ...]

# decoder future features (24 rows)
frows = df_future[(df_future["delivery_date"]==DELIVERY_DATE)].sort_values("delivery_interval")
if frows.shape[0] != H_FUT:
    raise RuntimeError("Spine missing 24 rows for delivery date.")
Xd_pred_full = frows[dec_future_cols].values.astype(np.float32)[None, ...]

# Apply the same keep mask used in training (drop columns that were all-NaN in train)
if 'dec_keep_mask' in globals():
    Xd_pred = Xd_pred_full[:, :, dec_keep_mask]
else:
    Xd_pred = Xd_pred_full

# impute + standardize pred using TRAIN stats
m = np.isnan(Xd_pred)
if m.any():
    Xd_pred[m] = np.broadcast_to(dec_med, Xd_pred.shape)[m]
Xe_pred_n = apply_standardizer(Xe_pred, enc_mu, enc_sd)
Xd_pred_n = apply_standardizer(Xd_pred, dec_mu, dec_sd)
# compute normalized scalar y0_n robustly
y0_n = np.float32(((y0 - y_mu) / y_sd)).ravel()[0]

model.eval()
with torch.no_grad():
    yhat_n = model(torch.from_numpy(Xe_pred_n).to(DEVICE),
                   torch.from_numpy(Xd_pred_n).to(DEVICE),
                   torch.tensor([y0_n], dtype=torch.float32, device=DEVICE),
                   y_teacher=None, tf_prob=0.0).detach().cpu().numpy()[0]
yhat = (yhat_n * y_sd + y_mu).reshape(-1)
# replace any residual NaN with the day's median
if np.isnan(yhat).any():
    day_med = float(np.nanmedian(yhat))
    yhat = np.nan_to_num(yhat, nan=day_med)

forecast = pd.DataFrame({
    "OperatingDTM": [DELIVERY_DATE]*H_FUT,
    "Interval": list(range(1, H_FUT+1)),
    "hb_houston_pred": yhat.tolist()
})
forecast


Dropping 6 decoder feature(s) with all-NaN in TRAIN: ['temp_avg', 'temp_spread', 'temp_avg_ramp1', 'hum_avg', 'hum_spread', 'hum_avg_ramp1']
Epoch 01  train_loss=0.0845  val_loss=0.0419
Epoch 02  train_loss=0.0758  val_loss=0.0196
Epoch 03  train_loss=0.0704  val_loss=0.0355
Epoch 04  train_loss=0.0656  val_loss=0.0257
Epoch 05  train_loss=0.0615  val_loss=0.0164
Epoch 06  train_loss=0.0610  val_loss=0.2036
Epoch 07  train_loss=0.0598  val_loss=0.0212
Epoch 08  train_loss=0.0539  val_loss=0.1410
Epoch 09  train_loss=0.0515  val_loss=0.0525
Epoch 10  train_loss=0.0513  val_loss=0.0224
Epoch 11  train_loss=0.0505  val_loss=0.0124
Epoch 12  train_loss=0.0496  val_loss=0.0114
Epoch 13  train_loss=0.0513  val_loss=0.0164
Epoch 14  train_loss=0.0484  val_loss=0.0120
Epoch 15  train_loss=0.0462  val_loss=0.0585
Epoch 16  train_loss=0.0439  val_loss=0.0155
Epoch 17  train_loss=0.0452  val_loss=0.0752
Epoch 18  train_loss=0.0440  val_loss=0.0272
Epoch 19  train_loss=0.0417  val_loss=0.0271
Epoc

,OperatingDTM,Interval,hb_houston_pred
0,2025-08-18,1,35.257851
1,2025-08-18,2,36.735786
2,2025-08-18,3,34.342613
3,2025-08-18,4,31.196121
4,2025-08-18,5,29.188484
5,2025-08-18,6,28.787140
6,2025-08-18,7,28.717094
7,2025-08-18,8,26.184206
8,2025-08-18,9,28.311657
9,2025-08-18,10,27.253433


Weather backfill (no more all-NaN drops):
create wx_hist_enh (historical weather enhanced) and modified the data pull to COALESCE forecast → historical (wx_fcst_enh → wx_hist_enh). We also add is_weather_proxy to see if using a proxy.

Spike-aware target transform:
Trained on a signed log of price (sgn(y)*log1p(|y|)) and invert with sgn(z)*expm1(|z|). This will hopefully compresses peaks during learning, helping the model fit surge hours without being dominated by MSE-like behavior. The scaler (μ,σ) is now fit on the transformed target.

Peak-hour weighting:
Added hour-of-day weights (2× for intervals 17–21) inside a weighted Huber loss. This gently prioritizes evening ramp/peak errors.

Keep all earlier robustness:
3-D median impute, zero-variance guards, strict finiteness checks, safe holdout MAE (now computed after inverse-transform).

In [16]:
# ==== PyTorch seq2seq with ROBUST preprocessing (median impute + safe standardize + safe MAE) ====
import duckdb, pandas as pd, numpy as np, datetime as dt, math
import torch, torch.nn as nn
from sklearn.metrics import mean_absolute_error

DB = "../ProjectMain/db/data.duckdb"
DELIVERY_DATE = pd.Timestamp("2025-08-18").date()   # change target date if needed
W_PAST = 168
H_FUT = 24
HOLDOUT_DAYS = 60
HIDDEN = 128
EPOCHS = 20
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
rng = np.random.default_rng(42)
torch.manual_seed(42)

# ---------- helpers ----------
def fit_standardizer(X2d):
    """Return (mean,std) with std guarded (==0 -> 1). X2d: (N, F)"""
    mu = np.nanmean(X2d, axis=0)
    sd = np.nanstd(X2d, axis=0)
    sd = np.where(sd <= 1e-8, 1.0, sd)
    return mu, sd

def apply_standardizer(X, mu, sd):
    return (X - mu) / sd

# robust 3D imputer
def impute_inplace_3d(X, med_vec):
    """X: (N, T, F) or (N, W, F); med_vec: (F,). Fills NaNs in-place with per-feature medians."""
    mask = np.isnan(X)
    if mask.any():
        med_broadcast = np.broadcast_to(med_vec, X.shape)
        X[mask] = med_broadcast[mask]

# sanity checker
def assert_finite(name, arr):
    if not np.isfinite(arr).all():
        bad = int(np.isnan(arr).sum() + np.isinf(arr).sum())
        raise RuntimeError(f"{name} has {bad} non-finite values")

# ---------- ensure weather-enhanced views (hist + fcst) exist & helper transforms ----------
con_ensure = duckdb.connect(DB)
# Historical weather enhanced (to backfill when forecast is missing during training)
con_ensure.execute("""
CREATE OR REPLACE VIEW wx_hist_enh AS
WITH w AS (
  SELECT
    OperatingDTM, "interval" AS Interval,
    temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx, temp_baytown_tx, temp_houston_tx,
    hum_the_woodlands_tx,  hum_katy_tx,  hum_friendswood_tx,  hum_baytown_tx,  hum_houston_tx
  FROM vw_historical_weather_by_city
),
agg AS (
  SELECT
    OperatingDTM, Interval,
    (temp_the_woodlands_tx + temp_katy_tx + temp_friendswood_tx + temp_baytown_tx + temp_houston_tx)/5.0 AS temp_avg,
    (hum_the_woodlands_tx  + hum_katy_tx  + hum_friendswood_tx  + hum_baytown_tx  + hum_houston_tx)/5.0  AS hum_avg,
    GREATEST(temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx, temp_baytown_tx, temp_houston_tx)
      - LEAST(temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx, temp_baytown_tx, temp_houston_tx) AS temp_spread,
    GREATEST(hum_the_woodlands_tx, hum_katy_tx, hum_friendswood_tx, hum_baytown_tx, hum_houston_tx)
      - LEAST(hum_the_woodlands_tx, hum_katy_tx, hum_friendswood_tx, hum_baytown_tx, hum_houston_tx) AS hum_spread
  FROM w
)
SELECT
  a.*,
  a.temp_avg - LAG(a.temp_avg) OVER (ORDER BY OperatingDTM, Interval) AS temp_avg_ramp1,
  a.hum_avg  - LAG(a.hum_avg)  OVER (ORDER BY OperatingDTM, Interval) AS hum_avg_ramp1
FROM agg a;
""")
con_ensure.close()

# Signed log transforms for spikes
sgn = np.sign
abs_ = np.abs

def sgn_log1p(y):
    return sgn(y) * np.log1p(abs_(y))

def sgn_expm1(z):
    return sgn(z) * np.expm1(abs_(z))

# Hour-of-day weights (emphasize 17-21 local hours)
HOUR_WEIGHTS = np.ones(24, dtype=np.float32)
HOUR_WEIGHTS[16:21] = 2.0  # intervals 17..21

# ---------- pull data ----------
con = duckdb.connect(DB)
df_lags = con.execute("""
SELECT
  ts, OperatingDTM, Interval, hb_houston,
  p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
  dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean
FROM vw_master_spine_lags
ORDER BY ts
""").df()
df_lags["ts"] = pd.to_datetime(df_lags["ts"])

df_future = con.execute("""
SELECT
  f.OperatingDTM AS delivery_date,
  f.Interval     AS delivery_interval,
  f.hb_houston   AS target,
  -- base future features
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
  f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f,
  f.cal_sin_dow  AS cal_sin_dow_f,  f.cal_cos_dow  AS cal_cos_dow_f,
  -- load-enhanced joins
  l.net_wz, l.net_lz, l.net_wz_ramp1, l.net_lz_ramp1, l.lz_houston_ramp1, l.wz_spread, l.lz_spread,
  -- weather: COALESCE forecast -> historical
  COALESCE(wf.temp_avg,      wh.temp_avg)      AS temp_avg,
  COALESCE(wf.temp_spread,   wh.temp_spread)   AS temp_spread,
  COALESCE(wf.temp_avg_ramp1,wh.temp_avg_ramp1)AS temp_avg_ramp1,
  COALESCE(wf.hum_avg,       wh.hum_avg)       AS hum_avg,
  COALESCE(wf.hum_spread,    wh.hum_spread)    AS hum_spread,
  COALESCE(wf.hum_avg_ramp1, wh.hum_avg_ramp1) AS hum_avg_ramp1,
  CASE WHEN wf.temp_avg IS NULL THEN 1 ELSE 0 END AS is_weather_proxy
FROM vw_master_spine_ts f
LEFT JOIN load_fcst_enh l
  ON l.OperatingDTM = f.OperatingDTM AND l.Interval = f.Interval
LEFT JOIN wx_fcst_enh  wf
  ON wf.OperatingDTM = f.OperatingDTM AND wf.Interval = f.Interval
LEFT JOIN wx_hist_enh  wh
  ON wh.OperatingDTM = f.OperatingDTM AND wh.Interval = f.Interval
ORDER BY f.OperatingDTM, f.Interval
""").df()
con.close()

df_future["delivery_date"] = pd.to_datetime(df_future["delivery_date"]).dt.date
df_future["delivery_interval"] = df_future["delivery_interval"].astype(int)

# ---------- assemble samples (day-ahead) ----------
enc_cols = [
    "hb_houston","p_lag1","p_lag2","p_lag3","p_lag6","p_lag12","p_lag24","p_lag48","p_lag72","p_lag168",
    "dp1","dp24","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean"
]
dec_future_cols = [
    "cal_hour_f","cal_dow_f","cal_is_weekend_f","cal_sin_hour_f","cal_cos_hour_f","cal_sin_dow_f","cal_cos_dow_f",
    "net_wz","net_lz","net_wz_ramp1","net_lz_ramp1","lz_houston_ramp1","wz_spread","lz_spread",
    "temp_avg","temp_spread","temp_avg_ramp1","hum_avg","hum_spread","hum_avg_ramp1"
]

# base rows = end-of-day with price
base_rows = df_lags[(df_lags["hb_houston"].notna()) & (df_lags["Interval"]==24)][["ts","OperatingDTM","hb_houston"]].copy()
base_rows["base_date"] = pd.to_datetime(base_rows["OperatingDTM"]).dt.date
base_rows["delivery_date"] = base_rows["base_date"] + dt.timedelta(days=1)

# only use delivery days with 24 actuals for training/val
has24 = df_future.groupby("delivery_date")["target"].apply(lambda s: s.notna().sum()==24)
valid_delivery_dates = set(has24[has24].index)

df_lags_idx = df_lags.set_index("ts").sortindex() if hasattr(pd.DataFrame, 'sortindex') else df_lags.set_index("ts").sort_index()

samples = []
for _, r in base_rows.iterrows():
    ddate = r["delivery_date"]
    ts0 = r["ts"]
    # encoder window
    start = ts0 - pd.Timedelta(hours=W_PAST-1)
    enc_df = df_lags_idx.loc[start:ts0][enc_cols]
    if enc_df.shape[0] != W_PAST: 
        continue
    if enc_df.isna().any().any():
        continue  # strict: skip any encoder NaNs
    # decoder future rows
    fdf = df_future[(df_future["delivery_date"]==ddate)].sort_values("delivery_interval")
    if fdf.shape[0] != H_FUT:
        continue
    X_enc = enc_df.values.astype(np.float32)
    X_dec = fdf[dec_future_cols].values.astype(np.float32)
    y     = fdf["target"].values.astype(np.float32)
    y0    = np.float32(r["hb_houston"])
    samples.append((X_enc, X_dec, y, y0, ddate))

if len(samples) == 0:
    raise RuntimeError("No samples assembled; check data coverage or reduce W_PAST.")

# split by date
dates = sorted({s[4] for s in samples})
cut_date = dates[-1] - dt.timedelta(days=HOLDOUT_DAYS) if len(dates)>HOLDOUT_DAYS else dates[0]
train_idx = [i for i,s in enumerate(samples) if s[4] <= cut_date and s[4] in valid_delivery_dates]
val_idx   = [i for i,s in enumerate(samples) if s[4] >  cut_date and s[4] in valid_delivery_dates]

def stack(idxs):
    Xe = np.stack([samples[i][0] for i in idxs], axis=0)
    Xd = np.stack([samples[i][1] for i in idxs], axis=0)
    Y  = np.stack([samples[i][2] for i in idxs], axis=0)
    Y0 = np.stack([samples[i][3] for i in idxs], axis=0)
    return Xe, Xd, Y, Y0

Xe_tr, Xd_tr, Y_tr, Y0_tr = stack(train_idx)
Xe_va, Xd_va, Y_va, Y0_va = (None, None, None, None)
if len(val_idx) > 0:
    Xe_va, Xd_va, Y_va, Y0_va = stack(val_idx)

# ---------- diagnose + drop all-NaN decoder features, then impute (robust) ----------
# Flatten decoder train to 2D to inspect per-feature missingness
F_orig = Xd_tr.shape[2]
Xd_tr_2d_raw = Xd_tr.reshape(-1, F_orig)
nan_counts = np.isnan(Xd_tr_2d_raw).sum(axis=0)
all_nan_cols = nan_counts == Xd_tr_2d_raw.shape[0]

if all_nan_cols.any():
    dropped_cols = [dec_future_cols[i] for i,flag in enumerate(all_nan_cols) if flag]
    print(f"Dropping {int(all_nan_cols.sum())} decoder feature(s) with all-NaN in TRAIN: {dropped_cols}")
else:
    dropped_cols = []

# Keep mask to be reused at prediction time
dec_keep_mask = ~all_nan_cols

# Apply mask to decoder tensors
Xd_tr = Xd_tr[:, :, dec_keep_mask]
if Xe_va is not None:
    Xd_va = Xd_va[:, :, dec_keep_mask]

# Recompute medians on KEPT features
dec_med = np.nanmedian(Xd_tr.reshape(-1, Xd_tr.shape[2]), axis=0)
# Guard: if a median is still NaN for some reason, set to 0
dec_med = np.where(np.isnan(dec_med), 0.0, dec_med)

# Encoder median (rare but safe)
enc_med = np.nanmedian(Xe_tr.reshape(-1, Xe_tr.shape[2]), axis=0)
enc_med = np.where(np.isnan(enc_med), 0.0, enc_med)

# Impute in-place (3D-safe)
impute_inplace_3d(Xe_tr, enc_med)
impute_inplace_3d(Xd_tr, dec_med)
if Xe_va is not None:
    impute_inplace_3d(Xe_va, enc_med)
    impute_inplace_3d(Xd_va, dec_med)

# Targets should be complete for training/val; assert
assert np.isfinite(Y_tr).all(), "Y_tr has non-finite values"
if Y_va is not None:
    assert np.isfinite(Y_va).all(), "Y_va has non-finite values"

# ---------- safe standardization ----------
enc_mu, enc_sd = fit_standardizer(Xe_tr.reshape(-1, Xe_tr.shape[2]))
dec_mu, dec_sd = fit_standardizer(Xd_tr.reshape(-1, Xd_tr.shape[2]))
# ---- target transform (spike-aware) then standardize ----
Y_tr_t  = sgn_log1p(Y_tr)
y_mu, y_sd = fit_standardizer(Y_tr_t.reshape(-1,1))
y_mu = float(np.atleast_1d(y_mu)[0]); y_sd = float(np.atleast_1d(y_sd)[0])

Xe_tr_n = apply_standardizer(Xe_tr, enc_mu, enc_sd)
Xd_tr_n = apply_standardizer(Xd_tr, dec_mu, dec_sd)
Y_tr_n  = apply_standardizer(Y_tr_t, y_mu, y_sd).reshape(Y_tr.shape)
Y0_tr_t = sgn_log1p(Y0_tr.reshape(-1,1)).reshape(-1,1)
Y0_tr_n = apply_standardizer(Y0_tr_t, y_mu, y_sd).reshape(-1)

if Xe_va is not None:
    Xe_va_n = apply_standardizer(Xe_va, enc_mu, enc_sd)
    Xd_va_n = apply_standardizer(Xd_va, dec_mu, dec_sd)
    Y_va_t  = sgn_log1p(Y_va)
    Y_va_n  = apply_standardizer(Y_va_t,  y_mu,  y_sd).reshape(Y_va.shape)
    Y0_va_t = sgn_log1p(Y0_va.reshape(-1,1))
    Y0_va_n = apply_standardizer(Y0_va_t, y_mu, y_sd).reshape(-1)

if Xe_va is not None:
    Xe_va_n = apply_standardizer(Xe_va, enc_mu, enc_sd)
    Xd_va_n = apply_standardizer(Xd_va, dec_mu, dec_sd)
    Y_va_n  = apply_standardizer(Y_va,  y_mu,  y_sd).reshape(Y_va.shape)
    Y0_va_n = apply_standardizer(Y0_va.reshape(-1,1), y_mu, y_sd).reshape(-1)

# Final safety: ensure everything is finite
assert_finite("Xe_tr_n", Xe_tr_n)
assert_finite("Xd_tr_n", Xd_tr_n)
assert_finite("Y_tr_n",  Y_tr_n)
assert_finite("Y0_tr_n", Y0_tr_n)
if Xe_va is not None:
    assert_finite("Xe_va_n", Xe_va_n)
    assert_finite("Xd_va_n", Xd_va_n)
    assert_finite("Y_va_n",  Y_va_n)
    assert_finite("Y0_va_n", Y0_va_n)

# ---------- torch datasets ----------
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self, Xe, Xd, Y, Y0):
        self.Xe = torch.from_numpy(Xe).float()
        self.Xd = torch.from_numpy(Xd).float()
        self.Y  = torch.from_numpy(Y).float()
        self.Y0 = torch.from_numpy(Y0).float()
    def __len__(self): return self.Xe.shape[0]
    def __getitem__(self, i): return self.Xe[i], self.Xd[i], self.Y[i], self.Y0[i]

bs = 32
train_loader = torch.utils.data.DataLoader(Seq2SeqDataset(Xe_tr_n, Xd_tr_n, Y_tr_n, Y0_tr_n), batch_size=bs, shuffle=True)
val_loader = None if Xe_va is None else torch.utils.data.DataLoader(Seq2SeqDataset(Xe_va_n, Xd_va_n, Y_va_n, Y0_va_n), batch_size=bs, shuffle=False)

# ---------- model ----------
class Encoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim, hidden, batch_first=True)
    def forward(self, x):  # x: (B,W,E)
        _, h = self.rnn(x)
        return h  # (1,B,H)

class Decoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim + 1, hidden, batch_first=True)  # + prev y
        self.out = nn.Linear(hidden, 1)
    def forward(self, future_feats, h0, y0, teacher=None, tf_prob=0.5):
        B, T, D = future_feats.shape
        y_prev = y0.view(B,1)
        h = h0
        outs = []
        for t in range(T):
            x_t = torch.cat([future_feats[:,t,:], y_prev], dim=1).unsqueeze(1)  # (B,1,D+1)
            o, h = self.rnn(x_t, h)
            y_t = self.out(o[:, -1, :]).squeeze(1)
            outs.append(y_t)
            if (teacher is not None) and (rng.random() < tf_prob):
                y_prev = teacher[:,t].view(B,1)
            else:
                y_prev = y_t.view(B,1)
        return torch.stack(outs, dim=1)  # (B,24)

class Seq2Seq(nn.Module):
    def __init__(self, enc_in, dec_in, hidden):
        super().__init__()
        self.enc = Encoder(enc_in, hidden)
        self.dec = Decoder(dec_in, hidden)
    def forward(self, x_enc, x_dec, y0, y_teacher=None, tf_prob=0.5):
        h = self.enc(x_enc)
        return self.dec(x_dec, h, y0, y_teacher, tf_prob)

enc_in = Xe_tr_n.shape[2]
dec_in = Xd_tr_n.shape[2]
model = Seq2Seq(enc_in, dec_in, HIDDEN).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.HuberLoss()

def huber_weighted(yhat, y, w, delta=1.0):
    d = torch.abs(yhat - y)
    q = torch.clamp(d, max=delta)
    l = 0.5 * q**2 + delta * (d - q)
    return (l * w).mean()

def train_epoch():
    model.train()
    tot = 0.0
    for xenc, xdec, y, y0 in train_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        opt.zero_grad()
        yhat = model(xenc, xdec, y0, y_teacher=y, tf_prob=0.5)
        # time-step weights (B,24)
        w = torch.tensor(HOUR_WEIGHTS, device=y.device).unsqueeze(0).expand_as(y)
        loss = huber_weighted(yhat, y, w)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        tot += loss.item() * xenc.size(0)
    return tot / len(train_loader.dataset)

@torch.no_grad()
@torch.no_grad()
def eval_epoch():
    if val_loader is None: return None
    model.eval()
    tot = 0.0
    for xenc, xdec, y, y0 in val_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        yhat = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0)
        w = torch.tensor(HOUR_WEIGHTS, device=y.device).unsqueeze(0).expand_as(y)
        loss = huber_weighted(yhat, y, w)
        tot += loss.item() * xenc.size(0)
    return tot / len(val_loader.dataset)

for ep in range(1, EPOCHS+1):
    tr = train_epoch()
    va = eval_epoch()
    print(f"Epoch {ep:02d}  train_loss={tr:.4f}" + ("" if va is None else f"  val_loss={va:.4f}"))

# ---- evaluate holdout MAE in real units (SAFE) ----
if val_loader is not None:
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xenc, xdec, y, y0 in val_loader:
            xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
            yhat_n = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0).detach().cpu().numpy()
            y_n    = y.detach().cpu().numpy()
            # invert scaling
            yhat_t = (yhat_n * y_sd + y_mu)
yt_t   = (y_n    * y_sd + y_mu)
yhat = sgn_expm1(yhat_t).reshape(-1)
yt   = sgn_expm1(yt_t).reshape(-1)
preds.append(yhat); trues.append(yt)
preds = np.concatenate(preds)
trues = np.concatenate(trues)
mask = np.isfinite(preds) & np.isfinite(trues)
dropped = int((~mask).sum())
if dropped:
        print(f"Dropped {dropped} non-finite pairs from holdout scoring.")
print({"holdout_MAE": float(mean_absolute_error(trues[mask], preds[mask]))})

# ---- predict requested DELIVERY_DATE ----
# find base row (previous day EOD with price)
prev_day = DELIVERY_DATE - dt.timedelta(days=1)
base_row = df_lags[(pd.to_datetime(df_lags["OperatingDTM"]).dt.date == prev_day) & (df_lags["Interval"]==24) & df_lags["hb_houston"].notna()]
if base_row.empty:
    raise RuntimeError(f"No base EOD price for {prev_day}")
ts0 = pd.to_datetime(base_row.iloc[0]["ts"])
y0  = np.float32(base_row.iloc[0]["hb_houston"])

# encoder window
win = df_lags.set_index("ts").loc[ts0 - pd.Timedelta(hours=W_PAST-1): ts0][enc_cols]
if win.shape[0] != W_PAST or win.isna().any().any():
    raise RuntimeError("Insufficient/NaN history for encoder window.")
Xe_pred = win.values.astype(np.float32)[None, ...]

# decoder future features (24 rows)
frows = df_future[(df_future["delivery_date"]==DELIVERY_DATE)].sort_values("delivery_interval")
if frows.shape[0] != H_FUT:
    raise RuntimeError("Spine missing 24 rows for delivery date.")
Xd_pred_full = frows[dec_future_cols].values.astype(np.float32)[None, ...]

# Apply the same keep mask used in training (drop columns that were all-NaN in train)
if 'dec_keep_mask' in globals():
    Xd_pred = Xd_pred_full[:, :, dec_keep_mask]
else:
    Xd_pred = Xd_pred_full

# impute + standardize pred using TRAIN stats
m = np.isnan(Xd_pred)
if m.any():
    Xd_pred[m] = np.broadcast_to(dec_med, Xd_pred.shape)[m]
Xe_pred_n = apply_standardizer(Xe_pred, enc_mu, enc_sd)
Xd_pred_n = apply_standardizer(Xd_pred, dec_mu, dec_sd)
# compute normalized scalar y0_n using spike-aware transform
y0_t = sgn_log1p(y0)
y0_n = np.float32(((y0_t - y_mu) / y_sd)).ravel()[0]

model.eval()
with torch.no_grad():
    yhat_n = model(torch.from_numpy(Xe_pred_n).to(DEVICE),
                   torch.from_numpy(Xd_pred_n).to(DEVICE),
                   torch.tensor([y0_n], dtype=torch.float32, device=DEVICE),
                   y_teacher=None, tf_prob=0.0).detach().cpu().numpy()[0]
yhat_t = (yhat_n * y_sd + y_mu).reshape(-1)
yhat = sgn_expm1(yhat_t)
# replace any residual NaN with the day's median
if np.isnan(yhat).any():
    day_med = float(np.nanmedian(yhat))
    yhat = np.nan_to_num(yhat, nan=day_med)

forecast = pd.DataFrame({
    "OperatingDTM": [DELIVERY_DATE]*H_FUT,
    "Interval": list(range(1, H_FUT+1)),
    "hb_houston_pred": yhat.tolist()
})
forecast


BinderException: Binder Error: Referenced column "temp_the_woodlands_tx" not found in FROM clause!
Candidate bindings: "hist_temp_the_woodlands_tx", "hist_temp_friendswood_tx", "hist_precip_the_woodlands_tx", "hist_hum_the_woodlands_tx", "hist_wind_the_woodlands_tx"

LINE 6:     temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx...
            ^

In [17]:
# ==== PyTorch seq2seq with ROBUST preprocessing (median impute + safe standardize + safe MAE) ====
import duckdb, pandas as pd, numpy as np, datetime as dt, math
import torch, torch.nn as nn
from sklearn.metrics import mean_absolute_error

DB = "../ProjectMain/db/data.duckdb"
DELIVERY_DATE = pd.Timestamp("2025-08-18").date()   # change target date if needed
W_PAST = 168
H_FUT = 24
HOLDOUT_DAYS = 60
HIDDEN = 128
EPOCHS = 20
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
rng = np.random.default_rng(42)
torch.manual_seed(42)

# ---------- helpers ----------
def fit_standardizer(X2d):
    """Return (mean,std) with std guarded (==0 -> 1). X2d: (N, F)"""
    mu = np.nanmean(X2d, axis=0)
    sd = np.nanstd(X2d, axis=0)
    sd = np.where(sd <= 1e-8, 1.0, sd)
    return mu, sd

def apply_standardizer(X, mu, sd):
    return (X - mu) / sd

# robust 3D imputer
def impute_inplace_3d(X, med_vec):
    """X: (N, T, F) or (N, W, F); med_vec: (F,). Fills NaNs in-place with per-feature medians."""
    mask = np.isnan(X)
    if mask.any():
        med_broadcast = np.broadcast_to(med_vec, X.shape)
        X[mask] = med_broadcast[mask]

# sanity checker
def assert_finite(name, arr):
    if not np.isfinite(arr).all():
        bad = int(np.isnan(arr).sum() + np.isinf(arr).sum())
        raise RuntimeError(f"{name} has {bad} non-finite values")

# ---------- ensure weather-enhanced views (hist + fcst) exist & helper transforms ----------
con_ensure = duckdb.connect(DB)
# Historical weather enhanced (to backfill when forecast is missing during training)
con_ensure.execute("""
CREATE OR REPLACE VIEW wx_hist_enh AS
WITH w AS (
  SELECT
    OperatingDTM, "interval" AS Interval,
    temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx, temp_baytown_tx, temp_houston_tx,
    hum_the_woodlands_tx,  hum_katy_tx,  hum_friendswood_tx,  hum_baytown_tx,  hum_houston_tx
  FROM vw_historical_weather_by_city
),
agg AS (
  SELECT
    OperatingDTM, Interval,
    (temp_the_woodlands_tx + temp_katy_tx + temp_friendswood_tx + temp_baytown_tx + temp_houston_tx)/5.0 AS temp_avg,
    (hum_the_woodlands_tx  + hum_katy_tx  + hum_friendswood_tx  + hum_baytown_tx  + hum_houston_tx)/5.0  AS hum_avg,
    GREATEST(temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx, temp_baytown_tx, temp_houston_tx)
      - LEAST(temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx, temp_baytown_tx, temp_houston_tx) AS temp_spread,
    GREATEST(hum_the_woodlands_tx, hum_katy_tx, hum_friendswood_tx, hum_baytown_tx, hum_houston_tx)
      - LEAST(hum_the_woodlands_tx, hum_katy_tx, hum_friendswood_tx, hum_baytown_tx, hum_houston_tx) AS hum_spread
  FROM w
)
SELECT
  a.*,
  a.temp_avg - LAG(a.temp_avg) OVER (ORDER BY OperatingDTM, Interval) AS temp_avg_ramp1,
  a.hum_avg  - LAG(a.hum_avg)  OVER (ORDER BY OperatingDTM, Interval) AS hum_avg_ramp1
FROM agg a;
""")
con_ensure.close()

# Signed log transforms for spikes
sgn = np.sign
abs_ = np.abs

def sgn_log1p(y):
    return sgn(y) * np.log1p(abs_(y))

def sgn_expm1(z):
    return sgn(z) * np.expm1(abs_(z))

# Hour-of-day weights (emphasize 17-21 local hours)
HOUR_WEIGHTS = np.ones(24, dtype=np.float32)
HOUR_WEIGHTS[16:21] = 2.0  # intervals 17..21

# ---------- pull data ----------
con = duckdb.connect(DB)
df_lags = con.execute("""
SELECT
  ts, OperatingDTM, Interval, hb_houston,
  p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
  dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean
FROM vw_master_spine_lags
ORDER BY ts
""").df()
df_lags["ts"] = pd.to_datetime(df_lags["ts"])

df_future = con.execute("""
SELECT
  f.OperatingDTM AS delivery_date,
  f.Interval     AS delivery_interval,
  f.hb_houston   AS target,
  -- base future features
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
  f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f,
  f.cal_sin_dow  AS cal_sin_dow_f,  f.cal_cos_dow  AS cal_cos_dow_f,
  -- load-enhanced joins
  l.net_wz, l.net_lz, l.net_wz_ramp1, l.net_lz_ramp1, l.lz_houston_ramp1, l.wz_spread, l.lz_spread,
  -- weather: COALESCE forecast -> historical
  COALESCE(wf.temp_avg,      wh.temp_avg)      AS temp_avg,
  COALESCE(wf.temp_spread,   wh.temp_spread)   AS temp_spread,
  COALESCE(wf.temp_avg_ramp1,wh.temp_avg_ramp1)AS temp_avg_ramp1,
  COALESCE(wf.hum_avg,       wh.hum_avg)       AS hum_avg,
  COALESCE(wf.hum_spread,    wh.hum_spread)    AS hum_spread,
  COALESCE(wf.hum_avg_ramp1, wh.hum_avg_ramp1) AS hum_avg_ramp1,
  CASE WHEN wf.temp_avg IS NULL THEN 1 ELSE 0 END AS is_weather_proxy
FROM vw_master_spine_ts f
LEFT JOIN load_fcst_enh l
  ON l.OperatingDTM = f.OperatingDTM AND l.Interval = f.Interval
LEFT JOIN wx_fcst_enh  wf
  ON wf.OperatingDTM = f.OperatingDTM AND wf.Interval = f.Interval
LEFT JOIN wx_hist_enh  wh
  ON wh.OperatingDTM = f.OperatingDTM AND wh.Interval = f.Interval
ORDER BY f.OperatingDTM, f.Interval
""").df()
con.close()

df_future["delivery_date"] = pd.to_datetime(df_future["delivery_date"]).dt.date
df_future["delivery_interval"] = df_future["delivery_interval"].astype(int)

# ---------- assemble samples (day-ahead) ----------
enc_cols = [
    "hb_houston","p_lag1","p_lag2","p_lag3","p_lag6","p_lag12","p_lag24","p_lag48","p_lag72","p_lag168",
    "dp1","dp24","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean"
]
dec_future_cols = [
    "cal_hour_f","cal_dow_f","cal_is_weekend_f","cal_sin_hour_f","cal_cos_hour_f","cal_sin_dow_f","cal_cos_dow_f",
    "net_wz","net_lz","net_wz_ramp1","net_lz_ramp1","lz_houston_ramp1","wz_spread","lz_spread",
    "temp_avg","temp_spread","temp_avg_ramp1","hum_avg","hum_spread","hum_avg_ramp1"
]

# base rows = end-of-day with price
base_rows = df_lags[(df_lags["hb_houston"].notna()) & (df_lags["Interval"]==24)][["ts","OperatingDTM","hb_houston"]].copy()
base_rows["base_date"] = pd.to_datetime(base_rows["OperatingDTM"]).dt.date
base_rows["delivery_date"] = base_rows["base_date"] + dt.timedelta(days=1)

# only use delivery days with 24 actuals for training/val
has24 = df_future.groupby("delivery_date")["target"].apply(lambda s: s.notna().sum()==24)
valid_delivery_dates = set(has24[has24].index)

df_lags_idx = df_lags.set_index("ts").sort_index()

samples = []
for _, r in base_rows.iterrows():
    ddate = r["delivery_date"]
    ts0 = r["ts"]
    # encoder window
    start = ts0 - pd.Timedelta(hours=W_PAST-1)
    enc_df = df_lags_idx.loc[start:ts0][enc_cols]
    if enc_df.shape[0] != W_PAST: 
        continue
    if enc_df.isna().any().any():
        continue  # strict: skip any encoder NaNs
    # decoder future rows
    fdf = df_future[(df_future["delivery_date"]==ddate)].sort_values("delivery_interval")
    if fdf.shape[0] != H_FUT:
        continue
    X_enc = enc_df.values.astype(np.float32)
    X_dec = fdf[dec_future_cols].values.astype(np.float32)
    y     = fdf["target"].values.astype(np.float32)
    y0    = np.float32(r["hb_houston"])
    samples.append((X_enc, X_dec, y, y0, ddate))

if len(samples) == 0:
    raise RuntimeError("No samples assembled; check data coverage or reduce W_PAST.")

# split by date
dates = sorted({s[4] for s in samples})
cut_date = dates[-1] - dt.timedelta(days=HOLDOUT_DAYS) if len(dates)>HOLDOUT_DAYS else dates[0]
train_idx = [i for i,s in enumerate(samples) if s[4] <= cut_date and s[4] in valid_delivery_dates]
val_idx   = [i for i,s in enumerate(samples) if s[4] >  cut_date and s[4] in valid_delivery_dates]

def stack(idxs):
    Xe = np.stack([samples[i][0] for i in idxs], axis=0)
    Xd = np.stack([samples[i][1] for i in idxs], axis=0)
    Y  = np.stack([samples[i][2] for i in idxs], axis=0)
    Y0 = np.stack([samples[i][3] for i in idxs], axis=0)
    return Xe, Xd, Y, Y0

Xe_tr, Xd_tr, Y_tr, Y0_tr = stack(train_idx)
Xe_va, Xd_va, Y_va, Y0_va = (None, None, None, None)
if len(val_idx) > 0:
    Xe_va, Xd_va, Y_va, Y0_va = stack(val_idx)

# ---------- diagnose + drop all-NaN decoder features, then impute (robust) ----------
# Flatten decoder train to 2D to inspect per-feature missingness
F_orig = Xd_tr.shape[2]
Xd_tr_2d_raw = Xd_tr.reshape(-1, F_orig)
nan_counts = np.isnan(Xd_tr_2d_raw).sum(axis=0)
all_nan_cols = nan_counts == Xd_tr_2d_raw.shape[0]

if all_nan_cols.any():
    dropped_cols = [dec_future_cols[i] for i,flag in enumerate(all_nan_cols) if flag]
    print(f"Dropping {int(all_nan_cols.sum())} decoder feature(s) with all-NaN in TRAIN: {dropped_cols}")
else:
    dropped_cols = []

# Keep mask to be reused at prediction time
dec_keep_mask = ~all_nan_cols

# Apply mask to decoder tensors
Xd_tr = Xd_tr[:, :, dec_keep_mask]
if Xe_va is not None:
    Xd_va = Xd_va[:, :, dec_keep_mask]

# Recompute medians on KEPT features
dec_med = np.nanmedian(Xd_tr.reshape(-1, Xd_tr.shape[2]), axis=0)
# Guard: if a median is still NaN for some reason, set to 0
dec_med = np.where(np.isnan(dec_med), 0.0, dec_med)

# Encoder median (rare but safe)
enc_med = np.nanmedian(Xe_tr.reshape(-1, Xe_tr.shape[2]), axis=0)
enc_med = np.where(np.isnan(enc_med), 0.0, enc_med)

# Impute in-place (3D-safe)
impute_inplace_3d(Xe_tr, enc_med)
impute_inplace_3d(Xd_tr, dec_med)
if Xe_va is not None:
    impute_inplace_3d(Xe_va, enc_med)
    impute_inplace_3d(Xd_va, dec_med)

# Targets should be complete for training/val; assert
assert np.isfinite(Y_tr).all(), "Y_tr has non-finite values"
if Y_va is not None:
    assert np.isfinite(Y_va).all(), "Y_va has non-finite values"

# ---------- safe standardization ----------
enc_mu, enc_sd = fit_standardizer(Xe_tr.reshape(-1, Xe_tr.shape[2]))
dec_mu, dec_sd = fit_standardizer(Xd_tr.reshape(-1, Xd_tr.shape[2]))
# ---- target transform (spike-aware) then standardize ----
Y_tr_t  = sgn_log1p(Y_tr)
y_mu, y_sd = fit_standardizer(Y_tr_t.reshape(-1,1))
y_mu = float(np.atleast_1d(y_mu)[0]); y_sd = float(np.atleast_1d(y_sd)[0])

Xe_tr_n = apply_standardizer(Xe_tr, enc_mu, enc_sd)
Xd_tr_n = apply_standardizer(Xd_tr, dec_mu, dec_sd)
Y_tr_n  = apply_standardizer(Y_tr_t, y_mu, y_sd).reshape(Y_tr.shape)
Y0_tr_t = sgn_log1p(Y0_tr.reshape(-1,1)).reshape(-1,1)
Y0_tr_n = apply_standardizer(Y0_tr_t, y_mu, y_sd).reshape(-1)

if Xe_va is not None:
    Xe_va_n = apply_standardizer(Xe_va, enc_mu, enc_sd)
    Xd_va_n = apply_standardizer(Xd_va, dec_mu, dec_sd)
    Y_va_t  = sgn_log1p(Y_va)
    Y_va_n  = apply_standardizer(Y_va_t,  y_mu,  y_sd).reshape(Y_va.shape)
    Y0_va_t = sgn_log1p(Y0_va.reshape(-1,1))
    Y0_va_n = apply_standardizer(Y0_va_t, y_mu, y_sd).reshape(-1)


# Final safety: ensure everything is finite
assert_finite("Xe_tr_n", Xe_tr_n)
assert_finite("Xd_tr_n", Xd_tr_n)
assert_finite("Y_tr_n",  Y_tr_n)
assert_finite("Y0_tr_n", Y0_tr_n)
if Xe_va is not None:
    assert_finite("Xe_va_n", Xe_va_n)
    assert_finite("Xd_va_n", Xd_va_n)
    assert_finite("Y_va_n",  Y_va_n)
    assert_finite("Y0_va_n", Y0_va_n)

# ---------- torch datasets ----------
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self, Xe, Xd, Y, Y0):
        self.Xe = torch.from_numpy(Xe).float()
        self.Xd = torch.from_numpy(Xd).float()
        self.Y  = torch.from_numpy(Y).float()
        self.Y0 = torch.from_numpy(Y0).float()
    def __len__(self): return self.Xe.shape[0]
    def __getitem__(self, i): return self.Xe[i], self.Xd[i], self.Y[i], self.Y0[i]

bs = 32
train_loader = torch.utils.data.DataLoader(Seq2SeqDataset(Xe_tr_n, Xd_tr_n, Y_tr_n, Y0_tr_n), batch_size=bs, shuffle=True)
val_loader = None if Xe_va is None else torch.utils.data.DataLoader(Seq2SeqDataset(Xe_va_n, Xd_va_n, Y_va_n, Y0_va_n), batch_size=bs, shuffle=False)

# ---------- model ----------
class Encoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim, hidden, batch_first=True)
    def forward(self, x):  # x: (B,W,E)
        _, h = self.rnn(x)
        return h  # (1,B,H)

class Decoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim + 1, hidden, batch_first=True)  # + prev y
        self.out = nn.Linear(hidden, 1)
    def forward(self, future_feats, h0, y0, teacher=None, tf_prob=0.5):
        B, T, D = future_feats.shape
        y_prev = y0.view(B,1)
        h = h0
        outs = []
        for t in range(T):
            x_t = torch.cat([future_feats[:,t,:], y_prev], dim=1).unsqueeze(1)  # (B,1,D+1)
            o, h = self.rnn(x_t, h)
            y_t = self.out(o[:, -1, :]).squeeze(1)
            outs.append(y_t)
            if (teacher is not None) and (rng.random() < tf_prob):
                y_prev = teacher[:,t].view(B,1)
            else:
                y_prev = y_t.view(B,1)
        return torch.stack(outs, dim=1)  # (B,24)

class Seq2Seq(nn.Module):
    def __init__(self, enc_in, dec_in, hidden):
        super().__init__()
        self.enc = Encoder(enc_in, hidden)
        self.dec = Decoder(dec_in, hidden)
    def forward(self, x_enc, x_dec, y0, y_teacher=None, tf_prob=0.5):
        h = self.enc(x_enc)
        return self.dec(x_dec, h, y0, y_teacher, tf_prob)

enc_in = Xe_tr_n.shape[2]
dec_in = Xd_tr_n.shape[2]
model = Seq2Seq(enc_in, dec_in, HIDDEN).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.HuberLoss()

def huber_weighted(yhat, y, w, delta=1.0):
    d = torch.abs(yhat - y)
    q = torch.clamp(d, max=delta)
    l = 0.5 * q**2 + delta * (d - q)
    return (l * w).mean()

def train_epoch():
    model.train()
    tot = 0.0
    for xenc, xdec, y, y0 in train_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        opt.zero_grad()
        yhat = model(xenc, xdec, y0, y_teacher=y, tf_prob=0.5)
        # time-step weights (B,24)
        w = torch.tensor(HOUR_WEIGHTS, device=y.device).unsqueeze(0).expand_as(y)
        loss = huber_weighted(yhat, y, w)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        tot += loss.item() * xenc.size(0)
    return tot / len(train_loader.dataset)

@torch.no_grad()
def eval_epoch():
    if val_loader is None: return None
    model.eval()
    tot = 0.0
    for xenc, xdec, y, y0 in val_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        yhat = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0)
        w = torch.tensor(HOUR_WEIGHTS, device=y.device).unsqueeze(0).expand_as(y)
        loss = huber_weighted(yhat, y, w)
        tot += loss.item() * xenc.size(0)
    return tot / len(val_loader.dataset)

for ep in range(1, EPOCHS+1):
    tr = train_epoch()
    va = eval_epoch()
    print(f"Epoch {ep:02d}  train_loss={tr:.4f}" + ("" if va is None else f"  val_loss={va:.4f}"))

# ---- evaluate holdout MAE in real units (SAFE) ----
if val_loader is not None:
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xenc, xdec, y, y0 in val_loader:
            xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
            yhat_n = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0).detach().cpu().numpy()
            y_n    = y.detach().cpu().numpy()
            # invert scaling + inverse spike transform
            yhat_t = (yhat_n * y_sd + y_mu)
            yt_t   = (y_n    * y_sd + y_mu)
            yhat = sgn_expm1(yhat_t).reshape(-1)
            yt   = sgn_expm1(yt_t).reshape(-1)
            preds.append(yhat)
            trues.append(yt)
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    mask = np.isfinite(preds) & np.isfinite(trues)
    dropped = int((~mask).sum())
    if dropped:
        print(f"Dropped {dropped} non-finite pairs from holdout scoring.")
    print({"holdout_MAE": float(mean_absolute_error(trues[mask], preds[mask]))})

# ---- predict requested DELIVERY_DATE ----

# find base row (previous day EOD with price)
prev_day = DELIVERY_DATE - dt.timedelta(days=1)
base_row = df_lags[(pd.to_datetime(df_lags["OperatingDTM"]).dt.date == prev_day) & (df_lags["Interval"]==24) & df_lags["hb_houston"].notna()]
if base_row.empty:
    raise RuntimeError(f"No base EOD price for {prev_day}")
ts0 = pd.to_datetime(base_row.iloc[0]["ts"])
y0  = np.float32(base_row.iloc[0]["hb_houston"])

# encoder window
win = df_lags.set_index("ts").loc[ts0 - pd.Timedelta(hours=W_PAST-1): ts0][enc_cols]
if win.shape[0] != W_PAST or win.isna().any().any():
    raise RuntimeError("Insufficient/NaN history for encoder window.")
Xe_pred = win.values.astype(np.float32)[None, ...]

# decoder future features (24 rows)
frows = df_future[(df_future["delivery_date"]==DELIVERY_DATE)].sort_values("delivery_interval")
if frows.shape[0] != H_FUT:
    raise RuntimeError("Spine missing 24 rows for delivery date.")
Xd_pred_full = frows[dec_future_cols].values.astype(np.float32)[None, ...]

# Apply the same keep mask used in training (drop columns that were all-NaN in train)
if 'dec_keep_mask' in globals():
    Xd_pred = Xd_pred_full[:, :, dec_keep_mask]
else:
    Xd_pred = Xd_pred_full

# impute + standardize pred using TRAIN stats
m = np.isnan(Xd_pred)
if m.any():
    Xd_pred[m] = np.broadcast_to(dec_med, Xd_pred.shape)[m]
Xe_pred_n = apply_standardizer(Xe_pred, enc_mu, enc_sd)
Xd_pred_n = apply_standardizer(Xd_pred, dec_mu, dec_sd)
# compute normalized scalar y0_n using spike-aware transform
y0_t = sgn_log1p(y0)
y0_n = np.float32(((y0_t - y_mu) / y_sd)).ravel()[0]

model.eval()
with torch.no_grad():
    yhat_n = model(torch.from_numpy(Xe_pred_n).to(DEVICE),
                   torch.from_numpy(Xd_pred_n).to(DEVICE),
                   torch.tensor([y0_n], dtype=torch.float32, device=DEVICE),
                   y_teacher=None, tf_prob=0.0).detach().cpu().numpy()[0]
yhat_t = (yhat_n * y_sd + y_mu).reshape(-1)
yhat = sgn_expm1(yhat_t)
# replace any residual NaN with the day's median
if np.isnan(yhat).any():
    day_med = float(np.nanmedian(yhat))
    yhat = np.nan_to_num(yhat, nan=day_med)

forecast = pd.DataFrame({
    "OperatingDTM": [DELIVERY_DATE]*H_FUT,
    "Interval": list(range(1, H_FUT+1)),
    "hb_houston_pred": yhat.tolist()
})
forecast


BinderException: Binder Error: Referenced column "temp_the_woodlands_tx" not found in FROM clause!
Candidate bindings: "hist_temp_the_woodlands_tx", "hist_temp_friendswood_tx", "hist_precip_the_woodlands_tx", "hist_hum_the_woodlands_tx", "hist_wind_the_woodlands_tx"

LINE 6:     temp_the_woodlands_tx, temp_katy_tx, temp_friendswood_tx...
            ^

In [18]:
# ==== PyTorch seq2seq with ROBUST preprocessing (median impute + safe standardize + safe MAE) ====
import duckdb, pandas as pd, numpy as np, datetime as dt, math
import torch, torch.nn as nn
from sklearn.metrics import mean_absolute_error

DB = "../ProjectMain/db/data.duckdb"
DELIVERY_DATE = pd.Timestamp("2025-08-18").date()   # change target date if needed
W_PAST = 168
H_FUT = 24
HOLDOUT_DAYS = 60
HIDDEN = 128
EPOCHS = 20
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
rng = np.random.default_rng(42)
torch.manual_seed(42)

# ---------- helpers ----------
def fit_standardizer(X2d):
    """Return (mean,std) with std guarded (==0 -> 1). X2d: (N, F)"""
    mu = np.nanmean(X2d, axis=0)
    sd = np.nanstd(X2d, axis=0)
    sd = np.where(sd <= 1e-8, 1.0, sd)
    return mu, sd

def apply_standardizer(X, mu, sd):
    return (X - mu) / sd

# robust 3D imputer
def impute_inplace_3d(X, med_vec):
    """X: (N, T, F) or (N, W, F); med_vec: (F,). Fills NaNs in-place with per-feature medians."""
    mask = np.isnan(X)
    if mask.any():
        med_broadcast = np.broadcast_to(med_vec, X.shape)
        X[mask] = med_broadcast[mask]

# sanity checker
def assert_finite(name, arr):
    if not np.isfinite(arr).all():
        bad = int(np.isnan(arr).sum() + np.isinf(arr).sum())
        raise RuntimeError(f"{name} has {bad} non-finite values")

# ---------- ensure weather-enhanced views (hist + fcst) exist & helper transforms ----------
con_ensure = duckdb.connect(DB)
# Historical weather enhanced (to backfill when forecast is missing during training)
con_ensure.execute("""
CREATE OR REPLACE VIEW wx_hist_enh AS
WITH agg AS (
  SELECT
    OperatingDTM, "interval" AS Interval,
    (hist_temp_the_woodlands_tx + hist_temp_katy_tx + hist_temp_friendswood_tx + hist_temp_baytown_tx + hist_temp_houston_tx)/5.0 AS temp_avg,
    (hist_hum_the_woodlands_tx  + hist_hum_katy_tx  + hist_hum_friendswood_tx  + hist_hum_baytown_tx  + hist_hum_houston_tx)/5.0  AS hum_avg,
    GREATEST(hist_temp_the_woodlands_tx, hist_temp_katy_tx, hist_temp_friendswood_tx, hist_temp_baytown_tx, hist_temp_houston_tx)
      - LEAST(hist_temp_the_woodlands_tx, hist_temp_katy_tx, hist_temp_friendswood_tx, hist_temp_baytown_tx, hist_temp_houston_tx) AS temp_spread,
    GREATEST(hist_hum_the_woodlands_tx, hist_hum_katy_tx, hist_hum_friendswood_tx, hist_hum_baytown_tx, hist_hum_houston_tx)
      - LEAST(hist_hum_the_woodlands_tx, hist_hum_katy_tx, hist_hum_friendswood_tx, hist_hum_baytown_tx, hist_hum_houston_tx) AS hum_spread
  FROM vw_historical_weather_by_city
)
SELECT
  a.*,
  a.temp_avg - LAG(a.temp_avg) OVER (ORDER BY OperatingDTM, Interval) AS temp_avg_ramp1,
  a.hum_avg  - LAG(a.hum_avg)  OVER (ORDER BY OperatingDTM, Interval) AS hum_avg_ramp1
FROM agg a;
""")
con_ensure.close()

# Signed log transforms for spikes
sgn = np.sign
abs_ = np.abs

def sgn_log1p(y):
    return sgn(y) * np.log1p(abs_(y))

def sgn_expm1(z):
    return sgn(z) * np.expm1(abs_(z))

# Hour-of-day weights (emphasize 17-21 local hours)
HOUR_WEIGHTS = np.ones(24, dtype=np.float32)
HOUR_WEIGHTS[16:21] = 2.0  # intervals 17..21

# ---------- pull data ----------
con = duckdb.connect(DB)
df_lags = con.execute("""
SELECT
  ts, OperatingDTM, Interval, hb_houston,
  p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
  dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean
FROM vw_master_spine_lags
ORDER BY ts
""").df()
df_lags["ts"] = pd.to_datetime(df_lags["ts"])

df_future = con.execute("""
SELECT
  f.OperatingDTM AS delivery_date,
  f.Interval     AS delivery_interval,
  f.hb_houston   AS target,
  -- base future features
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
  f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f,
  f.cal_sin_dow  AS cal_sin_dow_f,  f.cal_cos_dow  AS cal_cos_dow_f,
  -- load-enhanced joins
  l.net_wz, l.net_lz, l.net_wz_ramp1, l.net_lz_ramp1, l.lz_houston_ramp1, l.wz_spread, l.lz_spread,
  -- weather: COALESCE forecast -> historical
  COALESCE(wf.temp_avg,      wh.temp_avg)      AS temp_avg,
  COALESCE(wf.temp_spread,   wh.temp_spread)   AS temp_spread,
  COALESCE(wf.temp_avg_ramp1,wh.temp_avg_ramp1)AS temp_avg_ramp1,
  COALESCE(wf.hum_avg,       wh.hum_avg)       AS hum_avg,
  COALESCE(wf.hum_spread,    wh.hum_spread)    AS hum_spread,
  COALESCE(wf.hum_avg_ramp1, wh.hum_avg_ramp1) AS hum_avg_ramp1,
  CASE WHEN wf.temp_avg IS NULL THEN 1 ELSE 0 END AS is_weather_proxy
FROM vw_master_spine_ts f
LEFT JOIN load_fcst_enh l
  ON l.OperatingDTM = f.OperatingDTM AND l.Interval = f.Interval
LEFT JOIN wx_fcst_enh  wf
  ON wf.OperatingDTM = f.OperatingDTM AND wf.Interval = f.Interval
LEFT JOIN wx_hist_enh  wh
  ON wh.OperatingDTM = f.OperatingDTM AND wh.Interval = f.Interval
ORDER BY f.OperatingDTM, f.Interval
""").df()
con.close()

df_future["delivery_date"] = pd.to_datetime(df_future["delivery_date"]).dt.date
df_future["delivery_interval"] = df_future["delivery_interval"].astype(int)

# ---------- assemble samples (day-ahead) ----------
enc_cols = [
    "hb_houston","p_lag1","p_lag2","p_lag3","p_lag6","p_lag12","p_lag24","p_lag48","p_lag72","p_lag168",
    "dp1","dp24","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean"
]
dec_future_cols = [
    "cal_hour_f","cal_dow_f","cal_is_weekend_f","cal_sin_hour_f","cal_cos_hour_f","cal_sin_dow_f","cal_cos_dow_f",
    "net_wz","net_lz","net_wz_ramp1","net_lz_ramp1","lz_houston_ramp1","wz_spread","lz_spread",
    "temp_avg","temp_spread","temp_avg_ramp1","hum_avg","hum_spread","hum_avg_ramp1"
]

# base rows = end-of-day with price
base_rows = df_lags[(df_lags["hb_houston"].notna()) & (df_lags["Interval"]==24)][["ts","OperatingDTM","hb_houston"]].copy()
base_rows["base_date"] = pd.to_datetime(base_rows["OperatingDTM"]).dt.date
base_rows["delivery_date"] = base_rows["base_date"] + dt.timedelta(days=1)

# only use delivery days with 24 actuals for training/val
has24 = df_future.groupby("delivery_date")["target"].apply(lambda s: s.notna().sum()==24)
valid_delivery_dates = set(has24[has24].index)

df_lags_idx = df_lags.set_index("ts").sort_index()

samples = []
for _, r in base_rows.iterrows():
    ddate = r["delivery_date"]
    ts0 = r["ts"]
    # encoder window
    start = ts0 - pd.Timedelta(hours=W_PAST-1)
    enc_df = df_lags_idx.loc[start:ts0][enc_cols]
    if enc_df.shape[0] != W_PAST: 
        continue
    if enc_df.isna().any().any():
        continue  # strict: skip any encoder NaNs
    # decoder future rows
    fdf = df_future[(df_future["delivery_date"]==ddate)].sort_values("delivery_interval")
    if fdf.shape[0] != H_FUT:
        continue
    X_enc = enc_df.values.astype(np.float32)
    X_dec = fdf[dec_future_cols].values.astype(np.float32)
    y     = fdf["target"].values.astype(np.float32)
    y0    = np.float32(r["hb_houston"])
    samples.append((X_enc, X_dec, y, y0, ddate))

if len(samples) == 0:
    raise RuntimeError("No samples assembled; check data coverage or reduce W_PAST.")

# split by date
dates = sorted({s[4] for s in samples})
cut_date = dates[-1] - dt.timedelta(days=HOLDOUT_DAYS) if len(dates)>HOLDOUT_DAYS else dates[0]
train_idx = [i for i,s in enumerate(samples) if s[4] <= cut_date and s[4] in valid_delivery_dates]
val_idx   = [i for i,s in enumerate(samples) if s[4] >  cut_date and s[4] in valid_delivery_dates]

def stack(idxs):
    Xe = np.stack([samples[i][0] for i in idxs], axis=0)
    Xd = np.stack([samples[i][1] for i in idxs], axis=0)
    Y  = np.stack([samples[i][2] for i in idxs], axis=0)
    Y0 = np.stack([samples[i][3] for i in idxs], axis=0)
    return Xe, Xd, Y, Y0

Xe_tr, Xd_tr, Y_tr, Y0_tr = stack(train_idx)
Xe_va, Xd_va, Y_va, Y0_va = (None, None, None, None)
if len(val_idx) > 0:
    Xe_va, Xd_va, Y_va, Y0_va = stack(val_idx)

# ---------- diagnose + drop all-NaN decoder features, then impute (robust) ----------
# Flatten decoder train to 2D to inspect per-feature missingness
F_orig = Xd_tr.shape[2]
Xd_tr_2d_raw = Xd_tr.reshape(-1, F_orig)
nan_counts = np.isnan(Xd_tr_2d_raw).sum(axis=0)
all_nan_cols = nan_counts == Xd_tr_2d_raw.shape[0]

if all_nan_cols.any():
    dropped_cols = [dec_future_cols[i] for i,flag in enumerate(all_nan_cols) if flag]
    print(f"Dropping {int(all_nan_cols.sum())} decoder feature(s) with all-NaN in TRAIN: {dropped_cols}")
else:
    dropped_cols = []

# Keep mask to be reused at prediction time
dec_keep_mask = ~all_nan_cols

# Apply mask to decoder tensors
Xd_tr = Xd_tr[:, :, dec_keep_mask]
if Xe_va is not None:
    Xd_va = Xd_va[:, :, dec_keep_mask]

# Recompute medians on KEPT features
dec_med = np.nanmedian(Xd_tr.reshape(-1, Xd_tr.shape[2]), axis=0)
# Guard: if a median is still NaN for some reason, set to 0
dec_med = np.where(np.isnan(dec_med), 0.0, dec_med)

# Encoder median (rare but safe)
enc_med = np.nanmedian(Xe_tr.reshape(-1, Xe_tr.shape[2]), axis=0)
enc_med = np.where(np.isnan(enc_med), 0.0, enc_med)

# Impute in-place (3D-safe)
impute_inplace_3d(Xe_tr, enc_med)
impute_inplace_3d(Xd_tr, dec_med)
if Xe_va is not None:
    impute_inplace_3d(Xe_va, enc_med)
    impute_inplace_3d(Xd_va, dec_med)

# Targets should be complete for training/val; assert
assert np.isfinite(Y_tr).all(), "Y_tr has non-finite values"
if Y_va is not None:
    assert np.isfinite(Y_va).all(), "Y_va has non-finite values"

# ---------- safe standardization ----------
enc_mu, enc_sd = fit_standardizer(Xe_tr.reshape(-1, Xe_tr.shape[2]))
dec_mu, dec_sd = fit_standardizer(Xd_tr.reshape(-1, Xd_tr.shape[2]))
# ---- target transform (spike-aware) then standardize ----
Y_tr_t  = sgn_log1p(Y_tr)
y_mu, y_sd = fit_standardizer(Y_tr_t.reshape(-1,1))
y_mu = float(np.atleast_1d(y_mu)[0]); y_sd = float(np.atleast_1d(y_sd)[0])

Xe_tr_n = apply_standardizer(Xe_tr, enc_mu, enc_sd)
Xd_tr_n = apply_standardizer(Xd_tr, dec_mu, dec_sd)
Y_tr_n  = apply_standardizer(Y_tr_t, y_mu, y_sd).reshape(Y_tr.shape)
Y0_tr_t = sgn_log1p(Y0_tr.reshape(-1,1)).reshape(-1,1)
Y0_tr_n = apply_standardizer(Y0_tr_t, y_mu, y_sd).reshape(-1)

if Xe_va is not None:
    Xe_va_n = apply_standardizer(Xe_va, enc_mu, enc_sd)
    Xd_va_n = apply_standardizer(Xd_va, dec_mu, dec_sd)
    Y_va_t  = sgn_log1p(Y_va)
    Y_va_n  = apply_standardizer(Y_va_t,  y_mu,  y_sd).reshape(Y_va.shape)
    Y0_va_t = sgn_log1p(Y0_va.reshape(-1,1))
    Y0_va_n = apply_standardizer(Y0_va_t, y_mu, y_sd).reshape(-1)


# Final safety: ensure everything is finite
assert_finite("Xe_tr_n", Xe_tr_n)
assert_finite("Xd_tr_n", Xd_tr_n)
assert_finite("Y_tr_n",  Y_tr_n)
assert_finite("Y0_tr_n", Y0_tr_n)
if Xe_va is not None:
    assert_finite("Xe_va_n", Xe_va_n)
    assert_finite("Xd_va_n", Xd_va_n)
    assert_finite("Y_va_n",  Y_va_n)
    assert_finite("Y0_va_n", Y0_va_n)

# ---------- torch datasets ----------
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self, Xe, Xd, Y, Y0):
        self.Xe = torch.from_numpy(Xe).float()
        self.Xd = torch.from_numpy(Xd).float()
        self.Y  = torch.from_numpy(Y).float()
        self.Y0 = torch.from_numpy(Y0).float()
    def __len__(self): return self.Xe.shape[0]
    def __getitem__(self, i): return self.Xe[i], self.Xd[i], self.Y[i], self.Y0[i]

bs = 32
train_loader = torch.utils.data.DataLoader(Seq2SeqDataset(Xe_tr_n, Xd_tr_n, Y_tr_n, Y0_tr_n), batch_size=bs, shuffle=True)
val_loader = None if Xe_va is None else torch.utils.data.DataLoader(Seq2SeqDataset(Xe_va_n, Xd_va_n, Y_va_n, Y0_va_n), batch_size=bs, shuffle=False)

# ---------- model ----------
class Encoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim, hidden, batch_first=True)
    def forward(self, x):  # x: (B,W,E)
        _, h = self.rnn(x)
        return h  # (1,B,H)

class Decoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim + 1, hidden, batch_first=True)  # + prev y
        self.out = nn.Linear(hidden, 1)
    def forward(self, future_feats, h0, y0, teacher=None, tf_prob=0.5):
        B, T, D = future_feats.shape
        y_prev = y0.view(B,1)
        h = h0
        outs = []
        for t in range(T):
            x_t = torch.cat([future_feats[:,t,:], y_prev], dim=1).unsqueeze(1)  # (B,1,D+1)
            o, h = self.rnn(x_t, h)
            y_t = self.out(o[:, -1, :]).squeeze(1)
            outs.append(y_t)
            if (teacher is not None) and (rng.random() < tf_prob):
                y_prev = teacher[:,t].view(B,1)
            else:
                y_prev = y_t.view(B,1)
        return torch.stack(outs, dim=1)  # (B,24)

class Seq2Seq(nn.Module):
    def __init__(self, enc_in, dec_in, hidden):
        super().__init__()
        self.enc = Encoder(enc_in, hidden)
        self.dec = Decoder(dec_in, hidden)
    def forward(self, x_enc, x_dec, y0, y_teacher=None, tf_prob=0.5):
        h = self.enc(x_enc)
        return self.dec(x_dec, h, y0, y_teacher, tf_prob)

enc_in = Xe_tr_n.shape[2]
dec_in = Xd_tr_n.shape[2]
model = Seq2Seq(enc_in, dec_in, HIDDEN).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.HuberLoss()

def huber_weighted(yhat, y, w, delta=1.0):
    d = torch.abs(yhat - y)
    q = torch.clamp(d, max=delta)
    l = 0.5 * q**2 + delta * (d - q)
    return (l * w).mean()

def train_epoch():
    model.train()
    tot = 0.0
    for xenc, xdec, y, y0 in train_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        opt.zero_grad()
        yhat = model(xenc, xdec, y0, y_teacher=y, tf_prob=0.5)
        # time-step weights (B,24)
        w = torch.tensor(HOUR_WEIGHTS, device=y.device).unsqueeze(0).expand_as(y)
        loss = huber_weighted(yhat, y, w)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        tot += loss.item() * xenc.size(0)
    return tot / len(train_loader.dataset)

@torch.no_grad()
def eval_epoch():
    if val_loader is None: return None
    model.eval()
    tot = 0.0
    for xenc, xdec, y, y0 in val_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        yhat = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0)
        w = torch.tensor(HOUR_WEIGHTS, device=y.device).unsqueeze(0).expand_as(y)
        loss = huber_weighted(yhat, y, w)
        tot += loss.item() * xenc.size(0)
    return tot / len(val_loader.dataset)

for ep in range(1, EPOCHS+1):
    tr = train_epoch()
    va = eval_epoch()
    print(f"Epoch {ep:02d}  train_loss={tr:.4f}" + ("" if va is None else f"  val_loss={va:.4f}"))

# ---- evaluate holdout MAE in real units (SAFE) ----
if val_loader is not None:
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xenc, xdec, y, y0 in val_loader:
            xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
            yhat_n = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0).detach().cpu().numpy()
            y_n    = y.detach().cpu().numpy()
            # invert scaling + inverse spike transform
            yhat_t = (yhat_n * y_sd + y_mu)
            yt_t   = (y_n    * y_sd + y_mu)
            yhat = sgn_expm1(yhat_t).reshape(-1)
            yt   = sgn_expm1(yt_t).reshape(-1)
            preds.append(yhat)
            trues.append(yt)
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    mask = np.isfinite(preds) & np.isfinite(trues)
    dropped = int((~mask).sum())
    if dropped:
        print(f"Dropped {dropped} non-finite pairs from holdout scoring.")
    print({"holdout_MAE": float(mean_absolute_error(trues[mask], preds[mask]))})

# ---- predict requested DELIVERY_DATE ----

# find base row (previous day EOD with price)
prev_day = DELIVERY_DATE - dt.timedelta(days=1)
base_row = df_lags[(pd.to_datetime(df_lags["OperatingDTM"]).dt.date == prev_day) & (df_lags["Interval"]==24) & df_lags["hb_houston"].notna()]
if base_row.empty:
    raise RuntimeError(f"No base EOD price for {prev_day}")
ts0 = pd.to_datetime(base_row.iloc[0]["ts"])
y0  = np.float32(base_row.iloc[0]["hb_houston"])

# encoder window
win = df_lags.set_index("ts").loc[ts0 - pd.Timedelta(hours=W_PAST-1): ts0][enc_cols]
if win.shape[0] != W_PAST or win.isna().any().any():
    raise RuntimeError("Insufficient/NaN history for encoder window.")
Xe_pred = win.values.astype(np.float32)[None, ...]

# decoder future features (24 rows)
frows = df_future[(df_future["delivery_date"]==DELIVERY_DATE)].sort_values("delivery_interval")
if frows.shape[0] != H_FUT:
    raise RuntimeError("Spine missing 24 rows for delivery date.")
Xd_pred_full = frows[dec_future_cols].values.astype(np.float32)[None, ...]

# Apply the same keep mask used in training (drop columns that were all-NaN in train)
if 'dec_keep_mask' in globals():
    Xd_pred = Xd_pred_full[:, :, dec_keep_mask]
else:
    Xd_pred = Xd_pred_full

# impute + standardize pred using TRAIN stats
m = np.isnan(Xd_pred)
if m.any():
    Xd_pred[m] = np.broadcast_to(dec_med, Xd_pred.shape)[m]
Xe_pred_n = apply_standardizer(Xe_pred, enc_mu, enc_sd)
Xd_pred_n = apply_standardizer(Xd_pred, dec_mu, dec_sd)
# compute normalized scalar y0_n using spike-aware transform
y0_t = sgn_log1p(y0)
y0_n = np.float32(((y0_t - y_mu) / y_sd)).ravel()[0]

model.eval()
with torch.no_grad():
    yhat_n = model(torch.from_numpy(Xe_pred_n).to(DEVICE),
                   torch.from_numpy(Xd_pred_n).to(DEVICE),
                   torch.tensor([y0_n], dtype=torch.float32, device=DEVICE),
                   y_teacher=None, tf_prob=0.0).detach().cpu().numpy()[0]
yhat_t = (yhat_n * y_sd + y_mu).reshape(-1)
yhat = sgn_expm1(yhat_t)
# replace any residual NaN with the day's median
if np.isnan(yhat).any():
    day_med = float(np.nanmedian(yhat))
    yhat = np.nan_to_num(yhat, nan=day_med)

forecast = pd.DataFrame({
    "OperatingDTM": [DELIVERY_DATE]*H_FUT,
    "Interval": list(range(1, H_FUT+1)),
    "hb_houston_pred": yhat.tolist()
})
forecast


Epoch 01  train_loss=0.3078  val_loss=0.1501
Epoch 02  train_loss=0.2225  val_loss=0.1873
Epoch 03  train_loss=0.1700  val_loss=0.1696
Epoch 04  train_loss=0.1443  val_loss=0.3007
Epoch 05  train_loss=0.1277  val_loss=0.1242
Epoch 06  train_loss=0.1166  val_loss=0.1986
Epoch 07  train_loss=0.1162  val_loss=0.2015
Epoch 08  train_loss=0.1009  val_loss=0.1260
Epoch 09  train_loss=0.0988  val_loss=0.1296
Epoch 10  train_loss=0.0936  val_loss=0.1493
Epoch 11  train_loss=0.0932  val_loss=0.1031
Epoch 12  train_loss=0.0966  val_loss=0.1198
Epoch 13  train_loss=0.0889  val_loss=0.1190
Epoch 14  train_loss=0.0875  val_loss=0.1079
Epoch 15  train_loss=0.0814  val_loss=0.1033
Epoch 16  train_loss=0.0795  val_loss=0.0920
Epoch 17  train_loss=0.0810  val_loss=0.1136
Epoch 18  train_loss=0.0793  val_loss=0.1885
Epoch 19  train_loss=0.0782  val_loss=0.1102
Epoch 20  train_loss=0.0785  val_loss=0.1913
{'holdout_MAE': 10.914621353149414}


,OperatingDTM,Interval,hb_houston_pred
0,2025-08-18,1,32.939453
1,2025-08-18,2,31.030563
2,2025-08-18,3,28.780090
3,2025-08-18,4,26.337210
4,2025-08-18,5,24.641678
5,2025-08-18,6,23.785067
6,2025-08-18,7,23.253466
7,2025-08-18,8,21.878345
8,2025-08-18,9,18.707254
9,2025-08-18,10,16.998575


Peak-hour weighting bump
Weighted-Huber now uses 3× weight for intervals 17–21 (late-day surge). This pulls the model to fit the ramp more aggressively.

New ramp & day-over-day features (no schema changes needed):
In the df_future query, added window-derived drivers that are super predictive for peaks:

net_wz_ramp3, net_wz_ramp6, net_lz_ramp3, net_lz_ramp6

lz_houston_ramp3, lz_houston_ramp6

net_wz_dod, net_lz_dod, lz_houston_dod (value − value 24 hours ago)

These are computed inline with LAG(…,3/6/24) over (ORDER BY OperatingDTM, Interval) and wired into dec_future_cols. No extra views needed.

Weather backfill stays on
We keep COALESCE(forecast→historical) and the spike-aware target transform. (Key in getting to to 10.9 MAE.)

In [20]:
# ==== PyTorch seq2seq with ROBUST preprocessing (median impute + safe standardize + safe MAE) ====
import duckdb, pandas as pd, numpy as np, datetime as dt, math
import torch, torch.nn as nn
from sklearn.metrics import mean_absolute_error

DB = "../ProjectMain/db/data.duckdb"
DELIVERY_DATE = pd.Timestamp("2025-08-18").date()   # change target date if needed
W_PAST = 168
H_FUT = 24
HOLDOUT_DAYS = 60
HIDDEN = 128
EPOCHS = 20
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
rng = np.random.default_rng(42)
torch.manual_seed(42)

# ---------- helpers ----------
def fit_standardizer(X2d):
    """Return (mean,std) with std guarded (==0 -> 1). X2d: (N, F)"""
    mu = np.nanmean(X2d, axis=0)
    sd = np.nanstd(X2d, axis=0)
    sd = np.where(sd <= 1e-8, 1.0, sd)
    return mu, sd

def apply_standardizer(X, mu, sd):
    return (X - mu) / sd

# robust 3D imputer
def impute_inplace_3d(X, med_vec):
    """X: (N, T, F) or (N, W, F); med_vec: (F,). Fills NaNs in-place with per-feature medians."""
    mask = np.isnan(X)
    if mask.any():
        med_broadcast = np.broadcast_to(med_vec, X.shape)
        X[mask] = med_broadcast[mask]

# sanity checker
def assert_finite(name, arr):
    if not np.isfinite(arr).all():
        bad = int(np.isnan(arr).sum() + np.isinf(arr).sum())
        raise RuntimeError(f"{name} has {bad} non-finite values")

# ---------- ensure weather-enhanced views (hist + fcst) exist & helper transforms ----------
con_ensure = duckdb.connect(DB)
# Historical weather enhanced (to backfill when forecast is missing during training)
con_ensure.execute("""
CREATE OR REPLACE VIEW wx_hist_enh AS
WITH agg AS (
  SELECT
    OperatingDTM, "interval" AS Interval,
    (hist_temp_the_woodlands_tx + hist_temp_katy_tx + hist_temp_friendswood_tx + hist_temp_baytown_tx + hist_temp_houston_tx)/5.0 AS temp_avg,
    (hist_hum_the_woodlands_tx  + hist_hum_katy_tx  + hist_hum_friendswood_tx  + hist_hum_baytown_tx  + hist_hum_houston_tx)/5.0  AS hum_avg,
    GREATEST(hist_temp_the_woodlands_tx, hist_temp_katy_tx, hist_temp_friendswood_tx, hist_temp_baytown_tx, hist_temp_houston_tx)
      - LEAST(hist_temp_the_woodlands_tx, hist_temp_katy_tx, hist_temp_friendswood_tx, hist_temp_baytown_tx, hist_temp_houston_tx) AS temp_spread,
    GREATEST(hist_hum_the_woodlands_tx, hist_hum_katy_tx, hist_hum_friendswood_tx, hist_hum_baytown_tx, hist_hum_houston_tx)
      - LEAST(hist_hum_the_woodlands_tx, hist_hum_katy_tx, hist_hum_friendswood_tx, hist_hum_baytown_tx, hist_hum_houston_tx) AS hum_spread
  FROM vw_historical_weather_by_city
)
SELECT
  a.*,
  a.temp_avg - LAG(a.temp_avg) OVER (ORDER BY OperatingDTM, Interval) AS temp_avg_ramp1,
  a.hum_avg  - LAG(a.hum_avg)  OVER (ORDER BY OperatingDTM, Interval) AS hum_avg_ramp1
FROM agg a;
""")
con_ensure.close()

# Signed log transforms for spikes
sgn = np.sign
abs_ = np.abs

def sgn_log1p(y):
    return sgn(y) * np.log1p(abs_(y))

def sgn_expm1(z):
    return sgn(z) * np.expm1(abs_(z))

# Hour-of-day weights (emphasize 17-21 local hours)
HOUR_WEIGHTS = np.ones(24, dtype=np.float32)
# Emphasize 17..21 (late-day surge hours) a bit more
HOUR_WEIGHTS[16:21] = 3.0  # intervals 17..21

# ---------- pull data ----------
con = duckdb.connect(DB)
df_lags = con.execute("""
SELECT
  ts, OperatingDTM, Interval, hb_houston,
  p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
  dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean
FROM vw_master_spine_lags
ORDER BY ts
""").df()
df_lags["ts"] = pd.to_datetime(df_lags["ts"])

df_future = con.execute("""
SELECT
  f.OperatingDTM AS delivery_date,
  f.Interval     AS delivery_interval,
  f.hb_houston   AS target,
  -- base future features
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
  f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f,
  f.cal_sin_dow  AS cal_sin_dow_f,  f.cal_cos_dow  AS cal_cos_dow_f,
  -- load-enhanced joins
  l.net_wz, l.net_lz, l.net_wz_ramp1, l.net_lz_ramp1, l.lz_houston_ramp1, l.wz_spread, l.lz_spread,
  -- weather: COALESCE forecast -> historical
  COALESCE(wf.temp_avg,      wh.temp_avg)      AS temp_avg,
  COALESCE(wf.temp_spread,   wh.temp_spread)   AS temp_spread,
  COALESCE(wf.temp_avg_ramp1,wh.temp_avg_ramp1)AS temp_avg_ramp1,
  COALESCE(wf.hum_avg,       wh.hum_avg)       AS hum_avg,
  COALESCE(wf.hum_spread,    wh.hum_spread)    AS hum_spread,
  COALESCE(wf.hum_avg_ramp1, wh.hum_avg_ramp1) AS hum_avg_ramp1,
  CASE WHEN wf.temp_avg IS NULL THEN 1 ELSE 0 END AS is_weather_proxy,
  -- additional ramps & day-over-day deltas to help with peaks (computed over time order)
  (l.net_wz      - LAG(l.net_wz,      3)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_wz_ramp3,
  (l.net_wz      - LAG(l.net_wz,      6)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_wz_ramp6,
  (l.net_lz      - LAG(l.net_lz,      3)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_lz_ramp3,
  (l.net_lz      - LAG(l.net_lz,      6)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_lz_ramp6,
  (l.lz_houston  - LAG(l.lz_houston,  3)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS lz_houston_ramp3,
  (l.lz_houston  - LAG(l.lz_houston,  6)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS lz_houston_ramp6,
  (l.net_wz      - LAG(l.net_wz,     24)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_wz_dod,
  (l.net_lz      - LAG(l.net_lz,     24)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_lz_dod,
  (l.lz_houston  - LAG(l.lz_houston, 24)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS lz_houston_dod
FROM vw_master_spine_ts f
LEFT JOIN load_fcst_enh l
  ON l.OperatingDTM = f.OperatingDTM AND l.Interval = f.Interval
LEFT JOIN wx_fcst_enh  wf
  ON wf.OperatingDTM = f.OperatingDTM AND wf.Interval = f.Interval
LEFT JOIN wx_hist_enh  wh
  ON wh.OperatingDTM = f.OperatingDTM AND wh.Interval = f.Interval
ORDER BY f.OperatingDTM, f.Interval
""").df()
con.close()

df_future["delivery_date"] = pd.to_datetime(df_future["delivery_date"]).dt.date
df_future["delivery_interval"] = df_future["delivery_interval"].astype(int)

# ---------- assemble samples (day-ahead) ----------
enc_cols = [
    "hb_houston","p_lag1","p_lag2","p_lag3","p_lag6","p_lag12","p_lag24","p_lag48","p_lag72","p_lag168",
    "dp1","dp24","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean"
]
dec_future_cols = [
    "cal_hour_f","cal_dow_f","cal_is_weekend_f","cal_sin_hour_f","cal_cos_hour_f","cal_sin_dow_f","cal_cos_dow_f",
    "net_wz","net_lz","net_wz_ramp1","net_lz_ramp1","lz_houston_ramp1","wz_spread","lz_spread",
    "temp_avg","temp_spread","temp_avg_ramp1","hum_avg","hum_spread","hum_avg_ramp1",
    # NEW: multi-horizon ramps + day-over-day deltas
    "net_wz_ramp3","net_wz_ramp6","net_lz_ramp3","net_lz_ramp6",
    "lz_houston_ramp3","lz_houston_ramp6","net_wz_dod","net_lz_dod","lz_houston_dod"
]

# base rows = end-of-day with price
base_rows = df_lags[(df_lags["hb_houston"].notna()) & (df_lags["Interval"]==24)][["ts","OperatingDTM","hb_houston"]].copy()
base_rows["base_date"] = pd.to_datetime(base_rows["OperatingDTM"]).dt.date
base_rows["delivery_date"] = base_rows["base_date"] + dt.timedelta(days=1)

# only use delivery days with 24 actuals for training/val
has24 = df_future.groupby("delivery_date")["target"].apply(lambda s: s.notna().sum()==24)
valid_delivery_dates = set(has24[has24].index)

df_lags_idx = df_lags.set_index("ts").sort_index()

samples = []
for _, r in base_rows.iterrows():
    ddate = r["delivery_date"]
    ts0 = r["ts"]
    # encoder window
    start = ts0 - pd.Timedelta(hours=W_PAST-1)
    enc_df = df_lags_idx.loc[start:ts0][enc_cols]
    if enc_df.shape[0] != W_PAST: 
        continue
    if enc_df.isna().any().any():
        continue  # strict: skip any encoder NaNs
    # decoder future rows
    fdf = df_future[(df_future["delivery_date"]==ddate)].sort_values("delivery_interval")
    if fdf.shape[0] != H_FUT:
        continue
    X_enc = enc_df.values.astype(np.float32)
    X_dec = fdf[dec_future_cols].values.astype(np.float32)
    y     = fdf["target"].values.astype(np.float32)
    y0    = np.float32(r["hb_houston"])
    samples.append((X_enc, X_dec, y, y0, ddate))

if len(samples) == 0:
    raise RuntimeError("No samples assembled; check data coverage or reduce W_PAST.")

# split by date
dates = sorted({s[4] for s in samples})
cut_date = dates[-1] - dt.timedelta(days=HOLDOUT_DAYS) if len(dates)>HOLDOUT_DAYS else dates[0]
train_idx = [i for i,s in enumerate(samples) if s[4] <= cut_date and s[4] in valid_delivery_dates]
val_idx   = [i for i,s in enumerate(samples) if s[4] >  cut_date and s[4] in valid_delivery_dates]

def stack(idxs):
    Xe = np.stack([samples[i][0] for i in idxs], axis=0)
    Xd = np.stack([samples[i][1] for i in idxs], axis=0)
    Y  = np.stack([samples[i][2] for i in idxs], axis=0)
    Y0 = np.stack([samples[i][3] for i in idxs], axis=0)
    return Xe, Xd, Y, Y0

Xe_tr, Xd_tr, Y_tr, Y0_tr = stack(train_idx)
Xe_va, Xd_va, Y_va, Y0_va = (None, None, None, None)
if len(val_idx) > 0:
    Xe_va, Xd_va, Y_va, Y0_va = stack(val_idx)

# ---------- diagnose + drop all-NaN decoder features, then impute (robust) ----------
# Flatten decoder train to 2D to inspect per-feature missingness
F_orig = Xd_tr.shape[2]
Xd_tr_2d_raw = Xd_tr.reshape(-1, F_orig)
nan_counts = np.isnan(Xd_tr_2d_raw).sum(axis=0)
all_nan_cols = nan_counts == Xd_tr_2d_raw.shape[0]

if all_nan_cols.any():
    dropped_cols = [dec_future_cols[i] for i,flag in enumerate(all_nan_cols) if flag]
    print(f"Dropping {int(all_nan_cols.sum())} decoder feature(s) with all-NaN in TRAIN: {dropped_cols}")
else:
    dropped_cols = []

# Keep mask to be reused at prediction time
dec_keep_mask = ~all_nan_cols

# Apply mask to decoder tensors
Xd_tr = Xd_tr[:, :, dec_keep_mask]
if Xe_va is not None:
    Xd_va = Xd_va[:, :, dec_keep_mask]

# Recompute medians on KEPT features
dec_med = np.nanmedian(Xd_tr.reshape(-1, Xd_tr.shape[2]), axis=0)
# Guard: if a median is still NaN for some reason, set to 0
dec_med = np.where(np.isnan(dec_med), 0.0, dec_med)

# Encoder median (rare but safe)
enc_med = np.nanmedian(Xe_tr.reshape(-1, Xe_tr.shape[2]), axis=0)
enc_med = np.where(np.isnan(enc_med), 0.0, enc_med)

# Impute in-place (3D-safe)
impute_inplace_3d(Xe_tr, enc_med)
impute_inplace_3d(Xd_tr, dec_med)
if Xe_va is not None:
    impute_inplace_3d(Xe_va, enc_med)
    impute_inplace_3d(Xd_va, dec_med)

# Targets should be complete for training/val; assert
assert np.isfinite(Y_tr).all(), "Y_tr has non-finite values"
if Y_va is not None:
    assert np.isfinite(Y_va).all(), "Y_va has non-finite values"

# ---------- safe standardization ----------
enc_mu, enc_sd = fit_standardizer(Xe_tr.reshape(-1, Xe_tr.shape[2]))
dec_mu, dec_sd = fit_standardizer(Xd_tr.reshape(-1, Xd_tr.shape[2]))
# ---- target transform (spike-aware) then standardize ----
Y_tr_t  = sgn_log1p(Y_tr)
y_mu, y_sd = fit_standardizer(Y_tr_t.reshape(-1,1))
y_mu = float(np.atleast_1d(y_mu)[0]); y_sd = float(np.atleast_1d(y_sd)[0])

Xe_tr_n = apply_standardizer(Xe_tr, enc_mu, enc_sd)
Xd_tr_n = apply_standardizer(Xd_tr, dec_mu, dec_sd)
Y_tr_n  = apply_standardizer(Y_tr_t, y_mu, y_sd).reshape(Y_tr.shape)
Y0_tr_t = sgn_log1p(Y0_tr.reshape(-1,1)).reshape(-1,1)
Y0_tr_n = apply_standardizer(Y0_tr_t, y_mu, y_sd).reshape(-1)

if Xe_va is not None:
    Xe_va_n = apply_standardizer(Xe_va, enc_mu, enc_sd)
    Xd_va_n = apply_standardizer(Xd_va, dec_mu, dec_sd)
    Y_va_t  = sgn_log1p(Y_va)
    Y_va_n  = apply_standardizer(Y_va_t,  y_mu,  y_sd).reshape(Y_va.shape)
    Y0_va_t = sgn_log1p(Y0_va.reshape(-1,1))
    Y0_va_n = apply_standardizer(Y0_va_t, y_mu, y_sd).reshape(-1)


# Final safety: ensure everything is finite
assert_finite("Xe_tr_n", Xe_tr_n)
assert_finite("Xd_tr_n", Xd_tr_n)
assert_finite("Y_tr_n",  Y_tr_n)
assert_finite("Y0_tr_n", Y0_tr_n)
if Xe_va is not None:
    assert_finite("Xe_va_n", Xe_va_n)
    assert_finite("Xd_va_n", Xd_va_n)
    assert_finite("Y_va_n",  Y_va_n)
    assert_finite("Y0_va_n", Y0_va_n)

# ---------- torch datasets ----------
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self, Xe, Xd, Y, Y0):
        self.Xe = torch.from_numpy(Xe).float()
        self.Xd = torch.from_numpy(Xd).float()
        self.Y  = torch.from_numpy(Y).float()
        self.Y0 = torch.from_numpy(Y0).float()
    def __len__(self): return self.Xe.shape[0]
    def __getitem__(self, i): return self.Xe[i], self.Xd[i], self.Y[i], self.Y0[i]

bs = 32
train_loader = torch.utils.data.DataLoader(Seq2SeqDataset(Xe_tr_n, Xd_tr_n, Y_tr_n, Y0_tr_n), batch_size=bs, shuffle=True)
val_loader = None if Xe_va is None else torch.utils.data.DataLoader(Seq2SeqDataset(Xe_va_n, Xd_va_n, Y_va_n, Y0_va_n), batch_size=bs, shuffle=False)

# ---------- model ----------
class Encoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim, hidden, batch_first=True)
    def forward(self, x):  # x: (B,W,E)
        _, h = self.rnn(x)
        return h  # (1,B,H)

class Decoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim + 1, hidden, batch_first=True)  # + prev y
        self.out = nn.Linear(hidden, 1)
    def forward(self, future_feats, h0, y0, teacher=None, tf_prob=0.5):
        B, T, D = future_feats.shape
        y_prev = y0.view(B,1)
        h = h0
        outs = []
        for t in range(T):
            x_t = torch.cat([future_feats[:,t,:], y_prev], dim=1).unsqueeze(1)  # (B,1,D+1)
            o, h = self.rnn(x_t, h)
            y_t = self.out(o[:, -1, :]).squeeze(1)
            outs.append(y_t)
            if (teacher is not None) and (rng.random() < tf_prob):
                y_prev = teacher[:,t].view(B,1)
            else:
                y_prev = y_t.view(B,1)
        return torch.stack(outs, dim=1)  # (B,24)

class Seq2Seq(nn.Module):
    def __init__(self, enc_in, dec_in, hidden):
        super().__init__()
        self.enc = Encoder(enc_in, hidden)
        self.dec = Decoder(dec_in, hidden)
    def forward(self, x_enc, x_dec, y0, y_teacher=None, tf_prob=0.5):
        h = self.enc(x_enc)
        return self.dec(x_dec, h, y0, y_teacher, tf_prob)

enc_in = Xe_tr_n.shape[2]
dec_in = Xd_tr_n.shape[2]
model = Seq2Seq(enc_in, dec_in, HIDDEN).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.HuberLoss()

def huber_weighted(yhat, y, w, delta=1.0):
    d = torch.abs(yhat - y)
    q = torch.clamp(d, max=delta)
    l = 0.5 * q**2 + delta * (d - q)
    return (l * w).mean()

def train_epoch():
    model.train()
    tot = 0.0
    for xenc, xdec, y, y0 in train_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        opt.zero_grad()
        yhat = model(xenc, xdec, y0, y_teacher=y, tf_prob=0.5)
        # time-step weights (B,24)
        w = torch.tensor(HOUR_WEIGHTS, device=y.device).unsqueeze(0).expand_as(y)
        loss = huber_weighted(yhat, y, w)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        tot += loss.item() * xenc.size(0)
    return tot / len(train_loader.dataset)

@torch.no_grad()
def eval_epoch():
    if val_loader is None: return None
    model.eval()
    tot = 0.0
    for xenc, xdec, y, y0 in val_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        yhat = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0)
        w = torch.tensor(HOUR_WEIGHTS, device=y.device).unsqueeze(0).expand_as(y)
        loss = huber_weighted(yhat, y, w)
        tot += loss.item() * xenc.size(0)
    return tot / len(val_loader.dataset)

for ep in range(1, EPOCHS+1):
    tr = train_epoch()
    va = eval_epoch()
    print(f"Epoch {ep:02d}  train_loss={tr:.4f}" + ("" if va is None else f"  val_loss={va:.4f}"))

# ---- evaluate holdout MAE in real units (SAFE) ----
if val_loader is not None:
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xenc, xdec, y, y0 in val_loader:
            xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
            yhat_n = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0).detach().cpu().numpy()
            y_n    = y.detach().cpu().numpy()
            # invert scaling + inverse spike transform
            yhat_t = (yhat_n * y_sd + y_mu)
            yt_t   = (y_n    * y_sd + y_mu)
            yhat = sgn_expm1(yhat_t).reshape(-1)
            yt   = sgn_expm1(yt_t).reshape(-1)
            preds.append(yhat)
            trues.append(yt)
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    mask = np.isfinite(preds) & np.isfinite(trues)
    dropped = int((~mask).sum())
    if dropped:
        print(f"Dropped {dropped} non-finite pairs from holdout scoring.")
    print({"holdout_MAE": float(mean_absolute_error(trues[mask], preds[mask]))})

# ---- predict requested DELIVERY_DATE ----

# find base row (previous day EOD with price)
prev_day = DELIVERY_DATE - dt.timedelta(days=1)
base_row = df_lags[(pd.to_datetime(df_lags["OperatingDTM"]).dt.date == prev_day) & (df_lags["Interval"]==24) & df_lags["hb_houston"].notna()]
if base_row.empty:
    raise RuntimeError(f"No base EOD price for {prev_day}")
ts0 = pd.to_datetime(base_row.iloc[0]["ts"])
y0  = np.float32(base_row.iloc[0]["hb_houston"])

# encoder window
win = df_lags.set_index("ts").loc[ts0 - pd.Timedelta(hours=W_PAST-1): ts0][enc_cols]
if win.shape[0] != W_PAST or win.isna().any().any():
    raise RuntimeError("Insufficient/NaN history for encoder window.")
Xe_pred = win.values.astype(np.float32)[None, ...]

# decoder future features (24 rows)
frows = df_future[(df_future["delivery_date"]==DELIVERY_DATE)].sort_values("delivery_interval")
if frows.shape[0] != H_FUT:
    raise RuntimeError("Spine missing 24 rows for delivery date.")
Xd_pred_full = frows[dec_future_cols].values.astype(np.float32)[None, ...]

# Apply the same keep mask used in training (drop columns that were all-NaN in train)
if 'dec_keep_mask' in globals():
    Xd_pred = Xd_pred_full[:, :, dec_keep_mask]
else:
    Xd_pred = Xd_pred_full

# impute + standardize pred using TRAIN stats
m = np.isnan(Xd_pred)
if m.any():
    Xd_pred[m] = np.broadcast_to(dec_med, Xd_pred.shape)[m]
Xe_pred_n = apply_standardizer(Xe_pred, enc_mu, enc_sd)
Xd_pred_n = apply_standardizer(Xd_pred, dec_mu, dec_sd)
# compute normalized scalar y0_n using spike-aware transform
y0_t = sgn_log1p(y0)
y0_n = np.float32(((y0_t - y_mu) / y_sd)).ravel()[0]

model.eval()
with torch.no_grad():
    yhat_n = model(torch.from_numpy(Xe_pred_n).to(DEVICE),
                   torch.from_numpy(Xd_pred_n).to(DEVICE),
                   torch.tensor([y0_n], dtype=torch.float32, device=DEVICE),
                   y_teacher=None, tf_prob=0.0).detach().cpu().numpy()[0]
yhat_t = (yhat_n * y_sd + y_mu).reshape(-1)
yhat = sgn_expm1(yhat_t)
# replace any residual NaN with the day's median
if np.isnan(yhat).any():
    day_med = float(np.nanmedian(yhat))
    yhat = np.nan_to_num(yhat, nan=day_med)

forecast = pd.DataFrame({
    "OperatingDTM": [DELIVERY_DATE]*H_FUT,
    "Interval": list(range(1, H_FUT+1)),
    "hb_houston_pred": yhat.tolist()
})
forecast


Epoch 01  train_loss=0.3803  val_loss=0.2468
Epoch 02  train_loss=0.2899  val_loss=0.3045
Epoch 03  train_loss=0.2260  val_loss=0.2425
Epoch 04  train_loss=0.1887  val_loss=0.1840
Epoch 05  train_loss=0.1600  val_loss=0.2060
Epoch 06  train_loss=0.1455  val_loss=0.1757
Epoch 07  train_loss=0.1401  val_loss=0.3374
Epoch 08  train_loss=0.1185  val_loss=0.3643
Epoch 09  train_loss=0.1192  val_loss=0.2190
Epoch 10  train_loss=0.1121  val_loss=0.1660
Epoch 11  train_loss=0.1226  val_loss=0.3145
Epoch 12  train_loss=0.1163  val_loss=0.2120
Epoch 13  train_loss=0.1181  val_loss=0.3052
Epoch 14  train_loss=0.1066  val_loss=0.1803
Epoch 15  train_loss=0.1021  val_loss=0.1534
Epoch 16  train_loss=0.0991  val_loss=0.2331
Epoch 17  train_loss=0.1006  val_loss=0.1326
Epoch 18  train_loss=0.0953  val_loss=0.1384
Epoch 19  train_loss=0.0970  val_loss=0.1241
Epoch 20  train_loss=0.0966  val_loss=0.1566
{'holdout_MAE': 10.522457122802734}


,OperatingDTM,Interval,hb_houston_pred
0,2025-08-18,1,36.066525
1,2025-08-18,2,36.127316
2,2025-08-18,3,33.805424
3,2025-08-18,4,31.458508
4,2025-08-18,5,30.529922
5,2025-08-18,6,30.530283
6,2025-08-18,7,29.699850
7,2025-08-18,8,26.760725
8,2025-08-18,9,23.487507
9,2025-08-18,10,21.850462


 holdout MAE improved again ($10.52 vs ~10.9 before), which is a good sign.

The shape now captures the afternoon ramp and a sizeable evening surge.

What’s left: it still ramps a bit too early ( H16 122 vs actual ~66) and still underestimates the peak top (H19–H21 predicted ~112/135/74 vs actual ~224/216/135).

Quick read by key hours

    Early morning (H1–H10): within single-digit dollars most hours 

    Midday (H12–H16): over (e.g., ~37/51/70/95/122 vs ~35/43/51/55/66) → ramp starts too soon

    Evening (H19–H21): under (111/135/74 vs 224/216/135) → peak height still low

In [21]:
# ==== PyTorch seq2seq with ROBUST preprocessing (median impute + safe standardize + safe MAE) ====
import duckdb, pandas as pd, numpy as np, datetime as dt, math
import torch, torch.nn as nn
from sklearn.metrics import mean_absolute_error

DB = "../ProjectMain/db/data.duckdb"
DELIVERY_DATE = pd.Timestamp("2025-08-18").date()   # change target date if needed
W_PAST = 168
H_FUT = 24
HOLDOUT_DAYS = 60
HIDDEN = 128
EPOCHS = 20
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
rng = np.random.default_rng(42)
torch.manual_seed(42)

# ---------- helpers ----------
def fit_standardizer(X2d):
    """Return (mean,std) with std guarded (==0 -> 1). X2d: (N, F)"""
    mu = np.nanmean(X2d, axis=0)
    sd = np.nanstd(X2d, axis=0)
    sd = np.where(sd <= 1e-8, 1.0, sd)
    return mu, sd

def apply_standardizer(X, mu, sd):
    return (X - mu) / sd

# robust 3D imputer
def impute_inplace_3d(X, med_vec):
    """X: (N, T, F) or (N, W, F); med_vec: (F,). Fills NaNs in-place with per-feature medians."""
    mask = np.isnan(X)
    if mask.any():
        med_broadcast = np.broadcast_to(med_vec, X.shape)
        X[mask] = med_broadcast[mask]

# sanity checker
def assert_finite(name, arr):
    if not np.isfinite(arr).all():
        bad = int(np.isnan(arr).sum() + np.isinf(arr).sum())
        raise RuntimeError(f"{name} has {bad} non-finite values")

# ---------- ensure weather-enhanced views (hist + fcst) exist & helper transforms ----------
con_ensure = duckdb.connect(DB)
# Historical weather enhanced (to backfill when forecast is missing during training)
con_ensure.execute("""
CREATE OR REPLACE VIEW wx_hist_enh AS
WITH agg AS (
  SELECT
    OperatingDTM, "interval" AS Interval,
    (hist_temp_the_woodlands_tx + hist_temp_katy_tx + hist_temp_friendswood_tx + hist_temp_baytown_tx + hist_temp_houston_tx)/5.0 AS temp_avg,
    (hist_hum_the_woodlands_tx  + hist_hum_katy_tx  + hist_hum_friendswood_tx  + hist_hum_baytown_tx  + hist_hum_houston_tx)/5.0  AS hum_avg,
    GREATEST(hist_temp_the_woodlands_tx, hist_temp_katy_tx, hist_temp_friendswood_tx, hist_temp_baytown_tx, hist_temp_houston_tx)
      - LEAST(hist_temp_the_woodlands_tx, hist_temp_katy_tx, hist_temp_friendswood_tx, hist_temp_baytown_tx, hist_temp_houston_tx) AS temp_spread,
    GREATEST(hist_hum_the_woodlands_tx, hist_hum_katy_tx, hist_hum_friendswood_tx, hist_hum_baytown_tx, hist_hum_houston_tx)
      - LEAST(hist_hum_the_woodlands_tx, hist_hum_katy_tx, hist_hum_friendswood_tx, hist_hum_baytown_tx, hist_hum_houston_tx) AS hum_spread
  FROM vw_historical_weather_by_city
)
SELECT
  a.*,
  a.temp_avg - LAG(a.temp_avg) OVER (ORDER BY OperatingDTM, Interval) AS temp_avg_ramp1,
  a.hum_avg  - LAG(a.hum_avg)  OVER (ORDER BY OperatingDTM, Interval) AS hum_avg_ramp1
FROM agg a;
""")
con_ensure.close()

# Signed log transforms for spikes
sgn = np.sign
abs_ = np.abs

def sgn_log1p(y):
    return sgn(y) * np.log1p(abs_(y))

def sgn_expm1(z):
    return sgn(z) * np.expm1(abs_(z))

# Hour-of-day weights (emphasize 17-21 local hours)
HOUR_WEIGHTS = np.ones(24, dtype=np.float32)
# Reweight hours to reduce early overshoot and emphasize true peak
# index 0->H1, ..., 23->H24
HOUR_WEIGHTS[:] = 1.0
HOUR_WEIGHTS[15] = 1.5     # H16
HOUR_WEIGHTS[16:18] = 2.0  # H17-18
HOUR_WEIGHTS[18:20] = 5.0  # H19-20 (peak focus)
HOUR_WEIGHTS[20] = 3.0     # H21

# ---------- pull data ----------
con = duckdb.connect(DB)
df_lags = con.execute("""
SELECT
  ts, OperatingDTM, Interval, hb_houston,
  p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
  dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean
FROM vw_master_spine_lags
ORDER BY ts
""").df()
df_lags["ts"] = pd.to_datetime(df_lags["ts"])

df_future = con.execute("""
SELECT
  f.OperatingDTM AS delivery_date,
  f.Interval     AS delivery_interval,
  f.hb_houston   AS target,
  -- base future features
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
  f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f,
  f.cal_sin_dow  AS cal_sin_dow_f,  f.cal_cos_dow  AS cal_cos_dow_f,
  -- load-enhanced joins
  l.net_wz, l.net_lz, l.net_wz_ramp1, l.net_lz_ramp1, l.lz_houston_ramp1, l.wz_spread, l.lz_spread,
  -- weather: COALESCE forecast -> historical
  COALESCE(wf.temp_avg,      wh.temp_avg)      AS temp_avg,
  COALESCE(wf.temp_spread,   wh.temp_spread)   AS temp_spread,
  COALESCE(wf.temp_avg_ramp1,wh.temp_avg_ramp1)AS temp_avg_ramp1,
  COALESCE(wf.hum_avg,       wh.hum_avg)       AS hum_avg,
  COALESCE(wf.hum_spread,    wh.hum_spread)    AS hum_spread,
  COALESCE(wf.hum_avg_ramp1, wh.hum_avg_ramp1) AS hum_avg_ramp1,
  CASE WHEN wf.temp_avg IS NULL THEN 1 ELSE 0 END AS is_weather_proxy,
  -- additional ramps & day-over-day deltas to help with peaks (computed over time order)
  (l.net_wz      - LAG(l.net_wz,      3)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_wz_ramp3,
  (l.net_wz      - LAG(l.net_wz,      6)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_wz_ramp6,
  (l.net_lz      - LAG(l.net_lz,      3)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_lz_ramp3,
  (l.net_lz      - LAG(l.net_lz,      6)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_lz_ramp6,
  (l.lz_houston  - LAG(l.lz_houston,  3)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS lz_houston_ramp3,
  (l.lz_houston  - LAG(l.lz_houston,  6)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS lz_houston_ramp6,
  (l.net_wz      - LAG(l.net_wz,     24)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_wz_dod,
  (l.net_lz      - LAG(l.net_lz,     24)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_lz_dod,
  (l.lz_houston  - LAG(l.lz_houston, 24)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS lz_houston_dod
FROM vw_master_spine_ts f
LEFT JOIN load_fcst_enh l
  ON l.OperatingDTM = f.OperatingDTM AND l.Interval = f.Interval
LEFT JOIN wx_fcst_enh  wf
  ON wf.OperatingDTM = f.OperatingDTM AND wf.Interval = f.Interval
LEFT JOIN wx_hist_enh  wh
  ON wh.OperatingDTM = f.OperatingDTM AND wh.Interval = f.Interval
ORDER BY f.OperatingDTM, f.Interval
""").df()
con.close()

df_future["delivery_date"] = pd.to_datetime(df_future["delivery_date"]).dt.date
df_future["delivery_interval"] = df_future["delivery_interval"].astype(int)

# ---------- assemble samples (day-ahead) ----------
enc_cols = [
    "hb_houston","p_lag1","p_lag2","p_lag3","p_lag6","p_lag12","p_lag24","p_lag48","p_lag72","p_lag168",
    "dp1","dp24","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean"
]
dec_future_cols = [
    "cal_hour_f","cal_dow_f","cal_is_weekend_f","cal_sin_hour_f","cal_cos_hour_f","cal_sin_dow_f","cal_cos_dow_f",
    "net_wz","net_lz","net_wz_ramp1","net_lz_ramp1","lz_houston_ramp1","wz_spread","lz_spread",
    "temp_avg","temp_spread","temp_avg_ramp1","hum_avg","hum_spread","hum_avg_ramp1",
    # NEW: multi-horizon ramps + day-over-day deltas
    "net_wz_ramp3","net_wz_ramp6","net_lz_ramp3","net_lz_ramp6",
    "lz_houston_ramp3","lz_houston_ramp6","net_wz_dod","net_lz_dod","lz_houston_dod",
    # Identify when weather came from historical backfill instead of forecast
    "is_weather_proxy"
]

# base rows = end-of-day with price
base_rows = df_lags[(df_lags["hb_houston"].notna()) & (df_lags["Interval"]==24)][["ts","OperatingDTM","hb_houston"]].copy()
base_rows["base_date"] = pd.to_datetime(base_rows["OperatingDTM"]).dt.date
base_rows["delivery_date"] = base_rows["base_date"] + dt.timedelta(days=1)

# only use delivery days with 24 actuals for training/val
has24 = df_future.groupby("delivery_date")["target"].apply(lambda s: s.notna().sum()==24)
valid_delivery_dates = set(has24[has24].index)

df_lags_idx = df_lags.set_index("ts").sort_index()

samples = []
for _, r in base_rows.iterrows():
    ddate = r["delivery_date"]
    ts0 = r["ts"]
    # encoder window
    start = ts0 - pd.Timedelta(hours=W_PAST-1)
    enc_df = df_lags_idx.loc[start:ts0][enc_cols]
    if enc_df.shape[0] != W_PAST: 
        continue
    if enc_df.isna().any().any():
        continue  # strict: skip any encoder NaNs
    # decoder future rows
    fdf = df_future[(df_future["delivery_date"]==ddate)].sort_values("delivery_interval")
    if fdf.shape[0] != H_FUT:
        continue
    X_enc = enc_df.values.astype(np.float32)
    X_dec = fdf[dec_future_cols].values.astype(np.float32)
    y     = fdf["target"].values.astype(np.float32)
    y0    = np.float32(r["hb_houston"])
    samples.append((X_enc, X_dec, y, y0, ddate))

if len(samples) == 0:
    raise RuntimeError("No samples assembled; check data coverage or reduce W_PAST.")

# split by date
dates = sorted({s[4] for s in samples})
cut_date = dates[-1] - dt.timedelta(days=HOLDOUT_DAYS) if len(dates)>HOLDOUT_DAYS else dates[0]
train_idx = [i for i,s in enumerate(samples) if s[4] <= cut_date and s[4] in valid_delivery_dates]
val_idx   = [i for i,s in enumerate(samples) if s[4] >  cut_date and s[4] in valid_delivery_dates]

def stack(idxs):
    Xe = np.stack([samples[i][0] for i in idxs], axis=0)
    Xd = np.stack([samples[i][1] for i in idxs], axis=0)
    Y  = np.stack([samples[i][2] for i in idxs], axis=0)
    Y0 = np.stack([samples[i][3] for i in idxs], axis=0)
    return Xe, Xd, Y, Y0

Xe_tr, Xd_tr, Y_tr, Y0_tr = stack(train_idx)
Xe_va, Xd_va, Y_va, Y0_va = (None, None, None, None)
if len(val_idx) > 0:
    Xe_va, Xd_va, Y_va, Y0_va = stack(val_idx)

# ---------- diagnose + drop all-NaN decoder features, then impute (robust) ----------
# Flatten decoder train to 2D to inspect per-feature missingness
F_orig = Xd_tr.shape[2]
Xd_tr_2d_raw = Xd_tr.reshape(-1, F_orig)
nan_counts = np.isnan(Xd_tr_2d_raw).sum(axis=0)
all_nan_cols = nan_counts == Xd_tr_2d_raw.shape[0]

if all_nan_cols.any():
    dropped_cols = [dec_future_cols[i] for i,flag in enumerate(all_nan_cols) if flag]
    print(f"Dropping {int(all_nan_cols.sum())} decoder feature(s) with all-NaN in TRAIN: {dropped_cols}")
else:
    dropped_cols = []

# Keep mask to be reused at prediction time
dec_keep_mask = ~all_nan_cols

# Apply mask to decoder tensors
Xd_tr = Xd_tr[:, :, dec_keep_mask]
if Xe_va is not None:
    Xd_va = Xd_va[:, :, dec_keep_mask]

# Recompute medians on KEPT features
dec_med = np.nanmedian(Xd_tr.reshape(-1, Xd_tr.shape[2]), axis=0)
# Guard: if a median is still NaN for some reason, set to 0
dec_med = np.where(np.isnan(dec_med), 0.0, dec_med)

# Encoder median (rare but safe)
enc_med = np.nanmedian(Xe_tr.reshape(-1, Xe_tr.shape[2]), axis=0)
enc_med = np.where(np.isnan(enc_med), 0.0, enc_med)

# Impute in-place (3D-safe)
impute_inplace_3d(Xe_tr, enc_med)
impute_inplace_3d(Xd_tr, dec_med)
if Xe_va is not None:
    impute_inplace_3d(Xe_va, enc_med)
    impute_inplace_3d(Xd_va, dec_med)

# Targets should be complete for training/val; assert
assert np.isfinite(Y_tr).all(), "Y_tr has non-finite values"
if Y_va is not None:
    assert np.isfinite(Y_va).all(), "Y_va has non-finite values"

# ---------- safe standardization ----------
enc_mu, enc_sd = fit_standardizer(Xe_tr.reshape(-1, Xe_tr.shape[2]))
dec_mu, dec_sd = fit_standardizer(Xd_tr.reshape(-1, Xd_tr.shape[2]))
# ---- target transform (spike-aware) then standardize ----
Y_tr_t  = sgn_log1p(Y_tr)
y_mu, y_sd = fit_standardizer(Y_tr_t.reshape(-1,1))
y_mu = float(np.atleast_1d(y_mu)[0]); y_sd = float(np.atleast_1d(y_sd)[0])

Xe_tr_n = apply_standardizer(Xe_tr, enc_mu, enc_sd)
Xd_tr_n = apply_standardizer(Xd_tr, dec_mu, dec_sd)
Y_tr_n  = apply_standardizer(Y_tr_t, y_mu, y_sd).reshape(Y_tr.shape)
Y0_tr_t = sgn_log1p(Y0_tr.reshape(-1,1)).reshape(-1,1)
Y0_tr_n = apply_standardizer(Y0_tr_t, y_mu, y_sd).reshape(-1)

if Xe_va is not None:
    Xe_va_n = apply_standardizer(Xe_va, enc_mu, enc_sd)
    Xd_va_n = apply_standardizer(Xd_va, dec_mu, dec_sd)
    Y_va_t  = sgn_log1p(Y_va)
    Y_va_n  = apply_standardizer(Y_va_t,  y_mu,  y_sd).reshape(Y_va.shape)
    Y0_va_t = sgn_log1p(Y0_va.reshape(-1,1))
    Y0_va_n = apply_standardizer(Y0_va_t, y_mu, y_sd).reshape(-1)


# Final safety: ensure everything is finite
assert_finite("Xe_tr_n", Xe_tr_n)
assert_finite("Xd_tr_n", Xd_tr_n)
assert_finite("Y_tr_n",  Y_tr_n)
assert_finite("Y0_tr_n", Y0_tr_n)
if Xe_va is not None:
    assert_finite("Xe_va_n", Xe_va_n)
    assert_finite("Xd_va_n", Xd_va_n)
    assert_finite("Y_va_n",  Y_va_n)
    assert_finite("Y0_va_n", Y0_va_n)

# ---------- torch datasets ----------
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self, Xe, Xd, Y, Y0):
        self.Xe = torch.from_numpy(Xe).float()
        self.Xd = torch.from_numpy(Xd).float()
        self.Y  = torch.from_numpy(Y).float()
        self.Y0 = torch.from_numpy(Y0).float()
    def __len__(self): return self.Xe.shape[0]
    def __getitem__(self, i): return self.Xe[i], self.Xd[i], self.Y[i], self.Y0[i]

bs = 32
train_loader = torch.utils.data.DataLoader(Seq2SeqDataset(Xe_tr_n, Xd_tr_n, Y_tr_n, Y0_tr_n), batch_size=bs, shuffle=True)
val_loader = None if Xe_va is None else torch.utils.data.DataLoader(Seq2SeqDataset(Xe_va_n, Xd_va_n, Y_va_n, Y0_va_n), batch_size=bs, shuffle=False)

# ---------- model ----------
class Encoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim, hidden, batch_first=True)
    def forward(self, x):  # x: (B,W,E)
        _, h = self.rnn(x)
        return h  # (1,B,H)

class Decoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim + 1, hidden, batch_first=True)  # + prev y
        self.out = nn.Linear(hidden, 1)
    def forward(self, future_feats, h0, y0, teacher=None, tf_prob=0.5):
        B, T, D = future_feats.shape
        y_prev = y0.view(B,1)
        h = h0
        outs = []
        for t in range(T):
            x_t = torch.cat([future_feats[:,t,:], y_prev], dim=1).unsqueeze(1)  # (B,1,D+1)
            o, h = self.rnn(x_t, h)
            y_t = self.out(o[:, -1, :]).squeeze(1)
            outs.append(y_t)
            if (teacher is not None) and (rng.random() < tf_prob):
                y_prev = teacher[:,t].view(B,1)
            else:
                y_prev = y_t.view(B,1)
        return torch.stack(outs, dim=1)  # (B,24)

class Seq2Seq(nn.Module):
    def __init__(self, enc_in, dec_in, hidden):
        super().__init__()
        self.enc = Encoder(enc_in, hidden)
        self.dec = Decoder(dec_in, hidden)
    def forward(self, x_enc, x_dec, y0, y_teacher=None, tf_prob=0.5):
        h = self.enc(x_enc)
        return self.dec(x_dec, h, y0, y_teacher, tf_prob)

enc_in = Xe_tr_n.shape[2]
dec_in = Xd_tr_n.shape[2]
model = Seq2Seq(enc_in, dec_in, HIDDEN).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.HuberLoss()

def huber_weighted(yhat, y, w, delta=1.0):
    d = torch.abs(yhat - y)
    q = torch.clamp(d, max=delta)
    l = 0.5 * q**2 + delta * (d - q)
    return (l * w).mean()

def train_epoch():
    model.train()
    tot = 0.0
    for xenc, xdec, y, y0 in train_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        opt.zero_grad()
        yhat = model(xenc, xdec, y0, y_teacher=y, tf_prob=0.5)
        # time-step weights (B,24)
        w = torch.tensor(HOUR_WEIGHTS, device=y.device).unsqueeze(0).expand_as(y)
        loss = huber_weighted(yhat, y, w)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        tot += loss.item() * xenc.size(0)
    return tot / len(train_loader.dataset)

@torch.no_grad()
def eval_epoch():
    if val_loader is None: return None
    model.eval()
    tot = 0.0
    for xenc, xdec, y, y0 in val_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        yhat = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0)
        w = torch.tensor(HOUR_WEIGHTS, device=y.device).unsqueeze(0).expand_as(y)
        loss = huber_weighted(yhat, y, w)
        tot += loss.item() * xenc.size(0)
    return tot / len(val_loader.dataset)

for ep in range(1, EPOCHS+1):
    tr = train_epoch()
    va = eval_epoch()
    print(f"Epoch {ep:02d}  train_loss={tr:.4f}" + ("" if va is None else f"  val_loss={va:.4f}"))

# ---- evaluate holdout MAE in real units (SAFE) ----
if val_loader is not None:
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xenc, xdec, y, y0 in val_loader:
            xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
            yhat_n = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0).detach().cpu().numpy()
            y_n    = y.detach().cpu().numpy()
            # invert scaling + inverse spike transform
            yhat_t = (yhat_n * y_sd + y_mu)
            yt_t   = (y_n    * y_sd + y_mu)
            yhat = sgn_expm1(yhat_t).reshape(-1)
            yt   = sgn_expm1(yt_t).reshape(-1)
            preds.append(yhat)
            trues.append(yt)
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    mask = np.isfinite(preds) & np.isfinite(trues)
    dropped = int((~mask).sum())
    if dropped:
        print(f"Dropped {dropped} non-finite pairs from holdout scoring.")
    print({"holdout_MAE": float(mean_absolute_error(trues[mask], preds[mask]))})

# ---- optional LightGBM residual booster (train on TRAIN residuals) ----
USE_LGBM_RESIDUAL = True
booster = None
if USE_LGBM_RESIDUAL:
    try:
        import lightgbm as lgb
        # Build a non-shuffled loader to preserve order
        train_loader_eval = torch.utils.data.DataLoader(
            Seq2SeqDataset(Xe_tr_n, Xd_tr_n, Y_tr_n, Y0_tr_n), batch_size=64, shuffle=False)
        base_preds = []
        with torch.no_grad():
            model.eval()
            for xenc, xdec, y, y0 in train_loader_eval:
                xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
                yhat_n = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0).detach().cpu().numpy()
                base_preds.append(yhat_n)
        base_preds = np.concatenate(base_preds, axis=0)  # (N_train, 24)
        # inverse transforms -> true $/MWh
        base_preds_t = (base_preds * y_sd + y_mu)
        base_preds_true = sgn_expm1(base_preds_t)
        y_true_tr = Y_tr  # already true units
        # flatten
        y_base_flat = base_preds_true.reshape(-1)
        y_true_flat = y_true_tr.reshape(-1)
        X_flat = Xd_tr.reshape(-1, Xd_tr.shape[2])  # kept raw decoder features
        mask = np.isfinite(y_base_flat) & np.isfinite(y_true_flat) & np.isfinite(X_flat).all(axis=1)
        X_flat = X_flat[mask]
        y_resid_flat = (y_true_flat - y_base_flat)[mask]
        # Fit booster
        booster = lgb.LGBMRegressor(
            n_estimators=600, learning_rate=0.03, subsample=0.8, colsample_bytree=0.8,
            max_depth=-1, reg_alpha=0.0, reg_lambda=0.0, min_child_samples=20, random_state=42)
        booster.fit(X_flat, y_resid_flat)
        print("Residual booster trained (LightGBM).")
    except Exception as e:
        print(f"LightGBM residual booster skipped: {e}")

# ---- predict requested DELIVERY_DATE ----

# find base row (previous day EOD with price)
prev_day = DELIVERY_DATE - dt.timedelta(days=1)
base_row = df_lags[(pd.to_datetime(df_lags["OperatingDTM"]).dt.date == prev_day) & (df_lags["Interval"]==24) & df_lags["hb_houston"].notna()]
if base_row.empty:
    raise RuntimeError(f"No base EOD price for {prev_day}")
ts0 = pd.to_datetime(base_row.iloc[0]["ts"])
y0  = np.float32(base_row.iloc[0]["hb_houston"])

# encoder window
win = df_lags.set_index("ts").loc[ts0 - pd.Timedelta(hours=W_PAST-1): ts0][enc_cols]
if win.shape[0] != W_PAST or win.isna().any().any():
    raise RuntimeError("Insufficient/NaN history for encoder window.")
Xe_pred = win.values.astype(np.float32)[None, ...]

# decoder future features (24 rows)
frows = df_future[(df_future["delivery_date"]==DELIVERY_DATE)].sort_values("delivery_interval")
if frows.shape[0] != H_FUT:
    raise RuntimeError("Spine missing 24 rows for delivery date.")
Xd_pred_full = frows[dec_future_cols].values.astype(np.float32)[None, ...]

# Apply the same keep mask used in training (drop columns that were all-NaN in train)
if 'dec_keep_mask' in globals():
    Xd_pred = Xd_pred_full[:, :, dec_keep_mask]
else:
    Xd_pred = Xd_pred_full

# impute + standardize pred using TRAIN stats
m = np.isnan(Xd_pred)
if m.any():
    Xd_pred[m] = np.broadcast_to(dec_med, Xd_pred.shape)[m]
Xe_pred_n = apply_standardizer(Xe_pred, enc_mu, enc_sd)
Xd_pred_n = apply_standardizer(Xd_pred, dec_mu, dec_sd)
# compute normalized scalar y0_n using spike-aware transform
y0_t = sgn_log1p(y0)
y0_n = np.float32(((y0_t - y_mu) / y_sd)).ravel()[0]

model.eval()
with torch.no_grad():
    yhat_n = model(torch.from_numpy(Xe_pred_n).to(DEVICE),
                   torch.from_numpy(Xd_pred_n).to(DEVICE),
                   torch.tensor([y0_n], dtype=torch.float32, device=DEVICE),
                   y_teacher=None, tf_prob=0.0).detach().cpu().numpy()[0]
yhat_t = (yhat_n * y_sd + y_mu).reshape(-1)
yhat = sgn_expm1(yhat_t)
# replace any residual NaN with the day's median
if np.isnan(yhat).any():
    day_med = float(np.nanmedian(yhat))
    yhat = np.nan_to_num(yhat, nan=day_med)

forecast = pd.DataFrame({
    "OperatingDTM": [DELIVERY_DATE]*H_FUT,
    "Interval": list(range(1, H_FUT+1)),
    "hb_houston_pred": yhat.tolist()
})
forecast


Epoch 01  train_loss=0.4124  val_loss=0.2501
Epoch 02  train_loss=0.3173  val_loss=0.2876
Epoch 03  train_loss=0.2384  val_loss=0.2097
Epoch 04  train_loss=0.2014  val_loss=0.3906
Epoch 05  train_loss=0.1751  val_loss=0.3983
Epoch 06  train_loss=0.1753  val_loss=0.1780
Epoch 07  train_loss=0.1625  val_loss=0.2344
Epoch 08  train_loss=0.1478  val_loss=0.2333
Epoch 09  train_loss=0.1466  val_loss=0.1654
Epoch 10  train_loss=0.1379  val_loss=0.5173
Epoch 11  train_loss=0.1425  val_loss=0.1813
Epoch 12  train_loss=0.1393  val_loss=0.7863
Epoch 13  train_loss=0.1332  val_loss=0.1645
Epoch 14  train_loss=0.1265  val_loss=0.2795
Epoch 15  train_loss=0.1274  val_loss=0.2342
Epoch 16  train_loss=0.1191  val_loss=0.1505
Epoch 17  train_loss=0.1209  val_loss=0.2534
Epoch 18  train_loss=0.1173  val_loss=0.1583
Epoch 19  train_loss=0.1149  val_loss=0.2554
Epoch 20  train_loss=0.1095  val_loss=0.1776
{'holdout_MAE': 10.248531341552734}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the ov

,OperatingDTM,Interval,hb_houston_pred
0,2025-08-18,1,36.722523
1,2025-08-18,2,37.694660
2,2025-08-18,3,34.816631
3,2025-08-18,4,32.699329
4,2025-08-18,5,32.672554
5,2025-08-18,6,33.808319
6,2025-08-18,7,33.666367
7,2025-08-18,8,30.430222
8,2025-08-18,9,27.215673
9,2025-08-18,10,27.650932


In [24]:
# ==== PyTorch seq2seq with ROBUST preprocessing (median impute + safe standardize + safe MAE) ====
import duckdb, pandas as pd, numpy as np, datetime as dt, math
import torch, torch.nn as nn
from sklearn.metrics import mean_absolute_error

DB = "../ProjectMain/db/data.duckdb"
DELIVERY_DATE = pd.Timestamp("2025-08-18").date()   # change target date if needed
W_PAST = 168
H_FUT = 24
HOLDOUT_DAYS = 60
HIDDEN = 128
EPOCHS = 20
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
rng = np.random.default_rng(42)
torch.manual_seed(42)

# ---------- helpers ----------
def fit_standardizer(X2d):
    """Return (mean,std) with std guarded (==0 -> 1). X2d: (N, F)"""
    mu = np.nanmean(X2d, axis=0)
    sd = np.nanstd(X2d, axis=0)
    sd = np.where(sd <= 1e-8, 1.0, sd)
    return mu, sd

def apply_standardizer(X, mu, sd):
    return (X - mu) / sd

# robust 3D imputer
def impute_inplace_3d(X, med_vec):
    """X: (N, T, F) or (N, W, F); med_vec: (F,). Fills NaNs in-place with per-feature medians."""
    mask = np.isnan(X)
    if mask.any():
        med_broadcast = np.broadcast_to(med_vec, X.shape)
        X[mask] = med_broadcast[mask]

# sanity checker
def assert_finite(name, arr):
    if not np.isfinite(arr).all():
        bad = int(np.isnan(arr).sum() + np.isinf(arr).sum())
        raise RuntimeError(f"{name} has {bad} non-finite values")

# ---------- ensure weather-enhanced views (hist + fcst) exist & helper transforms ----------
con_ensure = duckdb.connect(DB)
# Historical weather enhanced (to backfill when forecast is missing during training)
con_ensure.execute("""
CREATE OR REPLACE VIEW wx_hist_enh AS
WITH agg AS (
  SELECT
    OperatingDTM, "interval" AS Interval,
    (hist_temp_the_woodlands_tx + hist_temp_katy_tx + hist_temp_friendswood_tx + hist_temp_baytown_tx + hist_temp_houston_tx)/5.0 AS temp_avg,
    (hist_hum_the_woodlands_tx  + hist_hum_katy_tx  + hist_hum_friendswood_tx  + hist_hum_baytown_tx  + hist_hum_houston_tx)/5.0  AS hum_avg,
    GREATEST(hist_temp_the_woodlands_tx, hist_temp_katy_tx, hist_temp_friendswood_tx, hist_temp_baytown_tx, hist_temp_houston_tx)
      - LEAST(hist_temp_the_woodlands_tx, hist_temp_katy_tx, hist_temp_friendswood_tx, hist_temp_baytown_tx, hist_temp_houston_tx) AS temp_spread,
    GREATEST(hist_hum_the_woodlands_tx, hist_hum_katy_tx, hist_hum_friendswood_tx, hist_hum_baytown_tx, hist_hum_houston_tx)
      - LEAST(hist_hum_the_woodlands_tx, hist_hum_katy_tx, hist_hum_friendswood_tx, hist_hum_baytown_tx, hist_hum_houston_tx) AS hum_spread
  FROM vw_historical_weather_by_city
)
SELECT
  a.*,
  a.temp_avg - LAG(a.temp_avg) OVER (ORDER BY OperatingDTM, Interval) AS temp_avg_ramp1,
  a.hum_avg  - LAG(a.hum_avg)  OVER (ORDER BY OperatingDTM, Interval) AS hum_avg_ramp1
FROM agg a;
""")
con_ensure.close()

# Signed log transforms for spikes
sgn = np.sign
abs_ = np.abs

def sgn_log1p(y):
    return sgn(y) * np.log1p(abs_(y))

def sgn_expm1(z):
    return sgn(z) * np.expm1(abs_(z))

# Hour-of-day weights (emphasize 17-21 local hours)
HOUR_WEIGHTS = np.ones(24, dtype=np.float32)
# Reweight hours to reduce early overshoot and emphasize true peak
# index 0->H1, ..., 23->H24
HOUR_WEIGHTS[:] = 1.0
HOUR_WEIGHTS[15] = 1.5     # H16
HOUR_WEIGHTS[16:18] = 2.0  # H17-18
HOUR_WEIGHTS[18:20] = 5.0  # H19-20 (peak focus)
HOUR_WEIGHTS[20] = 3.0     # H21

# ---------- pull data ----------
con = duckdb.connect(DB)
df_lags = con.execute("""
SELECT
  ts, OperatingDTM, Interval, hb_houston,
  p_lag1,p_lag2,p_lag3,p_lag6,p_lag12,p_lag24,p_lag48,p_lag72,p_lag168,
  dp1,dp24,p_roll24_mean,p_roll24_std,p_roll72_mean,p_roll168_mean
FROM vw_master_spine_lags
ORDER BY ts
""").df()
df_lags["ts"] = pd.to_datetime(df_lags["ts"])

df_future = con.execute("""
SELECT
  f.OperatingDTM AS delivery_date,
  f.Interval     AS delivery_interval,
  f.hb_houston   AS target,
  -- base future features
  f.wz_southcentral, f.wz_east, f.wz_west, f.wz_northcentral, f.wz_farwest, f.wz_north, f.wz_southern, f.wz_coast,
  f.lz_north, f.lz_west, f.lz_south, f.lz_houston,
  f.cal_hour AS cal_hour_f, f.cal_dow AS cal_dow_f, f.cal_is_weekend AS cal_is_weekend_f,
  f.cal_sin_hour AS cal_sin_hour_f, f.cal_cos_hour AS cal_cos_hour_f,
  f.cal_sin_dow  AS cal_sin_dow_f,  f.cal_cos_dow  AS cal_cos_dow_f,
  -- load-enhanced joins
  l.net_wz, l.net_lz, l.net_wz_ramp1, l.net_lz_ramp1, l.lz_houston_ramp1, l.wz_spread, l.lz_spread,
  -- weather: COALESCE forecast -> historical
  COALESCE(wf.temp_avg,      wh.temp_avg)      AS temp_avg,
  COALESCE(wf.temp_spread,   wh.temp_spread)   AS temp_spread,
  COALESCE(wf.temp_avg_ramp1,wh.temp_avg_ramp1)AS temp_avg_ramp1,
  COALESCE(wf.hum_avg,       wh.hum_avg)       AS hum_avg,
  COALESCE(wf.hum_spread,    wh.hum_spread)    AS hum_spread,
  COALESCE(wf.hum_avg_ramp1, wh.hum_avg_ramp1) AS hum_avg_ramp1,
  CASE WHEN wf.temp_avg IS NULL THEN 1 ELSE 0 END AS is_weather_proxy,
  -- additional ramps & day-over-day deltas to help with peaks (computed over time order)
  (l.net_wz      - LAG(l.net_wz,      3)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_wz_ramp3,
  (l.net_wz      - LAG(l.net_wz,      6)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_wz_ramp6,
  (l.net_lz      - LAG(l.net_lz,      3)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_lz_ramp3,
  (l.net_lz      - LAG(l.net_lz,      6)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_lz_ramp6,
  (l.lz_houston  - LAG(l.lz_houston,  3)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS lz_houston_ramp3,
  (l.lz_houston  - LAG(l.lz_houston,  6)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS lz_houston_ramp6,
  (l.net_wz      - LAG(l.net_wz,     24)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_wz_dod,
  (l.net_lz      - LAG(l.net_lz,     24)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS net_lz_dod,
  (l.lz_houston  - LAG(l.lz_houston, 24)  OVER (ORDER BY f.OperatingDTM, f.Interval)) AS lz_houston_dod
FROM vw_master_spine_ts f
LEFT JOIN load_fcst_enh l
  ON l.OperatingDTM = f.OperatingDTM AND l.Interval = f.Interval
LEFT JOIN wx_fcst_enh  wf
  ON wf.OperatingDTM = f.OperatingDTM AND wf.Interval = f.Interval
LEFT JOIN wx_hist_enh  wh
  ON wh.OperatingDTM = f.OperatingDTM AND wh.Interval = f.Interval
ORDER BY f.OperatingDTM, f.Interval
""").df()
con.close()

df_future["delivery_date"] = pd.to_datetime(df_future["delivery_date"]).dt.date
df_future["delivery_interval"] = df_future["delivery_interval"].astype(int)

# ---------- assemble samples (day-ahead) ----------
enc_cols = [
    "hb_houston","p_lag1","p_lag2","p_lag3","p_lag6","p_lag12","p_lag24","p_lag48","p_lag72","p_lag168",
    "dp1","dp24","p_roll24_mean","p_roll24_std","p_roll72_mean","p_roll168_mean"
]
dec_future_cols = [
    "cal_hour_f","cal_dow_f","cal_is_weekend_f","cal_sin_hour_f","cal_cos_hour_f","cal_sin_dow_f","cal_cos_dow_f",
    "net_wz","net_lz","net_wz_ramp1","net_lz_ramp1","lz_houston_ramp1","wz_spread","lz_spread",
    "temp_avg","temp_spread","temp_avg_ramp1","hum_avg","hum_spread","hum_avg_ramp1",
    # NEW: multi-horizon ramps + day-over-day deltas
    "net_wz_ramp3","net_wz_ramp6","net_lz_ramp3","net_lz_ramp6",
    "lz_houston_ramp3","lz_houston_ramp6","net_wz_dod","net_lz_dod","lz_houston_dod",
    # Identify when weather came from historical backfill instead of forecast
    "is_weather_proxy"
]

# base rows = end-of-day with price
base_rows = df_lags[(df_lags["hb_houston"].notna()) & (df_lags["Interval"]==24)][["ts","OperatingDTM","hb_houston"]].copy()
base_rows["base_date"] = pd.to_datetime(base_rows["OperatingDTM"]).dt.date
base_rows["delivery_date"] = base_rows["base_date"] + dt.timedelta(days=1)

# only use delivery days with 24 actuals for training/val
has24 = df_future.groupby("delivery_date")["target"].apply(lambda s: s.notna().sum()==24)
valid_delivery_dates = set(has24[has24].index)

df_lags_idx = df_lags.set_index("ts").sort_index()

samples = []
for _, r in base_rows.iterrows():
    ddate = r["delivery_date"]
    ts0 = r["ts"]
    # encoder window
    start = ts0 - pd.Timedelta(hours=W_PAST-1)
    enc_df = df_lags_idx.loc[start:ts0][enc_cols]
    if enc_df.shape[0] != W_PAST: 
        continue
    if enc_df.isna().any().any():
        continue  # strict: skip any encoder NaNs
    # decoder future rows
    fdf = df_future[(df_future["delivery_date"]==ddate)].sort_values("delivery_interval")
    if fdf.shape[0] != H_FUT:
        continue
    X_enc = enc_df.values.astype(np.float32)
    X_dec = fdf[dec_future_cols].values.astype(np.float32)
    y     = fdf["target"].values.astype(np.float32)
    y0    = np.float32(r["hb_houston"])
    samples.append((X_enc, X_dec, y, y0, ddate))

if len(samples) == 0:
    raise RuntimeError("No samples assembled; check data coverage or reduce W_PAST.")

# split by date
dates = sorted({s[4] for s in samples})
cut_date = dates[-1] - dt.timedelta(days=HOLDOUT_DAYS) if len(dates)>HOLDOUT_DAYS else dates[0]
train_idx = [i for i,s in enumerate(samples) if s[4] <= cut_date and s[4] in valid_delivery_dates]
val_idx   = [i for i,s in enumerate(samples) if s[4] >  cut_date and s[4] in valid_delivery_dates]

def stack(idxs):
    Xe = np.stack([samples[i][0] for i in idxs], axis=0)
    Xd = np.stack([samples[i][1] for i in idxs], axis=0)
    Y  = np.stack([samples[i][2] for i in idxs], axis=0)
    Y0 = np.stack([samples[i][3] for i in idxs], axis=0)
    return Xe, Xd, Y, Y0

Xe_tr, Xd_tr, Y_tr, Y0_tr = stack(train_idx)
Xe_va, Xd_va, Y_va, Y0_va = (None, None, None, None)
if len(val_idx) > 0:
    Xe_va, Xd_va, Y_va, Y0_va = stack(val_idx)

# ---------- diagnose + drop all-NaN decoder features, then impute (robust) ----------
# Flatten decoder train to 2D to inspect per-feature missingness
F_orig = Xd_tr.shape[2]
Xd_tr_2d_raw = Xd_tr.reshape(-1, F_orig)
nan_counts = np.isnan(Xd_tr_2d_raw).sum(axis=0)
all_nan_cols = nan_counts == Xd_tr_2d_raw.shape[0]

if all_nan_cols.any():
    dropped_cols = [dec_future_cols[i] for i,flag in enumerate(all_nan_cols) if flag]
    print(f"Dropping {int(all_nan_cols.sum())} decoder feature(s) with all-NaN in TRAIN: {dropped_cols}")
else:
    dropped_cols = []

# Keep mask to be reused at prediction time
dec_keep_mask = ~all_nan_cols

# Apply mask to decoder tensors
Xd_tr = Xd_tr[:, :, dec_keep_mask]
if Xe_va is not None:
    Xd_va = Xd_va[:, :, dec_keep_mask]

# Recompute medians on KEPT features
dec_med = np.nanmedian(Xd_tr.reshape(-1, Xd_tr.shape[2]), axis=0)
# Guard: if a median is still NaN for some reason, set to 0
dec_med = np.where(np.isnan(dec_med), 0.0, dec_med)

# Encoder median (rare but safe)
enc_med = np.nanmedian(Xe_tr.reshape(-1, Xe_tr.shape[2]), axis=0)
enc_med = np.where(np.isnan(enc_med), 0.0, enc_med)

# Impute in-place (3D-safe)
impute_inplace_3d(Xe_tr, enc_med)
impute_inplace_3d(Xd_tr, dec_med)
if Xe_va is not None:
    impute_inplace_3d(Xe_va, enc_med)
    impute_inplace_3d(Xd_va, dec_med)

# Targets should be complete for training/val; assert
assert np.isfinite(Y_tr).all(), "Y_tr has non-finite values"
if Y_va is not None:
    assert np.isfinite(Y_va).all(), "Y_va has non-finite values"

# ---------- safe standardization ----------
enc_mu, enc_sd = fit_standardizer(Xe_tr.reshape(-1, Xe_tr.shape[2]))
dec_mu, dec_sd = fit_standardizer(Xd_tr.reshape(-1, Xd_tr.shape[2]))
# ---- target transform (spike-aware) then standardize ----
Y_tr_t  = sgn_log1p(Y_tr)
y_mu, y_sd = fit_standardizer(Y_tr_t.reshape(-1,1))
y_mu = float(np.atleast_1d(y_mu)[0]); y_sd = float(np.atleast_1d(y_sd)[0])

Xe_tr_n = apply_standardizer(Xe_tr, enc_mu, enc_sd)
Xd_tr_n = apply_standardizer(Xd_tr, dec_mu, dec_sd)
Y_tr_n  = apply_standardizer(Y_tr_t, y_mu, y_sd).reshape(Y_tr.shape)
Y0_tr_t = sgn_log1p(Y0_tr.reshape(-1,1)).reshape(-1,1)
Y0_tr_n = apply_standardizer(Y0_tr_t, y_mu, y_sd).reshape(-1)

if Xe_va is not None:
    Xe_va_n = apply_standardizer(Xe_va, enc_mu, enc_sd)
    Xd_va_n = apply_standardizer(Xd_va, dec_mu, dec_sd)
    Y_va_t  = sgn_log1p(Y_va)
    Y_va_n  = apply_standardizer(Y_va_t,  y_mu,  y_sd).reshape(Y_va.shape)
    Y0_va_t = sgn_log1p(Y0_va.reshape(-1,1))
    Y0_va_n = apply_standardizer(Y0_va_t, y_mu, y_sd).reshape(-1)


# Final safety: ensure everything is finite
assert_finite("Xe_tr_n", Xe_tr_n)
assert_finite("Xd_tr_n", Xd_tr_n)
assert_finite("Y_tr_n",  Y_tr_n)
assert_finite("Y0_tr_n", Y0_tr_n)
if Xe_va is not None:
    assert_finite("Xe_va_n", Xe_va_n)
    assert_finite("Xd_va_n", Xd_va_n)
    assert_finite("Y_va_n",  Y_va_n)
    assert_finite("Y0_va_n", Y0_va_n)

# ---------- torch datasets ----------
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self, Xe, Xd, Y, Y0):
        self.Xe = torch.from_numpy(Xe).float()
        self.Xd = torch.from_numpy(Xd).float()
        self.Y  = torch.from_numpy(Y).float()
        self.Y0 = torch.from_numpy(Y0).float()
    def __len__(self): return self.Xe.shape[0]
    def __getitem__(self, i): return self.Xe[i], self.Xd[i], self.Y[i], self.Y0[i]

bs = 32
train_loader = torch.utils.data.DataLoader(Seq2SeqDataset(Xe_tr_n, Xd_tr_n, Y_tr_n, Y0_tr_n), batch_size=bs, shuffle=True)
val_loader = None if Xe_va is None else torch.utils.data.DataLoader(Seq2SeqDataset(Xe_va_n, Xd_va_n, Y_va_n, Y0_va_n), batch_size=bs, shuffle=False)

# ---------- model ----------
class Encoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim, hidden, batch_first=True)
    def forward(self, x):  # x: (B,W,E)
        _, h = self.rnn(x)
        return h  # (1,B,H)

class Decoder(nn.Module):
    def __init__(self, in_dim, hidden):
        super().__init__()
        self.rnn = nn.GRU(in_dim + 1, hidden, batch_first=True)  # + prev y
        self.out = nn.Linear(hidden, 1)
    def forward(self, future_feats, h0, y0, teacher=None, tf_prob=0.5):
        B, T, D = future_feats.shape
        y_prev = y0.view(B,1)
        h = h0
        outs = []
        for t in range(T):
            x_t = torch.cat([future_feats[:,t,:], y_prev], dim=1).unsqueeze(1)  # (B,1,D+1)
            o, h = self.rnn(x_t, h)
            y_t = self.out(o[:, -1, :]).squeeze(1)
            outs.append(y_t)
            if (teacher is not None) and (rng.random() < tf_prob):
                y_prev = teacher[:,t].view(B,1)
            else:
                y_prev = y_t.view(B,1)
        return torch.stack(outs, dim=1)  # (B,24)

class Seq2Seq(nn.Module):
    def __init__(self, enc_in, dec_in, hidden):
        super().__init__()
        self.enc = Encoder(enc_in, hidden)
        self.dec = Decoder(dec_in, hidden)
    def forward(self, x_enc, x_dec, y0, y_teacher=None, tf_prob=0.5):
        h = self.enc(x_enc)
        return self.dec(x_dec, h, y0, y_teacher, tf_prob)

enc_in = Xe_tr_n.shape[2]
dec_in = Xd_tr_n.shape[2]
model = Seq2Seq(enc_in, dec_in, HIDDEN).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.HuberLoss()

def huber_weighted(yhat, y, w, delta=1.0):
    d = torch.abs(yhat - y)
    q = torch.clamp(d, max=delta)
    l = 0.5 * q**2 + delta * (d - q)
    return (l * w).mean()

def train_epoch():
    model.train()
    tot = 0.0
    for xenc, xdec, y, y0 in train_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        opt.zero_grad()
        yhat = model(xenc, xdec, y0, y_teacher=y, tf_prob=0.5)
        # time-step weights (B,24)
        w = torch.tensor(HOUR_WEIGHTS, device=y.device).unsqueeze(0).expand_as(y)
        loss = huber_weighted(yhat, y, w)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        tot += loss.item() * xenc.size(0)
    return tot / len(train_loader.dataset)

@torch.no_grad()
def eval_epoch():
    if val_loader is None: return None
    model.eval()
    tot = 0.0
    for xenc, xdec, y, y0 in val_loader:
        xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
        yhat = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0)
        w = torch.tensor(HOUR_WEIGHTS, device=y.device).unsqueeze(0).expand_as(y)
        loss = huber_weighted(yhat, y, w)
        tot += loss.item() * xenc.size(0)
    return tot / len(val_loader.dataset)

for ep in range(1, EPOCHS+1):
    tr = train_epoch()
    va = eval_epoch()
    print(f"Epoch {ep:02d}  train_loss={tr:.4f}" + ("" if va is None else f"  val_loss={va:.4f}"))

# ---- evaluate holdout MAE in real units (SAFE) ----
if val_loader is not None:
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xenc, xdec, y, y0 in val_loader:
            xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
            yhat_n = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0).detach().cpu().numpy()
            y_n    = y.detach().cpu().numpy()
            # invert scaling + inverse spike transform
            yhat_t = (yhat_n * y_sd + y_mu)
            yt_t   = (y_n    * y_sd + y_mu)
            yhat = sgn_expm1(yhat_t).reshape(-1)
            yt   = sgn_expm1(yt_t).reshape(-1)
            preds.append(yhat)
            trues.append(yt)
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    mask = np.isfinite(preds) & np.isfinite(trues)
    dropped = int((~mask).sum())
    if dropped:
        print(f"Dropped {dropped} non-finite pairs from holdout scoring.")
    print({"holdout_MAE": float(mean_absolute_error(trues[mask], preds[mask]))})

# ---- optional LightGBM residual booster (train on TRAIN residuals) ----
USE_LGBM_RESIDUAL = True
booster = None
if USE_LGBM_RESIDUAL:
    try:
        import lightgbm as lgb
        # Build a non-shuffled loader to preserve order
        train_loader_eval = torch.utils.data.DataLoader(
            Seq2SeqDataset(Xe_tr_n, Xd_tr_n, Y_tr_n, Y0_tr_n), batch_size=64, shuffle=False)
        base_preds = []
        with torch.no_grad():
            model.eval()
            for xenc, xdec, y, y0 in train_loader_eval:
                xenc, xdec, y, y0 = xenc.to(DEVICE), xdec.to(DEVICE), y.to(DEVICE), y0.to(DEVICE)
                yhat_n = model(xenc, xdec, y0, y_teacher=None, tf_prob=0.0).detach().cpu().numpy()
                base_preds.append(yhat_n)
        base_preds = np.concatenate(base_preds, axis=0)  # (N_train, 24)
        # inverse transforms -> true $/MWh
        base_preds_t = (base_preds * y_sd + y_mu)
        base_preds_true = sgn_expm1(base_preds_t)
        y_true_tr = Y_tr  # already true units
        # flatten
        y_base_flat = base_preds_true.reshape(-1)
        y_true_flat = y_true_tr.reshape(-1)
        X_flat = Xd_tr.reshape(-1, Xd_tr.shape[2])  # kept raw decoder features
        mask = np.isfinite(y_base_flat) & np.isfinite(y_true_flat) & np.isfinite(X_flat).all(axis=1)
        X_flat = X_flat[mask]
        y_resid_flat = (y_true_flat - y_base_flat)[mask]
        # Fit booster
        booster = lgb.LGBMRegressor(
            n_estimators=600, learning_rate=0.03, subsample=0.8, colsample_bytree=0.8,
            max_depth=-1, reg_alpha=0.0, reg_lambda=0.0, min_child_samples=20, random_state=42)
        booster.fit(X_flat, y_resid_flat)
        print("Residual booster trained (LightGBM).")
    except Exception as e:
        print(f"LightGBM residual booster skipped: {e}")

# ---- predict requested DELIVERY_DATE ----

# find base row (previous day EOD with price)
prev_day = DELIVERY_DATE - dt.timedelta(days=1)
base_row = df_lags[(pd.to_datetime(df_lags["OperatingDTM"]).dt.date == prev_day) & (df_lags["Interval"]==24) & df_lags["hb_houston"].notna()]
if base_row.empty:
    raise RuntimeError(f"No base EOD price for {prev_day}")
ts0 = pd.to_datetime(base_row.iloc[0]["ts"])
y0  = np.float32(base_row.iloc[0]["hb_houston"])

# encoder window
win = df_lags.set_index("ts").loc[ts0 - pd.Timedelta(hours=W_PAST-1): ts0][enc_cols]
if win.shape[0] != W_PAST or win.isna().any().any():
    raise RuntimeError("Insufficient/NaN history for encoder window.")
Xe_pred = win.values.astype(np.float32)[None, ...]

# decoder future features (24 rows)
frows = df_future[(df_future["delivery_date"]==DELIVERY_DATE)].sort_values("delivery_interval")
if frows.shape[0] != H_FUT:
    raise RuntimeError("Spine missing 24 rows for delivery date.")
Xd_pred_full = frows[dec_future_cols].values.astype(np.float32)[None, ...]

# Apply the same keep mask used in training (drop columns that were all-NaN in train)
if 'dec_keep_mask' in globals():
    Xd_pred = Xd_pred_full[:, :, dec_keep_mask]
else:
    Xd_pred = Xd_pred_full

# impute + standardize pred using TRAIN stats
m = np.isnan(Xd_pred)
if m.any():
    Xd_pred[m] = np.broadcast_to(dec_med, Xd_pred.shape)[m]
Xe_pred_n = apply_standardizer(Xe_pred, enc_mu, enc_sd)
Xd_pred_n = apply_standardizer(Xd_pred, dec_mu, dec_sd)
# compute normalized scalar y0_n using spike-aware transform
y0_t = sgn_log1p(y0)
y0_n = np.float32(((y0_t - y_mu) / y_sd)).ravel()[0]

model.eval()
with torch.no_grad():
    yhat_n = model(torch.from_numpy(Xe_pred_n).to(DEVICE),
                   torch.from_numpy(Xd_pred_n).to(DEVICE),
                   torch.tensor([y0_n], dtype=torch.float32, device=DEVICE),
                   y_teacher=None, tf_prob=0.0).detach().cpu().numpy()[0]
yhat_t = (yhat_n * y_sd + y_mu).reshape(-1)
yhat = sgn_expm1(yhat_t)
# --- optional: add residual booster only on peak hours (H18–H21) ---
if booster is not None:
    Xd_pred_kept = Xd_pred.copy()  # (1,24,F_kept)
    resid_pred = booster.predict(Xd_pred_kept.reshape(24, Xd_pred_kept.shape[2]))
    hour_mask = np.zeros(24, dtype=np.float32)
    hour_mask[17:21] = 1.0  # apply only to intervals 18,19,20,21
    yhat = yhat + resid_pred * hour_mask

# replace any residual NaN with the day's median
if np.isnan(yhat).any():
    day_med = float(np.nanmedian(yhat))
    yhat = np.nan_to_num(yhat, nan=day_med)

forecast = pd.DataFrame({
    "OperatingDTM": [DELIVERY_DATE]*H_FUT,
    "Interval": list(range(1, H_FUT+1)),
    "hb_houston_pred": yhat.tolist()
})
forecast


Epoch 01  train_loss=0.4124  val_loss=0.2501
Epoch 02  train_loss=0.3173  val_loss=0.2876
Epoch 03  train_loss=0.2384  val_loss=0.2097
Epoch 04  train_loss=0.2014  val_loss=0.3906
Epoch 05  train_loss=0.1751  val_loss=0.3983
Epoch 06  train_loss=0.1753  val_loss=0.1780
Epoch 07  train_loss=0.1625  val_loss=0.2344
Epoch 08  train_loss=0.1478  val_loss=0.2333
Epoch 09  train_loss=0.1466  val_loss=0.1654
Epoch 10  train_loss=0.1379  val_loss=0.5173
Epoch 11  train_loss=0.1425  val_loss=0.1813
Epoch 12  train_loss=0.1393  val_loss=0.7863
Epoch 13  train_loss=0.1332  val_loss=0.1645
Epoch 14  train_loss=0.1265  val_loss=0.2795
Epoch 15  train_loss=0.1274  val_loss=0.2342
Epoch 16  train_loss=0.1191  val_loss=0.1505
Epoch 17  train_loss=0.1209  val_loss=0.2534
Epoch 18  train_loss=0.1173  val_loss=0.1583
Epoch 19  train_loss=0.1149  val_loss=0.2554
Epoch 20  train_loss=0.1095  val_loss=0.1776
{'holdout_MAE': 10.248531341552734}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the ov

c:\Users\db\DrewBCapstone\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


,OperatingDTM,Interval,hb_houston_pred
0,2025-08-18,1,36.722523
1,2025-08-18,2,37.694660
2,2025-08-18,3,34.816631
3,2025-08-18,4,32.699329
4,2025-08-18,5,32.672554
5,2025-08-18,6,33.808319
6,2025-08-18,7,33.666367
7,2025-08-18,8,30.430222
8,2025-08-18,9,27.215673
9,2025-08-18,10,27.650932


In [ ]:

import duckdb

con = duckdb.connect("../ProjectMain/db/data.duckdb")

df = con.execute("""

select * from vw_forecast_weather_by_city order by 1 desc
                 --WHERE OperatingDTM >= X and OperatingDTM <= Y
                 ;

""").fetchdf()
print(df)
print(df.info())
con.close()


    OperatingDTM           timestamp  interval  temp_the_woodlands_tx  \
0     2025-08-24 2025-08-24 01:00:00         1                   30.1   
1     2025-08-24 2025-08-24 02:00:00         2                   29.5   
2     2025-08-24 2025-08-24 03:00:00         3                   28.9   
3     2025-08-24 2025-08-24 04:00:00         4                   28.4   
4     2025-08-24 2025-08-24 05:00:00         5                   28.0   
..           ...                 ...       ...                    ...   
187   2025-08-17 2025-08-17 20:00:00        20                   35.5   
188   2025-08-17 2025-08-17 21:00:00        21                   33.8   
189   2025-08-17 2025-08-17 22:00:00        22                   32.3   
190   2025-08-17 2025-08-17 23:00:00        23                   30.6   
191   2025-08-17 2025-08-18 00:00:00        24                   29.6   

     hum_the_woodlands_tx  wind_the_woodlands_tx  precip_the_woodlands_tx  \
0                    60.0                   2.